In [1]:
# This produces the dataframe for QB

In [2]:
## Notes on the NFL Library ##
# the NFL python library seem to not work on Tuesday probably due to updates (not confirmed)
# unbalanced dataframe - pfr stats start at 2018; all other stats go back to 2017

In [3]:
## Notes: on the fantasypros data
# - I was unable to validate hash rows for the redzone data  

In [4]:
## Required installations
# 

In [5]:
## REQUIRED ACTIONS - Include in a README doc ## 
# modify the season start date in the 'get_current_week' function
# modify the number of weeks if the NFL adds regular season games to the schedule

In [6]:
# import the libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import glob
from IPython.display import display, HTML
from datetime import datetime
import nfl_data_py as nfl
import os
import re
import time
from random import sample, uniform, seed
import io
from rapidfuzz import fuzz, process
import numpy as np
import hashlib
import shutil

In [7]:
# Set Pandas options to display all columns in a single row without wrapping
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [8]:
# Function to calculate the current week of the NFL season
def get_current_week():
    current_date = datetime.now()
    season_start_date = datetime(2024, 9, 4)  # Update for the season start
    current_week = ((current_date - season_start_date).days // 7) + 1
    return current_week

# Define the current NFL year, week, and season type
current_year = datetime.now().year
current_week = get_current_week()
seasontype = 2 if current_week <= 18 else 3  # Regular season or playoffs

In [9]:
### Begin: Python NFL Library Dataframe ###

In [10]:
# define the years to pull
# nfl.import_weekly_data(years, columns, downcast)
def get_year_range(current_year, current_week, start_year=2017):
    if current_week <= 18:  # Regular season
        return list(range(start_year, current_year + 1))
    else:  # Playoffs
        return list(range(start_year, current_year))

# Use the function
years = get_year_range(current_year, current_week)

In [11]:
# define the base columns. 
base_columns = [
    'season', 'season_type', 'week', 'player_id', 'player_name', 
    'position', 'position_group', 'recent_team'
]

In [12]:
# Import the player IDs from nfl.import_ids() - without parameters
ids_data = nfl.import_ids()

# Drop the unnecessary columns
columns_to_drop = [
    'position', 'team', 'birthdate', 'age', 'draft_year', 
    'draft_round', 'draft_pick', 'draft_ovr', 'twitter_username', 
    'height', 'weight', 'college', 'db_season'
]
ids_data = ids_data.drop(columns=columns_to_drop, errors='ignore')

# Display the resulting dataframe for review
# print(f"Columns after dropping unnecessary ones: {ids_data.columns.tolist()}")
# display(ids_data)

In [13]:
# import the weekly data from nfl.import_weekly_data(years, columns, downcast)
weekly_data = nfl.import_weekly_data(
    years=years,
    columns=base_columns
)

# display(weekly_data)

Downcasting floats.


In [14]:
## Output: a dataframe of ALL NFL athletes info and ids since 2017

# Merge the two dataframes on 'player_id' and 'gsis_id'
# Align column names for merging
ids_data = ids_data.rename(columns={'gsis_id': 'player_id'})  
id_dataframe = pd.merge(weekly_data, ids_data, on='player_id', how='inner')

# Assign the resulting dataframe to a variable
all_players_id_data = id_dataframe

# Display the resulting ID dataframe
# display(all_players_id_data)

In [15]:
## Output: a dataframe of NFL QB info and ids since 2017
# Extract QB data from the dataframe
# Create a new dataframe with only quarterbacks
quarterback_ids = all_players_id_data[all_players_id_data['position'] == 'QB']

# Display the resulting dataframe for review
print(f"Shape of merged dataframe: {quarterback_ids.shape}")

# Display the resulting dataframe for review
# display(quarterback_ids)

Shape of merged dataframe: (5267, 29)


In [16]:
## Output: a dataframe of NFL QB info, ids, and stats since 2017

# QB-specific columns (passing-related)
qb_specific_columns = [
    'completions', 'attempts', 'passing_yards', 'passing_tds', 'rushing_yards',
    'interceptions', 'sacks', 'sack_yards', 'sack_fumbles', 
    'sack_fumbles_lost', 'passing_air_yards', 
    'passing_yards_after_catch', 'passing_first_downs', 
    'passing_epa', 'passing_2pt_conversions', 'pacr', 'dakota'
]

# Pull QB-specific columns from weekly data
qb_stats = nfl.import_weekly_data(
    years=years,
    columns=['player_id', 'season', 'week'] + qb_specific_columns  # Include keys for merging
)

# Merge QB-specific stats with quarterback_ids
qb_ids_weekly_stats = pd.merge(
    quarterback_ids,
    qb_stats,
    on=['player_id', 'season', 'week'],  # Ensure correct alignment
    how='inner'
)

# Display the resulting dataframe for review
print(f"Shape of merged dataframe: {qb_ids_weekly_stats.shape}")

# Row integrity check
print(
    f"Row count matches: {qb_ids_weekly_stats.shape[0] == quarterback_ids.shape[0]}"
)

# Display the dataframe
display(qb_ids_weekly_stats)

# ***csv file***
# qb_ids_weekly_stats.to_csv('qb_ids_weekly_stats.csv', index=False)

Downcasting floats.
Shape of merged dataframe: (5267, 46)
Row count matches: True


,season,season_type,week,player_id,player_name,position,position_group,recent_team,mfl_id,sportradar_id,fantasypros_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,completions,attempts,passing_yards,passing_tds,rushing_yards,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota
0,2017,REG,1,00-0019596,T.Brady,QB,QB,NE,5848,41c44740-d0f6-44ab-8347-3b5d515e5ecf,9603.0,698.0,167.0,tombrady/2504211,2330.0,5228.0,309.0,187741.0,BradTo00,tom-brady-1,1350.0,1163.0,NaN,5228.0,25347.0,4314.0,25347.0,Tom Brady,tom brady,16,36,267.0,0,0.0,0.0,3.0,20.0,0,0,567.0,112.0,11.0,1.753902,0,0.470899,0.042110
1,2017,REG,2,00-0019596,T.Brady,QB,QB,NE,5848,41c44740-d0f6-44ab-8347-3b5d515e5ecf,9603.0,698.0,167.0,tombrady/2504211,2330.0,5228.0,309.0,187741.0,BradTo00,tom-brady-1,1350.0,1163.0,NaN,5228.0,25347.0,4314.0,25347.0,Tom Brady,tom brady,30,39,447.0,3,9.0,0.0,2.0,11.0,0,0,281.0,237.0,19.0,25.089682,0,1.590747,0.304464
2,2017,REG,3,00-0019596,T.Brady,QB,QB,NE,5848,41c44740-d0f6-44ab-8347-3b5d515e5ecf,9603.0,698.0,167.0,tombrady/2504211,2330.0,5228.0,309.0,187741.0,BradTo00,tom-brady-1,1350.0,1163.0,NaN,5228.0,25347.0,4314.0,25347.0,Tom Brady,tom brady,25,35,378.0,5,6.0,0.0,5.0,41.0,3,1,415.0,107.0,16.0,10.521861,1,0.910843,0.240395
3,2017,REG,4,00-0019596,T.Brady,QB,QB,NE,5848,41c44740-d0f6-44ab-8347-3b5d515e5ecf,9603.0,698.0,167.0,tombrady/2504211,2330.0,5228.0,309.0,187741.0,BradTo00,tom-brady-1,1350.0,1163.0,NaN,5228.0,25347.0,4314.0,25347.0,Tom Brady,tom brady,32,45,307.0,2,2.0,0.0,3.0,14.0,0,0,341.0,132.0,18.0,9.758528,0,0.900293,0.149092
4,2017,REG,5,00-0019596,T.Brady,QB,QB,NE,5848,41c44740-d0f6-44ab-8347-3b5d515e5ecf,9603.0,698.0,167.0,tombrady/2504211,2330.0,5228.0,309.0,187741.0,BradTo00,tom-brady-1,1350.0,1163.0,NaN,5228.0,25347.0,4314.0,25347.0,Tom Brady,tom brady,30,40,303.0,1,5.0,1.0,3.0,14.0,1,1,248.0,130.0,15.0,2.653190,0,1.221774,0.115276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5262,2024,REG,14,00-0039918,C.Williams,QB,QB,CHI,16579,49d4b627-3f19-49a8-ae33-f539e0fb1f88,23084.0,NaN,11560.0,NaN,4431611.0,40900.0,NaN,26695775.0,WillCa03,caleb-williams-3,17695.0,NaN,1550.0,40900.0,0.0,NaN,1286445.0,Caleb Williams,caleb williams,17,23,134.0,2,27.0,0.0,7.0,46.0,1,1,137.0,69.0,9.0,-6.575400,0,0.978102,0.043877
5263,2024,REG,15,00-0039918,C.Williams,QB,QB,CHI,16579,49d4b627-3f19-49a8-ae33-f539e0fb1f88,23084.0,NaN,11560.0,NaN,4431611.0,40900.0,NaN,26695775.0,WillCa03,caleb-williams-3,17695.0,NaN,1550.0,40900.0,0.0,NaN,1286445.0,Caleb Williams,caleb williams,18,31,191.0,1,3.0,0.0,2.0,25.0,1,1,235.0,116.0,8.0,-3.401174,0,0.812766,-0.006490
5264,2024,REG,16,00-0039918,C.Williams,QB,QB,CHI,16579,49d4b627-3f19-49a8-ae33-f539e0fb1f88,23084.0,NaN,11560.0,NaN,4431611.0,40900.0,NaN,26695775.0,WillCa03,caleb-williams-3,17695.0,NaN,1550.0,40900.0,0.0,NaN,1286445.0,Caleb Williams,caleb williams,26,40,334.0,2,34.0,0.0,2.0,11.0,0,0,365.0,167.0,15.0,16.464520,0,0.915069,0.163077
5265,2024,REG,17,00-0039918,C.Williams,QB,QB,CHI,16579,49d4b627-3f19-49a8-ae33-f539e0fb1f88,23084.0,NaN,11560.0,NaN,4431611.0,40900.0,NaN,26695775.0,WillCa03,caleb-williams-3,17695.0,NaN,1550.0,40900.0,0.0,NaN,1286445.0,Caleb Williams,caleb williams,16,28,122.0,0,37.0,1.0,7.0,46.0,1,0,202.0,108.0,7.0,-10.225746,0,0.603960,-0.000481


In [17]:
# Output: imports the NFL next-generation stats for quarterbacks from the nfl python library

# Import the next-generation stats (NGS) for passing
# Note: NGS starts at week 0 (previous season totals) - these rows are excluded

# Pull NGS passing data for the specified years
ngs_qb_df = nfl.import_ngs_data('passing', years)

# Exclude rows where 'week' == 0 and filter for 'QB' position
ngs_qb_df = ngs_qb_df[(ngs_qb_df['week'] != 0) & (ngs_qb_df['player_position'] == 'QB')]

# Drop unnecessary columns (already in the nfl python baseline dataframe)
ngs_qb_df = ngs_qb_df.drop(
    columns=['season_type', 'player_position', 'player_jersey_number',
             'attempts','completions','interceptions'], errors='ignore'
)

# Display the resulting dataframe
print(f"Shape of NGS QB DataFrame after dropping columns: {ngs_qb_df.shape}")
display(ngs_qb_df)

# ***csv file***
# ngs_qb_df.to_csv('ngs_qb_df.csv', index=False)

Shape of NGS QB DataFrame after dropping columns: (4429, 23)


,season,week,player_display_name,team_abbr,avg_time_to_throw,avg_completed_air_yards,avg_intended_air_yards,avg_air_yards_differential,aggressiveness,max_completed_air_distance,avg_air_yards_to_sticks,pass_yards,pass_touchdowns,passer_rating,completion_percentage,expected_completion_percentage,completion_percentage_above_expectation,avg_air_distance,max_air_distance,player_gsis_id,player_first_name,player_last_name,player_short_name
614,2017,1,Russell Wilson,SEA,3.198296,7.710000,10.460000,-2.750000,18.518519,46.771380,2.460000,158,0,69.675926,51.851852,55.170944,-3.319092,23.691973,51.569332,00-0029263,Russell,Wilson,R.Wilson
615,2017,1,Scott Tolzien,IND,2.516167,5.704444,9.126111,-3.421667,27.777778,35.658975,0.681667,128,0,33.796296,50.000000,60.038921,-10.038921,22.228064,39.893389,00-0028595,Scott,Tolzien,S.Tolzien
616,2017,1,Josh McCown,NYJ,2.372103,2.592917,4.661351,-2.068435,10.256410,28.385873,-4.095405,187,0,56.250000,66.666667,69.385185,-2.718518,17.795295,36.042798,00-0021206,Joshua,McCown,J.McCown
617,2017,1,Alex Smith,KC,3.023914,5.651429,7.390571,-1.739143,14.285714,51.773205,-0.580857,368,4,148.571429,80.000000,70.111395,9.888605,18.876969,51.773205,00-0023436,Alexander,Smith,A.Smith
618,2017,1,Trevor Siemian,DEN,2.264107,8.797059,7.482500,1.314559,10.714286,38.528541,-0.374643,219,2,94.196429,60.714286,60.348870,0.365416,21.617479,38.528541,00-0032156,Trevor,Siemian,T.Siemian
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5323,2024,21,Patrick Mahomes,KC,2.594154,5.234444,6.793200,-1.558756,7.692308,36.393237,-1.926800,245,1,111.858974,69.230769,70.506506,-1.275737,19.948697,42.749331,00-0033873,Patrick,Mahomes,P.Mahomes
5324,2024,21,Jayden Daniels,WAS,2.709000,4.764138,7.153043,-2.388906,12.500000,30.169495,-1.020870,255,1,72.829861,60.416667,68.881243,-8.464577,21.308467,44.512399,00-0039910,Jayden,Daniels,J.Daniels
5325,2024,21,Josh Allen,BUF,2.918647,4.970000,7.555806,-2.585806,11.764706,44.509533,-0.960323,237,2,104.656863,64.705882,65.173744,-0.467862,20.560432,44.509533,00-0034857,Joshua,Allen,J.Allen
5326,2024,23,Patrick Mahomes,KC,3.067875,8.114286,7.977813,0.136473,15.625000,64.377967,-0.897188,257,3,95.442708,65.625000,70.818832,-5.193832,20.038168,64.377967,00-0033873,Patrick,Mahomes,P.Mahomes


In [18]:
# Output: a dataframe of NFL QB info, ids, weekly stats, and next-gen stats since 2017

# Joins qb_ids_weekly_stats dataframe with ngs_qb_df using the keys 'player_id' and 'player_gsis_id'
# Merge qb_ids_weekly_stats with ngs_qb_df using a left join
qb_ids_ngs_weekly_stats = pd.merge(
    qb_ids_weekly_stats,
    ngs_qb_df,
    left_on=['player_id', 'season', 'week'],  # Keys from qb_ids_weekly_stats
    right_on=['player_gsis_id', 'season', 'week'],  # Keys from ngs_qb_df
    how='left'  # Retain all rows from qb_ids_weekly_stats
)

# Display the shape of the resulting dataframe
print(f"Shape of merged dataframe: {qb_ids_ngs_weekly_stats.shape}")
print(f"Row count matches: {qb_ids_weekly_stats.shape[0] == qb_ids_ngs_weekly_stats.shape[0]}")

# Display a sample of the merged dataframe
display(qb_ids_ngs_weekly_stats)

# ***csv file***
# qb_ids_ngs_weekly_stats.to_csv('qb_ids_ngs_weekly_stats.csv', index=False)

Shape of merged dataframe: (5267, 67)
Row count matches: True


,season,season_type,week,player_id,player_name,position,position_group,recent_team,mfl_id,sportradar_id,fantasypros_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,completions,attempts,passing_yards,passing_tds,rushing_yards,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota,player_display_name,team_abbr,avg_time_to_throw,avg_completed_air_yards,avg_intended_air_yards,avg_air_yards_differential,aggressiveness,max_completed_air_distance,avg_air_yards_to_sticks,pass_yards,pass_touchdowns,passer_rating,completion_percentage,expected_completion_percentage,completion_percentage_above_expectation,avg_air_distance,max_air_distance,player_gsis_id,player_first_name,player_last_name,player_short_name
0,2017,REG,1,00-0019596,T.Brady,QB,QB,NE,5848,41c44740-d0f6-44ab-8347-3b5d515e5ecf,9603.0,698.0,167.0,tombrady/2504211,2330.0,5228.0,309.0,187741.0,BradTo00,tom-brady-1,1350.0,1163.0,NaN,5228.0,25347.0,4314.0,25347.0,Tom Brady,tom brady,16,36,267.0,0,0.0,0.0,3.0,20.0,0,0,567.0,112.0,11.0,1.753902,0,0.470899,0.042110,Tom Brady,NE,3.120750,9.219375,15.085833,-5.866458,33.333333,57.983004,6.196944,267.0,0.0,70.023148,44.444444,53.709919,-9.265475,27.996684,59.048235,00-0019596,Tom,Brady,T.Brady
1,2017,REG,2,00-0019596,T.Brady,QB,QB,NE,5848,41c44740-d0f6-44ab-8347-3b5d515e5ecf,9603.0,698.0,167.0,tombrady/2504211,2330.0,5228.0,309.0,187741.0,BradTo00,tom-brady-1,1350.0,1163.0,NaN,5228.0,25347.0,4314.0,25347.0,Tom Brady,tom brady,30,39,447.0,3,9.0,0.0,2.0,11.0,0,0,281.0,237.0,19.0,25.089682,0,1.590747,0.304464,Tom Brady,NE,2.301564,6.826333,7.545385,-0.719051,12.820513,45.270829,-1.095641,447.0,3.0,139.583333,76.923077,66.956657,9.966420,19.953223,45.270829,00-0019596,Tom,Brady,T.Brady
2,2017,REG,3,00-0019596,T.Brady,QB,QB,NE,5848,41c44740-d0f6-44ab-8347-3b5d515e5ecf,9603.0,698.0,167.0,tombrady/2504211,2330.0,5228.0,309.0,187741.0,BradTo00,tom-brady-1,1350.0,1163.0,NaN,5228.0,25347.0,4314.0,25347.0,Tom Brady,tom brady,25,35,378.0,5,6.0,0.0,5.0,41.0,3,1,415.0,107.0,16.0,10.521861,1,0.910843,0.240395,Tom Brady,NE,2.707400,10.840000,11.857143,-1.017143,20.000000,NaN,2.057143,378.0,5.0,146.190476,71.428571,NaN,NaN,NaN,NaN,00-0019596,Tom,Brady,T.Brady
3,2017,REG,4,00-0019596,T.Brady,QB,QB,NE,5848,41c44740-d0f6-44ab-8347-3b5d515e5ecf,9603.0,698.0,167.0,tombrady/2504211,2330.0,5228.0,309.0,187741.0,BradTo00,tom-brady-1,1350.0,1163.0,NaN,5228.0,25347.0,4314.0,25347.0,Tom Brady,tom brady,32,45,307.0,2,2.0,0.0,3.0,14.0,0,0,341.0,132.0,18.0,9.758528,0,0.900293,0.149092,Tom Brady,NE,2.456489,5.467500,7.515333,-2.047833,8.888889,32.986209,0.159778,307.0,2.0,104.583333,71.111111,74.774611,-3.663500,19.146993,49.445393,00-0019596,Tom,Brady,T.Brady
4,2017,REG,5,00-0019596,T.Brady,QB,QB,NE,5848,41c44740-d0f6-44ab-8347-3b5d515e5ecf,9603.0,698.0,167.0,tombrady/2504211,2330.0,5228.0,309.0,187741.0,BradTo00,tom-brady-1,1350.0,1163.0,NaN,5228.0,25347.0,4314.0,25347.0,Tom Brady,tom brady,30,40,303.0,1,5.0,1.0,3.0,14.0,1,1,248.0,130.0,15.0,2.653190,0,1.221774,0.115276,Tom Brady,NE,2.694575,5.328000,6.224500,-0.896500,10.000000,38.576066,-3.475500,303.0,1.0,94.062500,75.000000,67.303628,7.696372,18.631851,38.576066,00-0019596,Tom,Brady,T.Brady
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5262,2024,REG,14,00-0039918,C.Williams,QB,QB,CHI,16579,49d4b627-3f19-49a8-ae33-f539e0fb1f88,23084.0,NaN,11560.0,NaN,4431611.0,40900.0,NaN,26695775.0,WillCa03,caleb-williams-3,17695.0,NaN,1550.0,40900.0,0.0,NaN,1286445.0,Caleb Williams,caleb williams,17,23,134.0,2,27.0,0.0,7.0,46.0,1,1,137.0,69.0,9.0,-6.575400,0,0

In [19]:
# Output: pro-football reference dataframe for passing data from the python NFL library
# note: PFR data not available before 2018
# there is no position info so the data will pull QB, TE, RB, and WR passing data

# Define the range of years for PFR data (2018 to the current year)
pfr_years = list(range(2018, current_year))

# Import pro-football reference data for QBs
pfr_pass_df = nfl.import_weekly_pfr('pass', pfr_years)

# Drop unnecessary columns
pfr_pass_df = pfr_pass_df.drop(
    columns=['game_id', 'pfr_game_id', 'receiving_int', 'rushing_int'], errors='ignore'
)

# Display dataframe shape
print(f"Shape of PFR dataframe: {pfr_pass_df.shape}")

# Display a sample of the dataframe
display(pfr_pass_df)

# ***csv file***
# pfr_pass_df.to_csv('pfr_pass_df.csv', index=False)

Shape of PFR dataframe: (4740, 22)


,season,week,game_type,team,opponent,pfr_player_name,pfr_player_id,passing_drops,passing_drop_pct,receiving_drop,receiving_drop_pct,passing_bad_throws,passing_bad_throw_pct,times_sacked,times_blitzed,times_hurried,times_hit,times_pressured,times_pressured_pct,def_times_blitzed,def_times_hurried,def_times_hitqb
0,2018,1,REG,PHI,ATL,Nick Foles,FoleNi00,4.0,0.118,NaN,NaN,6.0,0.176,2.0,7.0,5.0,2.0,9.0,0.250,NaN,NaN,NaN
1,2018,1,REG,PHI,ATL,Nelson Agholor,AghoNe00,0.0,0.000,NaN,NaN,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000,NaN,NaN,NaN
2,2018,1,REG,ATL,PHI,Matt Ryan,RyanMa00,2.0,0.050,NaN,NaN,10.0,0.250,4.0,12.0,3.0,6.0,13.0,0.265,NaN,NaN,NaN
3,2018,1,REG,BAL,BUF,Joe Flacco,FlacJo00,2.0,0.059,NaN,NaN,3.0,0.088,1.0,3.0,8.0,1.0,10.0,0.286,NaN,NaN,NaN
4,2018,1,REG,BAL,BUF,Lamar Jackson,JackLa00,0.0,0.000,NaN,NaN,1.0,0.333,1.0,1.0,2.0,0.0,3.0,0.500,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692,2024,21,CON,KC,BUF,Patrick Mahomes,MahoPa00,2.0,0.080,NaN,NaN,4.0,0.160,2.0,6.0,0.0,5.0,7.0,0.206,NaN,NaN,NaN
693,2024,21,CON,BUF,KC,Josh Allen,AlleJo02,2.0,0.065,NaN,NaN,4.0,0.129,2.0,9.0,1.0,6.0,9.0,0.243,NaN,NaN,NaN
694,2024,22,SB,PHI,KC,Jalen Hurts,HurtJa00,1.0,0.045,NaN,NaN,4.0,0.182,2.0,6.0,4.0,1.0,7.0,0.241,NaN,NaN,NaN
695,2024,22,SB,PHI,KC,Kenny Pickett,PickKe00,0.0,0.000,NaN,NaN,1.0,1.000,0.0,0.0,0.0,0.0,0.0,0.000,NaN,NaN,NaN


In [20]:
# Output: a dataframe of NFL QB info, ids, weekly stats, next-gen stats, and pro-football reference data
# NOTE: unbalanced dataframe - PFR stats start at 2018

# Merge the pfr_pass_df with the qb_ids_ngs_weekly_stats dataframe
qb_ids_ngs_pfr_stats = pd.merge(
    qb_ids_ngs_weekly_stats,
    pfr_pass_df,  # Use the full PFR dataframe as position data is unavailable
    left_on=['pfr_id', 'season', 'week'],  # Keys from qb_ids_ngs_weekly_stats
    right_on=['pfr_player_id', 'season', 'week'],  # Keys from pfr_pass_df
    how='left'  # Retain all rows from qb_ids_ngs_weekly_stats
)

# Display the shape of the resulting dataframe
print(f"Shape of merged dataframe: {qb_ids_ngs_pfr_stats.shape}")

# Row integrity check
print(
    f"Row count matches: {qb_ids_weekly_stats.shape[0] == qb_ids_ngs_weekly_stats.shape[0] == qb_ids_ngs_pfr_stats.shape[0]}"
)

# Display the first few rows of the merged dataframe for review
display(qb_ids_ngs_pfr_stats)

# ***csv file***
# qb_ids_ngs_pfr_stats.to_csv('qb_ids_ngs_pfr_stats.csv', index=False)

Shape of merged dataframe: (5267, 87)
Row count matches: True


,season,season_type,week,player_id,player_name,position,position_group,recent_team,mfl_id,sportradar_id,fantasypros_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,completions,attempts,passing_yards,passing_tds,rushing_yards,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota,player_display_name,team_abbr,avg_time_to_throw,avg_completed_air_yards,avg_intended_air_yards,avg_air_yards_differential,aggressiveness,max_completed_air_distance,avg_air_yards_to_sticks,pass_yards,pass_touchdowns,passer_rating,completion_percentage,expected_completion_percentage,completion_percentage_above_expectation,avg_air_distance,max_air_distance,player_gsis_id,player_first_name,player_last_name,player_short_name,game_type,team,opponent,pfr_player_name,pfr_player_id,passing_drops,passing_drop_pct,receiving_drop,receiving_drop_pct,passing_bad_throws,passing_bad_throw_pct,times_sacked,times_blitzed,times_hurried,times_hit,times_pressured,times_pressured_pct,def_times_blitzed,def_times_hurried,def_times_hitqb
0,2017,REG,1,00-0019596,T.Brady,QB,QB,NE,5848,41c44740-d0f6-44ab-8347-3b5d515e5ecf,9603.0,698.0,167.0,tombrady/2504211,2330.0,5228.0,309.0,187741.0,BradTo00,tom-brady-1,1350.0,1163.0,NaN,5228.0,25347.0,4314.0,25347.0,Tom Brady,tom brady,16,36,267.0,0,0.0,0.0,3.0,20.0,0,0,567.0,112.0,11.0,1.753902,0,0.470899,0.042110,Tom Brady,NE,3.120750,9.219375,15.085833,-5.866458,33.333333,57.983004,6.196944,267.0,0.0,70.023148,44.444444,53.709919,-9.265475,27.996684,59.048235,00-0019596,Tom,Brady,T.Brady,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017,REG,2,00-0019596,T.Brady,QB,QB,NE,5848,41c44740-d0f6-44ab-8347-3b5d515e5ecf,9603.0,698.0,167.0,tombrady/2504211,2330.0,5228.0,309.0,187741.0,BradTo00,tom-brady-1,1350.0,1163.0,NaN,5228.0,25347.0,4314.0,25347.0,Tom Brady,tom brady,30,39,447.0,3,9.0,0.0,2.0,11.0,0,0,281.0,237.0,19.0,25.089682,0,1.590747,0.304464,Tom Brady,NE,2.301564,6.826333,7.545385,-0.719051,12.820513,45.270829,-1.095641,447.0,3.0,139.583333,76.923077,66.956657,9.966420,19.953223,45.270829,00-0019596,Tom,Brady,T.Brady,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017,REG,3,00-0019596,T.Brady,QB,QB,NE,5848,41c44740-d0f6-44ab-8347-3b5d515e5ecf,9603.0,698.0,167.0,tombrady/2504211,2330.0,5228.0,309.0,187741.0,BradTo00,tom-brady-1,1350.0,1163.0,NaN,5228.0,25347.0,4314.0,25347.0,Tom Brady,tom brady,25,35,378.0,5,6.0,0.0,5.0,41.0,3,1,415.0,107.0,16.0,10.521861,1,0.910843,0.240395,Tom Brady,NE,2.707400,10.840000,11.857143,-1.017143,20.000000,NaN,2.057143,378.0,5.0,146.190476,71.428571,NaN,NaN,NaN,NaN,00-0019596,Tom,Brady,T.Brady,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017,REG,4,00-0019596,T.Brady,QB,QB,NE,5848,41c44740-d0f6-44ab-8347-3b5d515e5ecf,9603.0,698.0,167.0,tombrady/2504211,2330.0,5228.0,309.0,187741.0,BradTo00,tom-brady-1,1350.0,1163.0,NaN,5228.0,25347.0,4314.0,25347.0,Tom Brady,tom brady,32,45,307.0,2,2.0,0.0,3.0,14.0,0,0,341.0,132.0,18.0,9.758528,0,0.900293,0.149092,Tom Brady,NE,2.456489,5.467500,7.515333,-2.047833,8.888889,32.986209,0.159778,307.0,2.0,104.583333,71.111111,74.774611,-3.663500,19.146993,49.445393,00-0019596,Tom,Brady,T.Brady,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017,REG,5,00-0019596,T.Brady,QB,QB,NE,5848,41c44740-d0f6-44ab-8347-3b5d515e5ecf,9603.0,698.0,167.0,tombrady/2504211,2330.0,5228.0,309.0,187741.0,BradTo00,tom-brady-1,1350.0,1163.0,NaN,5228.0,25347.0,4314.0,25347.0,Tom Brady,tom brady,30,40,303.0,1,5.0,1.0,3.0,14.0,1,1,248.0,130.0,15.0,2.653190,0,1.221774,0.115276,Tom Brady,NE,2.694575,5.328000,6.224500,-0.896500,10.000000,38.576066,-3.475500,303.0,1.0,94.062500,75.000000,67.303628,7.696372,18.631851,38.576

In [21]:
# *** Baseline Dataframe ***

# Output: a dataframe of NFL QB info, ids, weekly stats, next-gen stats, and pro-football reference data
# Output: Ordered the df by year, week, and passing yards
# NOTE: unbalanced dataframe - PFR stats start at 2018

# Order the dataframe by season (year), week, and passing_yards
qb_ids_ngs_pfr_stats_sorted = qb_ids_ngs_pfr_stats.sort_values(
    by=['season', 'week', 'passing_yards'], 
    ascending=[True, True, False]  # Ascending for season and week, descending for passing_yards
)

# Display the shape of the resulting dataframe
print(f"Shape of merged dataframe: {qb_ids_ngs_pfr_stats_sorted.shape}")

# Row integrity check
print(
    f"Row count matches: {qb_ids_weekly_stats.shape[0] == qb_ids_ngs_weekly_stats.shape[0] == qb_ids_ngs_pfr_stats.shape[0] == qb_ids_ngs_pfr_stats_sorted.shape[0]}"
)

# Display the sorted dataframe
print("Dataframe sorted by season, week, and passing_yards:")
display(qb_ids_ngs_pfr_stats_sorted)

# ***csv file***
# qb_ids_ngs_pfr_stats_sorted.to_csv('qb_ids_ngs_pfr_stats_sorted.csv', index=False)

Shape of merged dataframe: (5267, 87)
Row count matches: True
Dataframe sorted by season, week, and passing_yards:


,season,season_type,week,player_id,player_name,position,position_group,recent_team,mfl_id,sportradar_id,fantasypros_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,completions,attempts,passing_yards,passing_tds,rushing_yards,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota,player_display_name,team_abbr,avg_time_to_throw,avg_completed_air_yards,avg_intended_air_yards,avg_air_yards_differential,aggressiveness,max_completed_air_distance,avg_air_yards_to_sticks,pass_yards,pass_touchdowns,passer_rating,completion_percentage,expected_completion_percentage,completion_percentage_above_expectation,avg_air_distance,max_air_distance,player_gsis_id,player_first_name,player_last_name,player_short_name,game_type,team,opponent,pfr_player_name,pfr_player_id,passing_drops,passing_drop_pct,receiving_drop,receiving_drop_pct,passing_bad_throws,passing_bad_throw_pct,times_sacked,times_blitzed,times_hurried,times_hit,times_pressured,times_pressured_pct,def_times_blitzed,def_times_hurried,def_times_hitqb
104,2017,REG,1,00-0023436,A.Smith,QB,QB,KC,7813,2fda010a-8c62-4c07-b601-4ba03f57e6af,9012.0,2218.0,268.0,alexsmith/2506340,8416.0,7177.0,NaN,552642.0,SmitAl03,alex-smith-3,4306.0,3119.0,NaN,7177.0,217357.0,6739.0,NaN,Alex Smith,alex smith,28,35,368.0,4,3.0,0.0,3.0,16.0,0,0,260.0,201.0,17.0,17.560724,0,1.415385,0.300653,Alex Smith,KC,3.023914,5.651429,7.390571,-1.739143,14.285714,51.773205,-0.580857,368.0,4.0,148.571429,80.000000,70.111395,9.888605,18.876969,51.773205,00-0023436,Alexander,Smith,A.Smith,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
231,2017,REG,1,00-0027854,S.Bradford,QB,QB,MIN,9817,cc3640b0-7560-431f-84ab-599e9dc8cac6,9678.0,5526.0,621.0,sambradford/497095,13197.0,23976.0,6558.0,1123599.0,BradSa00,sam-bradford-1,6452.0,5161.0,NaN,23976.0,333276.0,11527.0,NaN,Sam Bradford,sam bradford,27,32,346.0,3,-3.0,0.0,1.0,5.0,0,0,239.0,136.0,14.0,21.096104,0,1.447699,0.349325,Sam Bradford,MIN,2.568281,8.178519,7.892500,0.286019,18.750000,48.379334,-0.232500,346.0,3.0,142.968750,84.375000,59.131884,25.243116,20.657261,48.379334,00-0027854,Samuel,Bradford,S.Bradford,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165,2017,REG,1,00-0026143,M.Ryan,QB,QB,ATL,9099,7e648a0b-fdc8-4661-a587-5826f2cac11b,9444.0,4317.0,24.0,mattryan/310,11237.0,8780.0,5371.0,420095.0,RyanMa00,matt-ryan-1,5610.0,4637.0,NaN,8780.0,216263.0,732.0,216263.0,Matt Ryan,matt ryan,21,30,321.0,1,11.0,0.0,2.0,13.0,0,0,216.0,158.0,12.0,13.515724,0,1.486111,0.257653,Matt Ryan,ATL,2.577133,7.064286,7.096667,-0.032381,26.666667,39.611728,-1.936667,321.0,1.0,116.111111,70.000000,60.505824,9.494176,19.851132,41.219875,00-0026143,Matthew,Ryan,M.Ryan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120,2017,REG,1,00-0023459,A.Rodgers,QB,QB,GB,7836,0ce48193-e2fa-466e-a986-33f751add206,9001.0,2241.0,96.0,aaronrodgers/2506363,8439.0,7200.0,3452.0,419780.0,RodgAa00,aaron-rodgers-1,4307.0,3118.0,235.0,7200.0,213957.0,2593.0,213957.0,Aaron Rodgers,aaron rodgers,28,42,311.0,1,21.0,1.0,4.0,25.0,0,0,289.0,170.0,16.0,2.171718,0,1.076125,0.098124,Aaron Rodgers,GB,2.997238,4.837857,7.115238,-2.277381,19.047619,39.088602,-2.480000,311.0,1.0,86.507937,66.666667,63.435491,3.231176,21.338579,48.441297,00-0023459,Aaron,Rodgers,A.Rodgers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
494,2017,REG,1,00-0032950,C.Wentz,QB,QB,PHI,12610,e9a5c16b-4472-4be9-8030-3f77be7890cb,15520.0,10636.0,3161.0,NaN,2573079.0,29236.0,NaN,1907522.0,WentCa00,carson-wentz-1,10856.0,11257.0,NaN,29236.0,613866.0,17920.0,613866.0,Carson Wentz,carson wentz,26,39,307.0,2,6.0,1.0,2.0,9.0,0,0,401.0,140.0,17.0,8.479346,1,0.765586,0.119461,Carson Wentz,PHI,2.9

In [22]:
### End: Python NFL Library Dataframe ###

In [23]:
### Begin: Python NFL Library Data Integrity Checks ###

In [24]:
## Data Integrity Checks ##
# load the datframe from the variables in memory or csv files in the current working directory
def load_dataframe(var_name, file_name):
    try:
        return globals()[var_name]  # Try to get the variable from the global namespace
    except KeyError:
        print(f"⚠️ {var_name} not found in memory. Reading from {file_name}")
        return pd.read_csv(file_name)

In [25]:
## Data Integrity Checks ##
# Output: A Stratified Random Sample of years and players from the python nfl library
# Test to ensure merged QB values match the unmerged weekly stats from the source

def test_qb_weekly_stats(years=range(2017, 2024), num_samples=25, tolerance=0.1):
    """
    Perform integrity checks on QB Weekly Stats data by comparing merged and unmerged dataframes.
    This function checks consistency across selected numerical columns.

    Parameters:
    years (range): Range of years to test.
    num_samples (int): Number of random samples per year.
    tolerance (float): Allowed numerical difference for matching values.

    Returns:
    None
    """
    print("\n🔹 Running Weekly Stats Integrity Check for QBs...")

    # Load DataFrames
    merged_df = load_dataframe("qb_ids_ngs_pfr_stats_sorted", "qb_ids_ngs_pfr_stats_sorted.csv")
    weekly_df = load_dataframe("qb_ids_weekly_stats", "qb_ids_weekly_stats.csv")

    print(f"✅ Using {'in-memory variable' if 'qb_ids_ngs_pfr_stats_sorted' in globals() else 'CSV file'}: qb_ids_ngs_pfr_stats_sorted")
    print(f"✅ Using {'in-memory variable' if 'qb_ids_weekly_stats' in globals() else 'CSV file'}: qb_ids_weekly_stats")

    mismatches = []

    for year in years:
        print(f"\nTesting Year: {year}")

        # Filter data for the specified year
        merged_year_df = merged_df[merged_df["season"] == year]
        weekly_year_df = weekly_df[weekly_df["season"] == year]

        # QB numeric columns to validate
        numeric_cols = [
            "completions", "attempts", "passing_yards", "passing_tds",
            "interceptions", "sacks", "rushing_yards"
        ]
        merged_year_df.loc[:, numeric_cols] = merged_year_df[numeric_cols].apply(pd.to_numeric, errors="coerce")
        weekly_year_df.loc[:, numeric_cols] = weekly_year_df[numeric_cols].apply(pd.to_numeric, errors="coerce")

        # Select random sample of players for this year
        sampled_players = sample(list(merged_year_df["player_id"].dropna()), min(num_samples, len(merged_year_df)))

        for player_id in sampled_players:
            # Filter player data from each DataFrame
            merged_player = merged_year_df[merged_year_df["player_id"] == player_id]
            weekly_player = weekly_year_df[weekly_year_df["player_id"] == player_id]

            # Loop through each week and compare values
            for week in merged_player["week"].unique():
                merged_row = merged_player[merged_player["week"] == week]
                weekly_row = weekly_player[weekly_player["week"] == week]

                if merged_row.empty or weekly_row.empty:
                    continue  # Skip if there's no data for this player/week

                # Compare numerical columns
                for col in numeric_cols:
                    merged_val = merged_row[col].values[0] if col in merged_row else None
                    weekly_val = weekly_row[col].values[0] if col in weekly_row else None

                    if merged_val is not None and weekly_val is not None:
                        if abs(float(merged_val) - float(weekly_val)) > tolerance:
                            mismatches.append((year, player_id, week, col, merged_val, weekly_val))

        print(f"Tested {len(sampled_players)} players in Year {year}.")

    # Summary of mismatches
    print(f"\nTotal mismatches found in Weekly Stats: {len(mismatches)}")
    if mismatches:
        print("\nMismatch Details (Year, Player ID, Week, Column, Merged Value, Weekly Value):")
        for m in mismatches[:10]:  # Print only first 10 mismatches
            print(m)
    else:
        print("✅ Weekly Stats Integrity Check Passed!")

# Run the test
test_qb_weekly_stats()



🔹 Running Weekly Stats Integrity Check for QBs...
✅ Using in-memory variable: qb_ids_ngs_pfr_stats_sorted
✅ Using in-memory variable: qb_ids_weekly_stats

Testing Year: 2017
Tested 25 players in Year 2017.

Testing Year: 2018
Tested 25 players in Year 2018.

Testing Year: 2019
Tested 25 players in Year 2019.

Testing Year: 2020
Tested 25 players in Year 2020.

Testing Year: 2021
Tested 25 players in Year 2021.

Testing Year: 2022
Tested 25 players in Year 2022.

Testing Year: 2023
Tested 25 players in Year 2023.

Total mismatches found in Weekly Stats: 0
✅ Weekly Stats Integrity Check Passed!


In [26]:
## Data Integrity Checks ##
# Output: A Stratified Random Sample of years and players from the python nfl library
# Test to ensure merged df values match the next-gen stats within the unmerged df

def test_qb_ngs_stats(years=range(2017, 2024), num_samples=25, tolerance=0.1):
    """
    Perform integrity checks on QB Next-Gen Stats (NGS) data.
    Compares key NGS metrics across merged and unmerged dataframes.
    """
    print("\n🔹 Running Next-Gen Stats (NGS) Integrity Check for QBs...")

    # Load dataframes
    merged_df = load_dataframe("qb_ids_ngs_pfr_stats_sorted", "qb_ids_ngs_pfr_stats_sorted.csv")
    ngs_df = load_dataframe("ngs_qb_df", "ngs_qb_df.csv")

    print("✅ Using in-memory variable: qb_ids_ngs_pfr_stats_sorted" if "qb_ids_ngs_pfr_stats_sorted" in globals() else "✅ Loaded qb_ids_ngs_pfr_stats_sorted from CSV")
    print("✅ Using in-memory variable: ngs_qb_df" if "ngs_qb_df" in globals() else "✅ Loaded ngs_qb_df from CSV")

    mismatches = []

    for year in years:
        print(f"\nTesting Season: {year}")

        merged_year_df = merged_df[merged_df["season"] == year]
        ngs_year_df = ngs_df[ngs_df["season"] == year]

        sampled_players = sample(list(merged_year_df["player_id"].dropna()), min(num_samples, len(merged_year_df)))

        for player_id in sampled_players:
            merged_player = merged_year_df[merged_year_df["player_id"] == player_id]
            ngs_player = ngs_year_df[ngs_year_df["player_gsis_id"] == player_id]

            for week in merged_player["week"].unique():
                merged_row = merged_player[merged_player["week"] == week]
                ngs_row = ngs_player[ngs_player["week"] == week]

                if merged_row.empty or ngs_row.empty:
                    continue

                # Common NGS passing metrics (adjust as needed based on actual NGS columns)
                for col in [
                    "avg_time_to_throw", "avg_air_yards_differential", "avg_air_yards_to_sticks",
                    "completion_percentage_above_expected", "epa", "xcomp", "xpass",
                    "pass_block_win_rate", "expected_pass_block_win_rate"
                ]:
                    merged_val = merged_row[col].values[0] if col in merged_row else None
                    ngs_val = ngs_row[col].values[0] if col in ngs_row else None

                    if merged_val is not None and ngs_val is not None:
                        merged_val = float(merged_val)
                        ngs_val = float(ngs_val)

                        if abs(merged_val - ngs_val) > tolerance:
                            mismatches.append((year, player_id, week, col, merged_val, ngs_val))

        print(f"Tested {len(sampled_players)} players in Season {year}.")

    print(f"\nTotal mismatches found in Next-Gen Stats: {len(mismatches)}")

    if mismatches:
        print("\nMismatch Details (Season, Player ID, Week, Column, Merged Value, NGS Value):")
        for m in mismatches[:10]:
            print(m)
    else:
        print("✅ Next-Gen Stats Integrity Check Passed!")

# Run the test
test_qb_ngs_stats()


🔹 Running Next-Gen Stats (NGS) Integrity Check for QBs...
✅ Using in-memory variable: qb_ids_ngs_pfr_stats_sorted
✅ Using in-memory variable: ngs_qb_df

Testing Season: 2017
Tested 25 players in Season 2017.

Testing Season: 2018
Tested 25 players in Season 2018.

Testing Season: 2019
Tested 25 players in Season 2019.

Testing Season: 2020
Tested 25 players in Season 2020.

Testing Season: 2021
Tested 25 players in Season 2021.

Testing Season: 2022
Tested 25 players in Season 2022.

Testing Season: 2023
Tested 25 players in Season 2023.

Total mismatches found in Next-Gen Stats: 0
✅ Next-Gen Stats Integrity Check Passed!


In [27]:
## Data Integrity Checks ##
# Output: A Stratified Random Sample of years and players from the python nfl library
# Test to ensure merged df values match the pro-football reference stats within the unmerged df

def test_qb_pfr_stats(years=range(2018, 2024), num_samples=25, tolerance=0.1):
    """
    Perform integrity checks on QB data from Pro-Football Reference (PFR).
    Compares key stats across merged and unmerged dataframes.
    """
    print("\n🔹 Running Pro-Football Reference (PFR) Integrity Check for QBs...")

    # Load dataframes
    merged_df = load_dataframe("qb_ids_ngs_pfr_stats_sorted", "qb_ids_ngs_pfr_stats_sorted.csv")
    pfr_df = load_dataframe("pfr_pass_df", "pfr_pass_df.csv")

    # Validation message
    print("✅ Using in-memory variable: qb_ids_ngs_pfr_stats_sorted" if "qb_ids_ngs_pfr_stats_sorted" in globals() else "✅ Loaded qb_ids_ngs_pfr_stats_sorted from CSV")
    print("✅ Using in-memory variable: pfr_pass_df" if "pfr_pass_df" in globals() else "✅ Loaded pfr_pass_df from CSV")

    mismatches = []

    for year in years:
        print(f"\nTesting Season: {year}")

        merged_year_df = merged_df[merged_df["season"] == year].copy()
        pfr_year_df = pfr_df[pfr_df["season"] == year].copy()

        merged_year_df["pfr_id"] = merged_year_df["pfr_id"].astype(str)
        pfr_year_df["pfr_player_id"] = pfr_year_df["pfr_player_id"].astype(str)

        sampled_players = sample(list(merged_year_df["pfr_id"].dropna()), min(num_samples, len(merged_year_df)))

        for player_id in sampled_players:
            merged_player = merged_year_df[merged_year_df["pfr_id"] == player_id]
            pfr_player = pfr_year_df[pfr_year_df["pfr_player_id"] == player_id]

            for week in merged_player["week"].unique():
                merged_row = merged_player[merged_player["week"] == week]
                pfr_row = pfr_player[pfr_player["week"] == week]

                if merged_row.empty or pfr_row.empty:
                    continue

                # Example QB PFR columns — adjust based on actual columns in `pfr_pass_df`
                for col in [
                    "pass_cmp", "pass_att", "pass_cmp_pct", "pass_ay_per_att",
                    "pass_rating", "pass_sk", "pass_sk_yds"
                ]:
                    merged_val = merged_row[col].values[0] if col in merged_row else None
                    pfr_val = pfr_row[col].values[0] if col in pfr_row else None

                    if merged_val is not None and pfr_val is not None:
                        merged_val = float(merged_val)
                        pfr_val = float(pfr_val)

                        if abs(merged_val - pfr_val) > tolerance:
                            mismatches.append((year, player_id, week, col, merged_val, pfr_val))

        print(f"Tested {len(sampled_players)} players in Season {year}.")

    print(f"\nTotal mismatches found in PFR Stats: {len(mismatches)}")

    if mismatches:
        print("\nMismatch Details (Season, Player ID, Week, Column, Merged Value, PFR Value):")
        for m in mismatches[:10]:
            print(m)
    else:
        print("✅ Pro-Football Reference (PFR) Integrity Check Passed!")

# Run the test
test_qb_pfr_stats()


🔹 Running Pro-Football Reference (PFR) Integrity Check for QBs...
✅ Using in-memory variable: qb_ids_ngs_pfr_stats_sorted
✅ Using in-memory variable: pfr_pass_df

Testing Season: 2018
Tested 25 players in Season 2018.

Testing Season: 2019
Tested 25 players in Season 2019.

Testing Season: 2020
Tested 25 players in Season 2020.

Testing Season: 2021
Tested 25 players in Season 2021.

Testing Season: 2022
Tested 25 players in Season 2022.

Testing Season: 2023
Tested 25 players in Season 2023.

Total mismatches found in PFR Stats: 0
✅ Pro-Football Reference (PFR) Integrity Check Passed!


In [28]:
### End: Python NFL Library Data Integrity Checks ###

In [29]:
### Begin:fantasypros webscraping ###

In [30]:
# Generates a list of (year, week) combinations for web scraping.
# - 2017-2020: Weeks 1-17
# - 2021 and beyond: Weeks 1-18
def generate_year_week_combinations(start_year, end_year):
    year_week_combinations = []
    for year in range(start_year, end_year + 1):
        max_week = 17 if year <= 2020 else 18
        year_week_combinations.extend([(year, week) for week in range(1, max_week + 1)])
    return year_week_combinations

In [31]:
# ✅ Output: a dataframe of weekly QB FantasyPros advanced stats week-by-week for defined years
# Define the function to scrape weekly QB advanced stats from Fantasy Pros

def scrape_fantasypros_all_weeks_years_qb(start_year, end_year):
    """
    Scrapes FantasyPros QB advanced stats for all weeks and years with correct ID extraction.
    """
    year_week_combinations = generate_year_week_combinations(start_year, end_year)
    all_data = []

    for year, week in year_week_combinations:
        try:
            # Construct the URL for QBs
            url = f"https://www.fantasypros.com/nfl/advanced-stats-qb.php?year={year}&week={week}&range=week&view=pergame"
            response = requests.get(url)
            response.raise_for_status()

            soup = BeautifulSoup(response.content, 'html.parser')
            table_headers = [header.text.strip() for header in soup.find('thead').find_all('th')]
            table_rows = soup.find('tbody').find_all('tr')

            for row in table_rows:
                fantasypros_id = "Unknown"
                player_name = "Unknown"

                player_link = row.find("a", class_="fp-player-link")
                if player_link:
                    class_list = player_link.get("class", [])
                    for class_name in class_list:
                        if class_name.startswith("fp-id-"):
                            fantasypros_id = class_name.replace("fp-id-", "")
                            break
                    player_name = player_link.text.strip()

                row_data = [cell.text.strip() for cell in row.find_all('td')]

                if len(row_data) == len(table_headers):
                    all_data.append([year, week, fantasypros_id, player_name] + row_data)

            time.sleep(uniform(0.3, 0.9))  # polite pause

        except Exception as e:
            print(f"⚠️ Error occurred while scraping Year: {year}, Week: {week}: {e}")

    column_names = ['Year', 'Week', 'FantasyPros_ID', 'Player'] + table_headers
    qb_fp_advanced_stats_df = pd.DataFrame(all_data, columns=column_names)
    qb_fp_advanced_stats_df.drop(columns=['Rank'], inplace=True, errors='ignore')

    return qb_fp_advanced_stats_df

## ✅ Run the QB scrape
qb_fp_advanced_stats_df = scrape_fantasypros_all_weeks_years_qb(2017, years[-1])

# ✅ Display dataset shape
print(f"\n📊 **Shape of FantasyPros QB DataFrame:** {qb_fp_advanced_stats_df.shape}")
display(qb_fp_advanced_stats_df.head(10))

# *** csv file ***
# qb_fp_advanced_stats_df.to_csv("fantasypros_qb_advanced_stats.csv", index=False)


📊 **Shape of FantasyPros QB DataFrame:** (1120, 24)


,Year,Week,FantasyPros_ID,Player,Player,G,COMP,ATT,PCT,YDS,AIR,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS,SACK,KNCK,HRRY,BLITZ,POOR,DROP,RZ ATT,RTG
0,2017,1,9012,Alex Smith,Alex Smith (FA),1,28,35,80%,368,182,10,3,2,2,2,3,1,6,7,2,0,2,149
1,2017,1,9451,Matthew Stafford,Matthew Stafford (LAR),1,29,41,71%,292,208,11,2,2,1,0,1,3,15,9,2,2,3,113
2,2017,1,9678,Sam Bradford,Sam Bradford (FA),1,27,32,84%,346,221,12,8,3,1,0,1,2,6,10,2,1,4,143
3,2017,1,14142,Trevor Siemian,Trevor Siemian (FA),1,17,28,61%,219,147,9,4,1,1,0,4,4,6,5,4,2,3,94
4,2017,1,15520,Carson Wentz,Carson Wentz (FA),1,26,39,67%,307,190,14,4,2,1,1,2,6,6,22,3,1,3,97
5,2017,1,13890,Marcus Mariota,Marcus Mariota (WAS),1,25,41,61%,256,191,13,3,0,0,0,1,1,8,4,11,0,2,79
6,2017,1,10007,Tyrod Taylor,Tyrod Taylor (NYJ),1,16,28,57%,224,97,10,5,2,1,0,2,3,6,13,3,1,6,92
7,2017,1,12092,Derek Carr,Derek Carr (FA),1,22,32,69%,262,134,13,3,0,0,0,2,1,2,17,3,4,4,114
8,2017,2,9603,Tom Brady,Tom Brady (FA),1,30,39,77%,447,213,19,9,2,1,1,2,3,0,12,1,4,6,140
9,2017,2,15520,Carson Wentz,Carson Wentz (FA),1,25,46,54%,333,238,20,3,1,1,1,6,5,13,12,11,3,4,83


In [32]:
# ✅ Output: a dataframe of QB fantasy points and % rostered stats week-by-week for defined years

# Scrape the weekly QB fantasy points and % rostered from the FantasyPros website
def qb_scrape_fantasypros_fpts_rost(start_year, end_year):
    """
    Scrapes FantasyPros QB FPTS and % Rostered data for all weeks and years, including FantasyPros IDs.
    """
    week_combinations = generate_year_week_combinations(start_year, end_year)
    all_data = []

    for year, week in week_combinations:
        try:
            url = f"https://www.fantasypros.com/nfl/stats/qb.php?year={year}&week={week}&range=week"
            response = requests.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, "html.parser")

            table = soup.find("table", {"id": "data"})
            if table:
                table_str = str(table)
                df = pd.read_html(io.StringIO(table_str), header=1)[0]

                table_rows = table.find("tbody").find_all("tr")

                fantasypros_ids = []
                player_names = []

                for row in table_rows:
                    fantasypros_id = "Unknown"
                    player_name = "Unknown"

                    player_link = row.find("a", class_="fp-player-link")
                    if player_link:
                        for class_name in player_link.get("class", []):
                            if class_name.startswith("fp-id-"):
                                fantasypros_id = class_name.replace("fp-id-", "")
                                break
                        player_name = player_link.text.strip()

                    fantasypros_ids.append(fantasypros_id)
                    player_names.append(player_name)

                df.insert(1, "FantasyPros_ID", fantasypros_ids)
                df["Player"] = player_names
                df["Year"] = year
                df["Week"] = week

                all_data.append(df)

            time.sleep(uniform(0.3, 0.9))

        except Exception as e:
            print(f"⚠️ Error occurred while scraping Year: {year}, Week: {week}: {e}")

    qb_fpts_perct_rost_df = pd.concat(all_data, ignore_index=True)

    columns_to_drop = [
        "Rank", "CMP", "ATT", "PCT", "YDS", "YPA", "TD", "INT",
        "Y/A", "SACKS", "ATT.1", "YDS.1", "TD.1", "FL", "G", "FPTS/G"
    ]
    qb_fpts_perct_rost_df.drop(columns=columns_to_drop, errors="ignore", inplace=True)

    # *** csv file ***
    # qb_fpts_perct_rost_df.to_csv("fantasypros_qb_fpts_perct_rost.csv", index=False)

    print(f"\n📊 **Shape of QB FPTS and % Rostered dataframe after column removal:** {qb_fpts_perct_rost_df.shape}")
    display(qb_fpts_perct_rost_df.head(10))

    return qb_fpts_perct_rost_df

## ✅ Run the scraping function for all years
qb_fpts_perct_rost_df = qb_scrape_fantasypros_fpts_rost(2017, years[-1])


📊 **Shape of QB FPTS and % Rostered dataframe after column removal:** (6966, 6)


,FantasyPros_ID,Player,FPTS,ROST,Year,Week
0,9012,Alex Smith,31.0,5.0%,2017,1
1,9451,Matthew Stafford,27.1,62.7%,2017,1
2,9678,Sam Bradford,25.5,0.0%,2017,1
3,14142,Trevor Siemian,22.7,1.0%,2017,1
4,15520,Carson Wentz,18.9,0.7%,2017,1
5,13890,Marcus Mariota,18.8,2.0%,2017,1
6,10007,Tyrod Taylor,18.8,2.3%,2017,1
7,12092,Derek Carr,18.7,15.1%,2017,1
8,16389,DeShone Kizer,18.6,0.0%,2017,1
9,9444,Matt Ryan,17.9,0.1%,2017,1


In [33]:
# ✅ Output: a dataframe of weekly QB redzone stats week-by-week for defined years

# Scrape the weekly QB redzone stats from the FantasyPros website
def qb_scrape_fantasypros_redzone_stats(start_year, end_year):
    """
    Scrapes FantasyPros QB Red Zone Stats for all weeks and years, including FantasyPros IDs.
    """
    week_combinations = generate_year_week_combinations(start_year, end_year)
    all_data = []

    for year, week in week_combinations:
        try:
            url = f"https://www.fantasypros.com/nfl/red-zone-stats/qb.php?year={year}&range=week&week={week}"
            response = requests.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, "html.parser")

            table = soup.find("table", {"id": "data"})
            if table:
                table_str = str(table)
                df = pd.read_html(io.StringIO(table_str))[0]
                df.columns = df.columns.droplevel(0)
                df = df[df['Player'] != 'Player']  # Remove duplicate headers

                table_rows = table.find("tbody").find_all("tr")

                fantasypros_ids = []
                player_names = []

                for row in table_rows:
                    fantasypros_id = "Unknown"
                    player_name = "Unknown"

                    player_link = row.find("a", class_="fp-player-link")
                    if player_link:
                        for class_name in player_link.get("class", []):
                            if class_name.startswith("fp-id-"):
                                fantasypros_id = class_name.replace("fp-id-", "")
                                break
                        player_name = player_link.text.strip()

                    fantasypros_ids.append(fantasypros_id)
                    player_names.append(player_name)

                df.insert(1, "FantasyPros_ID", fantasypros_ids)
                df["Player"] = player_names
                df["Year"] = year
                df["Week"] = week

                all_data.append(df)

            time.sleep(uniform(0.3, 0.9))

        except Exception as e:
            print(f"⚠️ Error occurred while scraping Year: {year}, Week: {week}: {e}")

    qb_redzone_stats_df = pd.concat(all_data, ignore_index=True)

    columns_to_drop = ['Rank', 'ATT', 'YDS', 'TD', 'PCT', 'FL', 'G', 'FPTS', 'FPTS/G', 'ROST %']
    qb_redzone_stats_df.drop(columns=columns_to_drop, errors='ignore', inplace=True)

    qb_redzone_stats_df.rename(
        columns={
            col: f"{col}_rz" for col in qb_redzone_stats_df.columns
            if col not in ['Player', 'FantasyPros_ID', 'Year', 'Week']
        },
        inplace=True
    )

    # *** csv file ***
    # qb_redzone_stats_df.to_csv("fantasypros_qb_redzone_stats.csv", index=False)

    print(f"\n📊 **Shape of QB Red Zone Stats dataframe after processing:** {qb_redzone_stats_df.shape}")
    display(qb_redzone_stats_df.head(10))

    return qb_redzone_stats_df

## ✅ Run the scraping function for all years
qb_redzone_stats_df = qb_scrape_fantasypros_redzone_stats(2017, years[-1])


📊 **Shape of QB Red Zone Stats dataframe after processing:** (4315, 8)


,FantasyPros_ID,Player,COMP_rz,Y/A_rz,INT_rz,SACKS_rz,Year,Week
0,14142,Trevor Siemian,4,6.2,0,2,2017,1
1,9678,Sam Bradford,6,5.1,0,0,2017,1
2,9451,Matthew Stafford,4,7.3,0,0,2017,1
3,16389,DeShone Kizer,2,3.3,0,1,2017,1
4,10007,Tyrod Taylor,3,1.2,1,0,2017,1
5,12092,Derek Carr,3,4.8,0,0,2017,1
6,9509,Philip Rivers,3,9.0,0,0,2017,1
7,9012,Alex Smith,4,3.2,0,0,2017,1
8,9039,Ben Roethlisberger,3,4.0,0,0,2017,1
9,13890,Marcus Mariota,3,4.2,0,0,2017,1


In [34]:
# *** FantasyPros Dataframes from each section can be used as an alternative for merging ***

# ✅ Output: merged FantasyPros dataframe for QBs
# ⚠️ Note: NOT finalized yet — data normalization still needed

# ✅ Display the shape of each QB FantasyPros dataframe before merging
print(f"\n📊 **Shape of QB Advanced Stats DataFrame:** {qb_fp_advanced_stats_df.shape}")
print(f"📊 **Shape of QB FPTS and % Rostered DataFrame:** {qb_fpts_perct_rost_df.shape}")
print(f"📊 **Shape of QB Red Zone Stats DataFrame:** {qb_redzone_stats_df.shape}")

# ✅ Drop 'Player' from fpts and redzone dataframes before merge
qb_fpts_perct_rost_df = qb_fpts_perct_rost_df.drop(columns=["Player"], errors="ignore")
qb_redzone_stats_df = qb_redzone_stats_df.drop(columns=["Player"], errors="ignore")

# ✅ Merge QB Advanced Stats with Fantasy Points and % Rostered
qb_adv_fpts_rost_merged_df = pd.merge(
    qb_fp_advanced_stats_df,  # Baseline DF (keeps 'Player')
    qb_fpts_perct_rost_df,
    on=['FantasyPros_ID', 'Year', 'Week'],
    how='left'
)

# ✅ Merge with Red Zone Stats
qb_adv_fpts_rost_rz_merged_df = pd.merge(
    qb_adv_fpts_rost_merged_df,
    qb_redzone_stats_df,
    on=['FantasyPros_ID', 'Year', 'Week'],
    how='left'
)

# ✅ Sort for easier verification
qb_adv_fpts_rost_rz_merged_df_sorted = qb_adv_fpts_rost_rz_merged_df.sort_values(
    by=['Year', 'Week', 'YDS'], 
    ascending=[True, True, False]
)

# ✅ Display the updated shape after merging
print(f"\n📊 **FantasyPros QB Shape After Merge:** {qb_adv_fpts_rost_rz_merged_df_sorted.shape}")

# *** csv file ***
# qb_adv_fpts_rost_rz_merged_df_sorted.to_csv('fantasypros_qb_adv_fpts_rost_rz_df.csv', index=False)
display(qb_adv_fpts_rost_rz_merged_df_sorted)


📊 **Shape of QB Advanced Stats DataFrame:** (1120, 24)
📊 **Shape of QB FPTS and % Rostered DataFrame:** (6966, 6)
📊 **Shape of QB Red Zone Stats DataFrame:** (4315, 8)

📊 **FantasyPros QB Shape After Merge:** (1120, 30)


,Year,Week,FantasyPros_ID,Player,Player,G,COMP,ATT,PCT,YDS,AIR,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS,SACK,KNCK,HRRY,BLITZ,POOR,DROP,RZ ATT,RTG,FPTS,ROST,COMP_rz,Y/A_rz,INT_rz,SACKS_rz
0,2017,1,9012,Alex Smith,Alex Smith (FA),1,28,35,80%,368,182,10,3,2,2,2,3,1,6,7,2,0,2,149,31.0,5.0%,4.0,3.2,0.0,0.0
2,2017,1,9678,Sam Bradford,Sam Bradford (FA),1,27,32,84%,346,221,12,8,3,1,0,1,2,6,10,2,1,4,143,25.5,0.0%,6.0,5.1,0.0,0.0
4,2017,1,15520,Carson Wentz,Carson Wentz (FA),1,26,39,67%,307,190,14,4,2,1,1,2,6,6,22,3,1,3,97,18.9,0.7%,2.0,2.0,0.0,0.0
1,2017,1,9451,Matthew Stafford,Matthew Stafford (LAR),1,29,41,71%,292,208,11,2,2,1,0,1,3,15,9,2,2,3,113,27.1,62.7%,4.0,7.3,0.0,0.0
7,2017,1,12092,Derek Carr,Derek Carr (FA),1,22,32,69%,262,134,13,3,0,0,0,2,1,2,17,3,4,4,114,18.7,15.1%,3.0,4.8,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1119,2024,18,9001,Aaron Rodgers,Aaron Rodgers (FA),1,23,36,64%,274,280,24,8,4,2,0,6,4,4,16,8,8,7,113,26.0,23.1%,5.0,5.3,0.0,0.0
1112,2024,18,22900,Bryce Young,Bryce Young (CAR),1,25,34,74%,251,173,11,3,1,0,0,0,0,3,5,5,0,7,124,36.4,57.2%,7.0,7.1,0.0,0.0
1115,2024,18,18600,Kyler Murray,Kyler Murray (ARI),1,25,35,71%,242,144,9,4,0,0,0,0,2,3,10,2,0,7,129,27.9,90.3%,7.0,5.5,0.0,0.0
1116,2024,18,11687,Geno Smith,Geno Smith (LV),1,20,27,74%,223,110,11,4,2,0,0,2,1,2,7,2,0,4,138,26.3,49.2%,5.0,10.8,0.0,0.0


In [35]:
### End:fantasypros webscraping ###

In [36]:
### Begin: FantasyPros Data Integrity Checks Section ###

In [37]:
## Data Integrity Checks ##
# Extract unique years and weeks from each FantasyPros QB dataframe

years_merged = sorted(qb_adv_fpts_rost_rz_merged_df_sorted['Year'].unique())
weeks_merged = sorted(qb_adv_fpts_rost_rz_merged_df_sorted['Week'].unique())

years_redzone = sorted(qb_redzone_stats_df['Year'].unique())
weeks_redzone = sorted(qb_redzone_stats_df['Week'].unique())

years_fpts = sorted(qb_fpts_perct_rost_df['Year'].unique())
weeks_fpts = sorted(qb_fpts_perct_rost_df['Week'].unique())

years_adv_stats = sorted(qb_fp_advanced_stats_df['Year'].unique())
weeks_adv_stats = sorted(qb_fp_advanced_stats_df['Week'].unique())

# Print the results for comparison
print(f"Years in Merged DF: {years_merged}")
print(f"Years in RedZone DF: {years_redzone}")
print(f"Years in FPTS DF: {years_fpts}")
print(f"Years in Advanced Stats DF: {years_adv_stats}\n")

print(f"Weeks in Merged DF: {weeks_merged}")
print(f"Weeks in RedZone DF: {weeks_redzone}")
print(f"Weeks in FPTS DF: {weeks_fpts}")
print(f"Weeks in Advanced Stats DF: {weeks_adv_stats}")


Years in Merged DF: [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
Years in RedZone DF: [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
Years in FPTS DF: [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
Years in Advanced Stats DF: [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]

Weeks in Merged DF: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
Weeks in RedZone DF: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
Weeks in FPTS DF: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
Weeks in Advanced Stats DF: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


In [38]:
## Data Integrity Checks ##
# Output: A Stratified Random Sample of years and players from the scraped FantasyPros data
# ✅ Integrity Check for QB Advanced Stats (using FantasyPros_ID)

def test_qb_advanced_stats(years=range(2017, 2024), num_samples=25, tolerance=0.1, random_seed=42):
    """
    Integrity check for QB advanced stats.

    - Uses Stratified Random Sampling by year.
    - Tests key stats (CMP, ATT, YDS, TD, INT, PCT).
    - Reports % of matches and mismatches found.
    - Logs whether data is loaded from variables or CSV files.
    """
    seed(random_seed)
    mismatches = []

    print("🔹 Running Advanced Stats Integrity Check...")
    print(f"📌 Test Details: Stratified Random Sample by Year")
    print(f"📌 Columns Tested: ['CMP', 'ATT', 'YDS', 'TD', 'INT', 'PCT']\n")

    merged_df = load_dataframe("qb_adv_fpts_rost_rz_merged_df_sorted", "fantasypros_qb_adv_fpts_rost_rz_df.csv")
    adv_stats_df = load_dataframe("qb_fp_advanced_stats_df", "fantasypros_qb_advanced_stats.csv")

    if isinstance(merged_df, pd.DataFrame) and isinstance(adv_stats_df, pd.DataFrame):
        print("📥 Data loaded from **variables** in memory.\n")
    else:
        print("📥 Data loaded from **CSV files**.\n")

    for year in years:
        print(f"\nTesting Year: {year}")

        merged_year_df = merged_df[merged_df['Year'] == year]
        adv_year_df = adv_stats_df[adv_stats_df['Year'] == year]

        if merged_year_df.empty or adv_year_df.empty:
            print(f"⚠️ Skipping Year {year} (No data available)")
            continue

        sampled_ids = sample(list(merged_year_df['FantasyPros_ID'].dropna()), min(num_samples, len(merged_year_df)))

        total_comparisons = 0
        match_count = 0

        for fp_id in sampled_ids:
            for week in merged_year_df[merged_year_df['FantasyPros_ID'] == fp_id]['Week'].unique():
                merged_row = merged_year_df[(merged_year_df['FantasyPros_ID'] == fp_id) & (merged_year_df['Week'] == week)]
                adv_row = adv_year_df[(adv_year_df['FantasyPros_ID'] == fp_id) & (adv_year_df['Week'] == week)]

                if merged_row.empty or adv_row.empty:
                    continue

                for col in ['CMP', 'ATT', 'YDS', 'TD', 'INT', 'PCT']:
                    merged_val = merged_row[col].values[0] if col in merged_row else None
                    adv_val = adv_row[col].values[0] if col in adv_row else None

                    if merged_val is not None and adv_val is not None:
                        total_comparisons += 1
                        if merged_val == adv_val:
                            match_count += 1
                        else:
                            mismatches.append((year, fp_id, week, col, merged_val, adv_val))

        match_percentage = (match_count / total_comparisons * 100) if total_comparisons else 0
        print(f"✅ Tested {len(sampled_ids)} IDs in Year {year} ({total_comparisons} values compared, {match_percentage:.2f}% matched)")

    print("\n🔍 Integrity Check Summary:")
    print(f"✔ Total Comparisons: {match_count + len(mismatches)}")
    print(f"✔ Overall Match Rate: {(match_count / (match_count + len(mismatches)) * 100) if (match_count + len(mismatches)) > 0 else 0:.2f}%")

    if mismatches:
        print("\n❌ Mismatch Details (Showing up to 10 cases):")
        for m in mismatches[:10]:
            print(m)
    else:
        print("✅ All values matched successfully!")

# ✅ Run the test
test_qb_advanced_stats()

🔹 Running Advanced Stats Integrity Check...
📌 Test Details: Stratified Random Sample by Year
📌 Columns Tested: ['CMP', 'ATT', 'YDS', 'TD', 'INT', 'PCT']

📥 Data loaded from **variables** in memory.


Testing Year: 2017
✅ Tested 25 IDs in Year 2017 (447 values compared, 100.00% matched)

Testing Year: 2018
✅ Tested 25 IDs in Year 2018 (411 values compared, 100.00% matched)

Testing Year: 2019
✅ Tested 25 IDs in Year 2019 (378 values compared, 100.00% matched)

Testing Year: 2020
✅ Tested 25 IDs in Year 2020 (444 values compared, 100.00% matched)

Testing Year: 2021
✅ Tested 25 IDs in Year 2021 (414 values compared, 100.00% matched)

Testing Year: 2022
✅ Tested 25 IDs in Year 2022 (474 values compared, 100.00% matched)

Testing Year: 2023
✅ Tested 25 IDs in Year 2023 (420 values compared, 100.00% matched)

🔍 Integrity Check Summary:
✔ Total Comparisons: 420
✔ Overall Match Rate: 100.00%
✅ All values matched successfully!


In [39]:
## Data Integrity Checks ##
# ✅ Integrity Check for QB Fantasy Points & % Rostered (using FantasyPros_ID)

def test_qb_fpts_rost(years=range(2017, 2024), num_samples=25, tolerance=0.1, random_seed=42):
    """
    Integrity check for QB Fantasy Points and % Rostered.

    - Uses Stratified Random Sampling by year.
    - Tests key stats (FPTS, ROST).
    - Reports % of matches and mismatches found.
    - Logs whether data is loaded from variables or CSV files.
    """
    seed(random_seed)
    mismatches = []

    print("\n🔹 Running Fantasy Points & % Rostered Integrity Check...")
    print(f"📌 Test Details: Stratified Random Sample by Year")
    print(f"📌 Columns Tested: ['FPTS', 'ROST']\n")

    merged_df = load_dataframe("qb_adv_fpts_rost_rz_merged_df_sorted", "fantasypros_qb_adv_fpts_rost_rz_df.csv")
    fpts_df = load_dataframe("qb_fpts_perct_rost_df", "fantasypros_qb_fpts_perct_rost.csv")

    if isinstance(merged_df, pd.DataFrame) and isinstance(fpts_df, pd.DataFrame):
        print("📥 Data loaded from **variables** in memory.\n")
    else:
        print("📥 Data loaded from **CSV files**.\n")

    for year in years:
        print(f"\nTesting Year: {year}")

        merged_year_df = merged_df[merged_df['Year'] == year]
        fpts_year_df = fpts_df[fpts_df['Year'] == year]

        if merged_year_df.empty or fpts_year_df.empty:
            print(f"⚠️ Skipping Year {year} (No data available)")
            continue

        sampled_ids = sample(list(merged_year_df['FantasyPros_ID'].dropna()), min(num_samples, len(merged_year_df)))

        total_comparisons = 0
        match_count = 0

        for fp_id in sampled_ids:
            for week in merged_year_df[merged_year_df['FantasyPros_ID'] == fp_id]['Week'].unique():

                merged_row = merged_year_df[(merged_year_df['FantasyPros_ID'] == fp_id) & (merged_year_df['Week'] == week)]
                fpts_row = fpts_year_df[(fpts_year_df['FantasyPros_ID'] == fp_id) & (fpts_year_df['Week'] == week)]

                if merged_row.empty or fpts_row.empty:
                    continue

                for col in ['FPTS', 'ROST']:
                    merged_val = merged_row[col].values[0] if col in merged_row else None
                    fpts_val = fpts_row[col].values[0] if col in fpts_row else None

                    if merged_val is not None and fpts_val is not None:
                        total_comparisons += 1
                        if merged_val == fpts_val:
                            match_count += 1
                        else:
                            mismatches.append((year, fp_id, week, col, merged_val, fpts_val))

        match_percentage = (match_count / total_comparisons * 100) if total_comparisons else 0
        print(f"✅ Tested {len(sampled_ids)} IDs in Year {year} ({total_comparisons} values compared, {match_percentage:.2f}% matched)")

    print("\n🔍 Integrity Check Summary:")
    print(f"✔ Total Comparisons: {match_count + len(mismatches)}")
    print(f"✔ Overall Match Rate: {(match_count / (match_count + len(mismatches)) * 100) if (match_count + len(mismatches)) > 0 else 0:.2f}%")

    if mismatches:
        print("\n❌ Mismatch Details (Showing up to 10 cases):")
        for m in mismatches[:10]:
            print(m)
    else:
        print("✅ All values matched successfully!")

# ✅ Run the test
test_qb_fpts_rost()


🔹 Running Fantasy Points & % Rostered Integrity Check...
📌 Test Details: Stratified Random Sample by Year
📌 Columns Tested: ['FPTS', 'ROST']

📥 Data loaded from **variables** in memory.


Testing Year: 2017
✅ Tested 25 IDs in Year 2017 (298 values compared, 100.00% matched)

Testing Year: 2018
✅ Tested 25 IDs in Year 2018 (274 values compared, 100.00% matched)

Testing Year: 2019
✅ Tested 25 IDs in Year 2019 (252 values compared, 100.00% matched)

Testing Year: 2020
✅ Tested 25 IDs in Year 2020 (296 values compared, 100.00% matched)

Testing Year: 2021
✅ Tested 25 IDs in Year 2021 (276 values compared, 100.00% matched)

Testing Year: 2022
✅ Tested 25 IDs in Year 2022 (316 values compared, 100.00% matched)

Testing Year: 2023
✅ Tested 25 IDs in Year 2023 (280 values compared, 100.00% matched)

🔍 Integrity Check Summary:
✔ Total Comparisons: 280
✔ Overall Match Rate: 100.00%
✅ All values matched successfully!


In [40]:
## Data Integrity Checks ##
# ✅ Integrity Check for QB Red Zone Stats (using FantasyPros_ID)

def test_qb_redzone_stats(years=range(2017, 2024), num_samples=25, tolerance=0.1, random_seed=42):
    """
    Integrity check for QB Red Zone stats.

    - Uses Stratified Random Sampling by year.
    - Tests key stats (PASS TD_rz, ATT_rz, PCT_rz, etc. depending on actual columns present).
    - Reports % of matches and mismatches found.
    - Logs whether data is loaded from variables or CSV files.
    """
    seed(random_seed)
    mismatches = []

    print("\n🔹 Running Red Zone Stats Integrity Check...")
    print(f"📌 Test Details: Stratified Random Sample by Year")
    print(f"📌 Columns Tested: ['ATT_rz', 'COMP_rz', 'TD_rz', 'PCT_rz', 'INT_rz']\n")

    merged_df = load_dataframe("qb_adv_fpts_rost_rz_merged_df_sorted", "fantasypros_qb_adv_fpts_rost_rz_df.csv")
    redzone_df = load_dataframe("qb_redzone_stats_df", "fantasypros_qb_redzone_stats.csv")

    if isinstance(merged_df, pd.DataFrame) and isinstance(redzone_df, pd.DataFrame):
        print("📥 Data loaded from **variables** in memory.\n")
    else:
        print("📥 Data loaded from **CSV files**.\n")

    for year in years:
        print(f"\nTesting Year: {year}")

        merged_year_df = merged_df[merged_df['Year'] == year]
        redzone_year_df = redzone_df[redzone_df['Year'] == year]

        if merged_year_df.empty or redzone_year_df.empty:
            print(f"⚠️ Skipping Year {year} (No data available)")
            continue

        sampled_ids = sample(list(merged_year_df['FantasyPros_ID'].dropna()), min(num_samples, len(merged_year_df)))

        total_comparisons = 0
        match_count = 0

        for fp_id in sampled_ids:
            for week in merged_year_df[merged_year_df['FantasyPros_ID'] == fp_id]['Week'].unique():

                merged_row = merged_year_df[(merged_year_df['FantasyPros_ID'] == fp_id) & (merged_year_df['Week'] == week)]
                redzone_row = redzone_year_df[(redzone_year_df['FantasyPros_ID'] == fp_id) & (redzone_year_df['Week'] == week)]

                if merged_row.empty or redzone_row.empty:
                    continue

                for col in ['ATT_rz', 'COMP_rz', 'TD_rz', 'PCT_rz', 'INT_rz']:  # Adjust to match actual columns
                    merged_val = merged_row[col].values[0] if col in merged_row else None
                    redzone_val = redzone_row[col].values[0] if col in redzone_row else None

                    if merged_val is not None and redzone_val is not None:
                        total_comparisons += 1
                        if merged_val == redzone_val:
                            match_count += 1
                        else:
                            mismatches.append((year, fp_id, week, col, merged_val, redzone_val))

        match_percentage = (match_count / total_comparisons * 100) if total_comparisons else 0
        print(f"✅ Tested {len(sampled_ids)} IDs in Year {year} ({total_comparisons} values compared, {match_percentage:.2f}% matched)")

    print("\n🔍 Integrity Check Summary:")
    print(f"✔ Total Comparisons: {match_count + len(mismatches)}")
    print(f"✔ Overall Match Rate: {(match_count / (match_count + len(mismatches)) * 100) if (match_count + len(mismatches)) > 0 else 0:.2f}%")

    if mismatches:
        print("\n❌ Mismatch Details (Showing up to 10 cases):")
        for m in mismatches[:10]:
            print(m)
    else:
        print("✅ All values matched successfully!")

# ✅ Run the test
test_qb_redzone_stats()


🔹 Running Red Zone Stats Integrity Check...
📌 Test Details: Stratified Random Sample by Year
📌 Columns Tested: ['ATT_rz', 'COMP_rz', 'TD_rz', 'PCT_rz', 'INT_rz']

📥 Data loaded from **variables** in memory.


Testing Year: 2017
✅ Tested 25 IDs in Year 2017 (298 values compared, 100.00% matched)

Testing Year: 2018
✅ Tested 25 IDs in Year 2018 (274 values compared, 100.00% matched)

Testing Year: 2019
✅ Tested 25 IDs in Year 2019 (252 values compared, 100.00% matched)

Testing Year: 2020
✅ Tested 25 IDs in Year 2020 (290 values compared, 100.00% matched)

Testing Year: 2021
✅ Tested 25 IDs in Year 2021 (276 values compared, 100.00% matched)

Testing Year: 2022
✅ Tested 25 IDs in Year 2022 (310 values compared, 100.00% matched)

Testing Year: 2023
✅ Tested 25 IDs in Year 2023 (280 values compared, 100.00% matched)

🔍 Integrity Check Summary:
✔ Total Comparisons: 280
✔ Overall Match Rate: 100.00%
✅ All values matched successfully!


In [41]:
### End: FantasyPros Data Integrity Checks Section ###

In [42]:
## Begin: data normalization for fantasypros dataframes ##

In [43]:
# ✅ Pre-Normalization
# ✅ Output: display the columns to confirm all QB FantasyPros dataframes are present

# Establish the initial dataframes without any normalization
dataframes = {
    "Merged DataFrame": qb_adv_fpts_rost_rz_merged_df_sorted,
    "Advanced Stats DataFrame": qb_fp_advanced_stats_df,
    "FPTS & ROST DataFrame": qb_fpts_perct_rost_df,
    "Redzone Stats DataFrame": qb_redzone_stats_df
}

# Extract and compare column names
column_details = {name: df.columns.tolist() for name, df in dataframes.items()}

# Convert to DataFrame for easier comparison
columns_df = pd.DataFrame.from_dict(column_details, orient="index").transpose()

# Display column comparison
print("🔍 Initial Column Comparison Across QB DataFrames:\n")
print(columns_df)

🔍 Initial Column Comparison Across QB DataFrames:

   Merged DataFrame Advanced Stats DataFrame FPTS & ROST DataFrame Redzone Stats DataFrame
0              Year                     Year        FantasyPros_ID          FantasyPros_ID
1              Week                     Week                  FPTS                 COMP_rz
2    FantasyPros_ID           FantasyPros_ID                  ROST                  Y/A_rz
3            Player                   Player                  Year                  INT_rz
4            Player                   Player                  Week                SACKS_rz
5                 G                        G                  None                    Year
6              COMP                     COMP                  None                    Week
7               ATT                      ATT                  None                    None
8               PCT                      PCT                  None                    None
9               YDS                    

In [44]:
# ✅ Data Normalization - Rename Columns for Consistency
# ✅ Output: display columns

# Initialize dataframes and rename 'Year' to 'Season' where needed
dataframes = {
    "Merged DataFrame": qb_adv_fpts_rost_rz_merged_df_sorted.copy(),
    "Advanced Stats DataFrame": qb_fp_advanced_stats_df.copy(),
    "FPTS & ROST DataFrame": qb_fpts_perct_rost_df.copy(),
    "Redzone Stats DataFrame": qb_redzone_stats_df.copy()
}

# Rename 'Year' to 'Season' for consistency
for name, df in dataframes.items():
    if 'Year' in df.columns:
        df.rename(columns={'Year': 'Season'}, inplace=True)

# Check renamed columns
for name, df in dataframes.items():
    print(f"\n📊 {name} Columns After Renaming:")
    print(df.columns.tolist())



📊 Merged DataFrame Columns After Renaming:
['Season', 'Week', 'FantasyPros_ID', 'Player', 'Player', 'G', 'COMP', 'ATT', 'PCT', 'YDS', 'AIR', '10+ YDS', '20+ YDS', '30+ YDS', '40+ YDS', '50+ YDS', 'SACK', 'KNCK', 'HRRY', 'BLITZ', 'POOR', 'DROP', 'RZ ATT', 'RTG', 'FPTS', 'ROST', 'COMP_rz', 'Y/A_rz', 'INT_rz', 'SACKS_rz']

📊 Advanced Stats DataFrame Columns After Renaming:
['Season', 'Week', 'FantasyPros_ID', 'Player', 'Player', 'G', 'COMP', 'ATT', 'PCT', 'YDS', 'AIR', '10+ YDS', '20+ YDS', '30+ YDS', '40+ YDS', '50+ YDS', 'SACK', 'KNCK', 'HRRY', 'BLITZ', 'POOR', 'DROP', 'RZ ATT', 'RTG']

📊 FPTS & ROST DataFrame Columns After Renaming:
['FantasyPros_ID', 'FPTS', 'ROST', 'Season', 'Week']

📊 Redzone Stats DataFrame Columns After Renaming:
['FantasyPros_ID', 'COMP_rz', 'Y/A_rz', 'INT_rz', 'SACKS_rz', 'Season', 'Week']


In [45]:
# ✅ Data Normalization - Handle duplicate 'Player' columns in the QB merged and advanced stats dataframes
# ✅ Output: display columns in the merged and advanced stats dataframes

# Rename duplicate 'Player' columns for the relevant QB dataframes
def rename_player_columns(dataframes):
    # Rename 'Player' column in the Merged DataFrame
    merged_df = dataframes["Merged DataFrame"]
    merged_df.columns.values[4] = "PlayerTM"  # Rename column at index 4

    # Rename 'Player' column in the Advanced Stats DataFrame
    adv_stats_df = dataframes["Advanced Stats DataFrame"]
    adv_stats_df.columns.values[4] = "PlayerTM"  # Rename column at index 4

    # Confirm renaming
    print("\n📊 Merged DataFrame Columns After Renaming:")
    print(merged_df.columns.tolist())

    print("\n📊 Advanced Stats DataFrame Columns After Renaming:")
    print(adv_stats_df.columns.tolist())

# Apply the renaming function
rename_player_columns(dataframes)


📊 Merged DataFrame Columns After Renaming:
['Season', 'Week', 'FantasyPros_ID', 'Player', 'PlayerTM', 'G', 'COMP', 'ATT', 'PCT', 'YDS', 'AIR', '10+ YDS', '20+ YDS', '30+ YDS', '40+ YDS', '50+ YDS', 'SACK', 'KNCK', 'HRRY', 'BLITZ', 'POOR', 'DROP', 'RZ ATT', 'RTG', 'FPTS', 'ROST', 'COMP_rz', 'Y/A_rz', 'INT_rz', 'SACKS_rz']

📊 Advanced Stats DataFrame Columns After Renaming:
['Season', 'Week', 'FantasyPros_ID', 'Player', 'PlayerTM', 'G', 'COMP', 'ATT', 'PCT', 'YDS', 'AIR', '10+ YDS', '20+ YDS', '30+ YDS', '40+ YDS', '50+ YDS', 'SACK', 'KNCK', 'HRRY', 'BLITZ', 'POOR', 'DROP', 'RZ ATT', 'RTG']


In [46]:
# ✅ Data normalization - confirm renamed 'Player' column in the merged and advanced stats dataframes
# ✅ Output: display first five rows in the merged and advanced stats dataframes

# Display the first 5 rows of the Merged DataFrame
print("\n🔍 First 5 Rows of Merged DataFrame:")
display(dataframes["Merged DataFrame"].head())

# Display the first 5 rows of the Advanced Stats DataFrame
print("\n🔍 First 5 Rows of Advanced Stats DataFrame:")
display(dataframes["Advanced Stats DataFrame"].head())


🔍 First 5 Rows of Merged DataFrame:


,Season,Week,FantasyPros_ID,Player,PlayerTM,G,COMP,ATT,PCT,YDS,AIR,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS,SACK,KNCK,HRRY,BLITZ,POOR,DROP,RZ ATT,RTG,FPTS,ROST,COMP_rz,Y/A_rz,INT_rz,SACKS_rz
0,2017,1,9012,Alex Smith,Alex Smith (FA),1,28,35,80%,368,182,10,3,2,2,2,3,1,6,7,2,0,2,149,31.0,5.0%,4.0,3.2,0.0,0.0
2,2017,1,9678,Sam Bradford,Sam Bradford (FA),1,27,32,84%,346,221,12,8,3,1,0,1,2,6,10,2,1,4,143,25.5,0.0%,6.0,5.1,0.0,0.0
4,2017,1,15520,Carson Wentz,Carson Wentz (FA),1,26,39,67%,307,190,14,4,2,1,1,2,6,6,22,3,1,3,97,18.9,0.7%,2.0,2.0,0.0,0.0
1,2017,1,9451,Matthew Stafford,Matthew Stafford (LAR),1,29,41,71%,292,208,11,2,2,1,0,1,3,15,9,2,2,3,113,27.1,62.7%,4.0,7.3,0.0,0.0
7,2017,1,12092,Derek Carr,Derek Carr (FA),1,22,32,69%,262,134,13,3,0,0,0,2,1,2,17,3,4,4,114,18.7,15.1%,3.0,4.8,0.0,0.0



🔍 First 5 Rows of Advanced Stats DataFrame:


,Season,Week,FantasyPros_ID,Player,PlayerTM,G,COMP,ATT,PCT,YDS,AIR,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS,SACK,KNCK,HRRY,BLITZ,POOR,DROP,RZ ATT,RTG
0,2017,1,9012,Alex Smith,Alex Smith (FA),1,28,35,80%,368,182,10,3,2,2,2,3,1,6,7,2,0,2,149
1,2017,1,9451,Matthew Stafford,Matthew Stafford (LAR),1,29,41,71%,292,208,11,2,2,1,0,1,3,15,9,2,2,3,113
2,2017,1,9678,Sam Bradford,Sam Bradford (FA),1,27,32,84%,346,221,12,8,3,1,0,1,2,6,10,2,1,4,143
3,2017,1,14142,Trevor Siemian,Trevor Siemian (FA),1,17,28,61%,219,147,9,4,1,1,0,4,4,6,5,4,2,3,94
4,2017,1,15520,Carson Wentz,Carson Wentz (FA),1,26,39,67%,307,190,14,4,2,1,1,2,6,6,22,3,1,3,97


In [47]:
# ✅ Data Normalization - Reorder Columns
# ✅ Output: first three columns for all QB dataframes: 'Season', 'Week', 'FantasyPros_ID'

# Define a helper function to reorder columns in-place
def reorder_columns_inplace(df, first_cols=['Season', 'Week', 'FantasyPros_ID']):
    """Reorders the DataFrame columns to place first_cols at the beginning in-place."""
    current_cols = list(df.columns)
    remaining_cols = [col for col in current_cols if col not in first_cols]
    reordered_df = df[first_cols + remaining_cols]
    return reordered_df

# Apply the function to each DataFrame separately
dataframes["Merged DataFrame"] = reorder_columns_inplace(dataframes["Merged DataFrame"])
dataframes["Advanced Stats DataFrame"] = reorder_columns_inplace(dataframes["Advanced Stats DataFrame"])
dataframes["FPTS & ROST DataFrame"] = reorder_columns_inplace(dataframes["FPTS & ROST DataFrame"])
dataframes["Redzone Stats DataFrame"] = reorder_columns_inplace(dataframes["Redzone Stats DataFrame"])

# Confirm column order
for name, df in dataframes.items():
    print(f"\n🔍 {name} First Three Columns: {df.columns[:3].tolist()}")


🔍 Merged DataFrame First Three Columns: ['Season', 'Week', 'FantasyPros_ID']

🔍 Advanced Stats DataFrame First Three Columns: ['Season', 'Week', 'FantasyPros_ID']

🔍 FPTS & ROST DataFrame First Three Columns: ['Season', 'Week', 'FantasyPros_ID']

🔍 Redzone Stats DataFrame First Three Columns: ['Season', 'Week', 'FantasyPros_ID']


In [48]:
# ✅ Pre-Normalization - Initial Data Type Inspection
# ✅ Output: columns and data types

# Inspect data types of each dataframe before applying normalization
for name, df in dataframes.items():
    print(f"\n📊 Data Types Summary for {name}:")
    print(df.dtypes)


📊 Data Types Summary for Merged DataFrame:
Season              int64
Week                int64
FantasyPros_ID     object
Player             object
PlayerTM           object
G                  object
COMP               object
ATT                object
PCT                object
YDS                object
AIR                object
10+ YDS            object
20+ YDS            object
30+ YDS            object
40+ YDS            object
50+ YDS            object
SACK               object
KNCK               object
HRRY               object
BLITZ              object
POOR               object
DROP               object
RZ ATT             object
RTG                object
FPTS              float64
ROST               object
COMP_rz           float64
Y/A_rz            float64
INT_rz            float64
SACKS_rz          float64
dtype: object

📊 Data Types Summary for Advanced Stats DataFrame:
Season             int64
Week               int64
FantasyPros_ID    object
Player            object
PlayerTM  

In [49]:
# data normalization - convert all columns to object (str) types
# no output
# Helper Function: Convert to Object & Strip Whitespaces
def convert_to_object_and_strip(df):
    """
    Converts all columns in a DataFrame to object type and strips leading/trailing whitespaces.
    """
    df_copy = df.copy()  # Create a copy to avoid modifying the original dataframe

    for col in df_copy.columns:
        # Convert to object type
        df_copy[col] = df_copy[col].astype(str).apply(lambda x: x.strip() if isinstance(x, str) else x)
    
    return df_copy

In [50]:
# Apply the function to all dataframes
dataframes = {name: convert_to_object_and_strip(df) for name, df in dataframes.items()}

# Confirm data type conversion
for name, df in dataframes.items():
    print(f"\n📊 Data Types Summary for {name}:")
    print(df.dtypes)


📊 Data Types Summary for Merged DataFrame:
Season            object
Week              object
FantasyPros_ID    object
Player            object
PlayerTM          object
G                 object
COMP              object
ATT               object
PCT               object
YDS               object
AIR               object
10+ YDS           object
20+ YDS           object
30+ YDS           object
40+ YDS           object
50+ YDS           object
SACK              object
KNCK              object
HRRY              object
BLITZ             object
POOR              object
DROP              object
RZ ATT            object
RTG               object
FPTS              object
ROST              object
COMP_rz           object
Y/A_rz            object
INT_rz            object
SACKS_rz          object
dtype: object

📊 Data Types Summary for Advanced Stats DataFrame:
Season            object
Week              object
FantasyPros_ID    object
Player            object
PlayerTM          object
G              

In [51]:
# data normalization - verification of data types
# output: a display of all columns in all dataframes as 'object' datatypes 

def compare_column_dtypes(dataframes, merged_df_name="Merged DataFrame"):
    """
    Compare column data types between the merged dataframe and each unmerged dataframe.

    Parameters:
    - dataframes: dict of dataframes to compare.
    - merged_df_name: Name of the merged dataframe in the dictionary.

    Returns:
    - Comparison results printed to the console.
    """
    merged_df = dataframes[merged_df_name]
    merged_dtypes = merged_df.dtypes.to_dict()

    comparison_results = {}

    for name, df in dataframes.items():
        if name == merged_df_name:
            continue  # Skip comparison with itself

        df_dtypes = df.dtypes.to_dict()
        mismatch_report = []

        for col, dtype in df_dtypes.items():
            if col in merged_dtypes:
                if dtype != merged_dtypes[col]:
                    mismatch_report.append((col, dtype, merged_dtypes[col]))

        if mismatch_report:
            comparison_results[name] = mismatch_report
        else:
            comparison_results[name] = "✅ All data types match"

    # Display results
    print("\n🔍 Column Data Type Comparison Results:\n")
    for name, result in comparison_results.items():
        if result == "✅ All data types match":
            print(f"{name}: ✅ All data types match")
        else:
            print(f"\n{name}: ❌ Mismatched Columns Detected")
            for col, dtype_unmerged, dtype_merged in result:
                print(f"  - Column: {col} | Unmerged Type: {dtype_unmerged} | Merged Type: {dtype_merged}")

# Run the comparison
compare_column_dtypes(dataframes)


🔍 Column Data Type Comparison Results:

Advanced Stats DataFrame: ✅ All data types match
FPTS & ROST DataFrame: ✅ All data types match
Redzone Stats DataFrame: ✅ All data types match


In [52]:
# data normalization - verification of column order
# output: display of the first three columns of all dataframes

# Define the correct order of columns
expected_first_columns = ['Season', 'Week', 'FantasyPros_ID']

# Function to check column order
def check_column_order(dataframes, expected_first_columns):
    """
    Check if the first columns of each dataframe match the expected columns.

    Parameters:
    - dataframes: dict of dataframes to check.
    - expected_first_columns: list of column names expected to appear first.

    Returns:
    - Prints the order check results for each dataframe.
    """
    print("\n🔍 Column Order Verification:\n")
    for name, df in dataframes.items():
        first_columns = df.columns[:len(expected_first_columns)].tolist()
        
        if first_columns == expected_first_columns:
            print(f"{name}: ✅ Columns are in the correct order: {first_columns}")
        else:
            print(f"{name}: ❌ Incorrect column order. Found: {first_columns} | Expected: {expected_first_columns}")

# Run the column order check
check_column_order(dataframes, expected_first_columns)


🔍 Column Order Verification:

Merged DataFrame: ✅ Columns are in the correct order: ['Season', 'Week', 'FantasyPros_ID']
Advanced Stats DataFrame: ✅ Columns are in the correct order: ['Season', 'Week', 'FantasyPros_ID']
FPTS & ROST DataFrame: ✅ Columns are in the correct order: ['Season', 'Week', 'FantasyPros_ID']
Redzone Stats DataFrame: ✅ Columns are in the correct order: ['Season', 'Week', 'FantasyPros_ID']


In [53]:
# ✅ Data Normalization - Check to ensure there are no missing columns
# ✅ Output: an empty dictionary {} if all match

# Define expected columns for each QB dataframe
expected_columns = {
    "Merged DataFrame": [
        'Season', 'Week', 'FantasyPros_ID', 'Player', 'PlayerTM', 'G',
        'COMP', 'ATT', 'PCT', 'YDS', 'AIR', '10+ YDS', '20+ YDS', '30+ YDS',
        '40+ YDS', '50+ YDS', 'SACK', 'KNCK', 'HRRY', 'BLITZ', 'POOR',
        'DROP', 'RZ ATT', 'RTG', 'FPTS', 'ROST', 'COMP_rz', 'Y/A_rz', 'INT_rz', 'SACKS_rz'
    ],
    "Advanced Stats DataFrame": [
        'Season', 'Week', 'FantasyPros_ID', 'Player', 'PlayerTM', 'G',
        'COMP', 'ATT', 'PCT', 'YDS', 'AIR', '10+ YDS', '20+ YDS', '30+ YDS',
        '40+ YDS', '50+ YDS', 'SACK', 'KNCK', 'HRRY', 'BLITZ', 'POOR',
        'DROP', 'RZ ATT', 'RTG'
    ],
    "FPTS & ROST DataFrame": [
        'Season', 'Week', 'FantasyPros_ID', 'FPTS', 'ROST'
    ],
    "Redzone Stats DataFrame": [
        'Season', 'Week', 'FantasyPros_ID', 'COMP_rz', 'Y/A_rz', 'INT_rz', 'SACKS_rz'
    ]
}

# Check for missing and unexpected columns in each QB dataframe
column_issues = {}

for df_name, df in dataframes.items():
    expected = set(expected_columns[df_name])
    actual = set(df.columns.tolist())

    missing = expected - actual
    unexpected = actual - expected

    if missing or unexpected:
        column_issues[df_name] = {
            "Missing Columns": list(missing) if missing else None,
            "Unexpected Columns": list(unexpected) if unexpected else None
        }

# Display results
if column_issues:
    print("🔍 Column Presence Issues Detected:\n")
    for df_name, issues in column_issues.items():
        print(f"{df_name}:")
        if issues["Missing Columns"]:
            print(f"  ❌ Missing Columns: {issues['Missing Columns']}")
        if issues["Unexpected Columns"]:
            print(f"  ⚠️  Unexpected Columns: {issues['Unexpected Columns']}")
else:
    print("✅ All columns are present and consistent across all QB dataframes.")


✅ All columns are present and consistent across all QB dataframes.


In [54]:
# data normalization - remove leading and trailing whitespaces from all object (string) columns 
# no output

def remove_whitespace(df, inplace=False):
    """
    Safely strips leading/trailing whitespace from all string values in object-type columns.
    """
    if not inplace:
        df = df.copy()

    for col in df.select_dtypes(include='object').columns:
        # Only apply .str.strip() if the column contains string values
        if df[col].apply(lambda x: isinstance(x, str)).any():
            df[col] = df[col].astype(str).str.strip()

    if not inplace:
        return df

In [55]:
# data normalization - remove white spaces
# output: dataframes stripped of whitespaces

# Apply whitespace removal to each dataframe separately
dataframes["Merged DataFrame"] = remove_whitespace(dataframes["Merged DataFrame"])
dataframes["Advanced Stats DataFrame"] = remove_whitespace(dataframes["Advanced Stats DataFrame"])
dataframes["FPTS & ROST DataFrame"] = remove_whitespace(dataframes["FPTS & ROST DataFrame"])
dataframes["Redzone Stats DataFrame"] = remove_whitespace(dataframes["Redzone Stats DataFrame"])

# Confirm whitespace removal by displaying first few rows
for name, df in dataframes.items():
    print(f"\n🔍 {name} - Sample Data (Post Whitespace Removal):")
    print(df.head(5))


🔍 Merged DataFrame - Sample Data (Post Whitespace Removal):
  Season Week FantasyPros_ID            Player                PlayerTM  G COMP ATT  PCT  YDS  AIR 10+ YDS 20+ YDS 30+ YDS 40+ YDS 50+ YDS SACK KNCK HRRY BLITZ POOR DROP RZ ATT  RTG  FPTS   ROST COMP_rz Y/A_rz INT_rz SACKS_rz
0   2017    1           9012        Alex Smith         Alex Smith (FA)  1   28  35  80%  368  182      10       3       2       2       2    3    1    6     7    2    0      2  149  31.0   5.0%     4.0    3.2    0.0      0.0
2   2017    1           9678      Sam Bradford       Sam Bradford (FA)  1   27  32  84%  346  221      12       8       3       1       0    1    2    6    10    2    1      4  143  25.5   0.0%     6.0    5.1    0.0      0.0
4   2017    1          15520      Carson Wentz       Carson Wentz (FA)  1   26  39  67%  307  190      14       4       2       1       1    2    6    6    22    3    1      3   97  18.9   0.7%     2.0    2.0    0.0      0.0
1   2017    1           9451  Matthew S

In [56]:
# data normalization  - standardize decimal places for numerical data and percentages
# no output
def format_precision(df, precision_map=None):
    if precision_map is None:
        precision_map = {
            'PCT': '.2f',         # Completion %
            'RTG': '.1f',         # Rating
            'ROST': '.3f',        # % Rostered
            'Y/A_rz': '.2f',      # Red zone yards/attempt
            'PCT_rz': '.2f'       # Red zone completion %
        }

    df_copy = df.copy()
    
    for col, fmt in precision_map.items():
        if col in df_copy.columns:
            try:
                df_copy[col] = pd.to_numeric(df_copy[col], errors='coerce')
                df_copy[col] = df_copy[col].map(lambda x: format(x, fmt) if pd.notna(x) else "nan")
            except Exception as e:
                print(f"⚠️ Error formatting column {col}: {e}")
    
    return df_copy

In [57]:
# ✅ Data normalization - apply decimal place precision for QB columns
# ✅ Output: formatted sample from each dataframe

# Define QB-specific precision formatting
precision_map = {
    'PCT': '.2f',        # Completion percentage
    'RTG': '.1f',        # Passer rating
    'ROST': '.3f',       # % Rostered
    'Y/A_rz': '.2f',     # Red zone yards/attempt
    'PCT_rz': '.2f'      # Red zone completion %
}

# Apply formatting function to each dataframe
dataframes["Merged DataFrame"] = format_precision(dataframes["Merged DataFrame"], precision_map)
dataframes["Advanced Stats DataFrame"] = format_precision(dataframes["Advanced Stats DataFrame"], precision_map)
dataframes["FPTS & ROST DataFrame"] = format_precision(dataframes["FPTS & ROST DataFrame"], precision_map)
dataframes["Redzone Stats DataFrame"] = format_precision(dataframes["Redzone Stats DataFrame"], precision_map)

# Confirm precision formatting by displaying relevant sample data
for name, df in dataframes.items():
    print(f"\n🔍 {name} - Sample Data (Post Precision Formatting):")
    display(df.head(5))


🔍 Merged DataFrame - Sample Data (Post Precision Formatting):


,Season,Week,FantasyPros_ID,Player,PlayerTM,G,COMP,ATT,PCT,YDS,AIR,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS,SACK,KNCK,HRRY,BLITZ,POOR,DROP,RZ ATT,RTG,FPTS,ROST,COMP_rz,Y/A_rz,INT_rz,SACKS_rz
0,2017,1,9012,Alex Smith,Alex Smith (FA),1,28,35,nan,368,182,10,3,2,2,2,3,1,6,7,2,0,2,149.0,31.0,nan,4.0,3.20,0.0,0.0
2,2017,1,9678,Sam Bradford,Sam Bradford (FA),1,27,32,nan,346,221,12,8,3,1,0,1,2,6,10,2,1,4,143.0,25.5,nan,6.0,5.10,0.0,0.0
4,2017,1,15520,Carson Wentz,Carson Wentz (FA),1,26,39,nan,307,190,14,4,2,1,1,2,6,6,22,3,1,3,97.0,18.9,nan,2.0,2.00,0.0,0.0
1,2017,1,9451,Matthew Stafford,Matthew Stafford (LAR),1,29,41,nan,292,208,11,2,2,1,0,1,3,15,9,2,2,3,113.0,27.1,nan,4.0,7.30,0.0,0.0
7,2017,1,12092,Derek Carr,Derek Carr (FA),1,22,32,nan,262,134,13,3,0,0,0,2,1,2,17,3,4,4,114.0,18.7,nan,3.0,4.80,0.0,0.0



🔍 Advanced Stats DataFrame - Sample Data (Post Precision Formatting):


,Season,Week,FantasyPros_ID,Player,PlayerTM,G,COMP,ATT,PCT,YDS,AIR,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS,SACK,KNCK,HRRY,BLITZ,POOR,DROP,RZ ATT,RTG
0,2017,1,9012,Alex Smith,Alex Smith (FA),1,28,35,nan,368,182,10,3,2,2,2,3,1,6,7,2,0,2,149.0
1,2017,1,9451,Matthew Stafford,Matthew Stafford (LAR),1,29,41,nan,292,208,11,2,2,1,0,1,3,15,9,2,2,3,113.0
2,2017,1,9678,Sam Bradford,Sam Bradford (FA),1,27,32,nan,346,221,12,8,3,1,0,1,2,6,10,2,1,4,143.0
3,2017,1,14142,Trevor Siemian,Trevor Siemian (FA),1,17,28,nan,219,147,9,4,1,1,0,4,4,6,5,4,2,3,94.0
4,2017,1,15520,Carson Wentz,Carson Wentz (FA),1,26,39,nan,307,190,14,4,2,1,1,2,6,6,22,3,1,3,97.0



🔍 FPTS & ROST DataFrame - Sample Data (Post Precision Formatting):


,Season,Week,FantasyPros_ID,FPTS,ROST
0,2017,1,9012,31.0,nan
1,2017,1,9451,27.1,nan
2,2017,1,9678,25.5,nan
3,2017,1,14142,22.7,nan
4,2017,1,15520,18.9,nan



🔍 Redzone Stats DataFrame - Sample Data (Post Precision Formatting):


,Season,Week,FantasyPros_ID,COMP_rz,Y/A_rz,INT_rz,SACKS_rz
0,2017,1,14142,4,6.20,0,2
1,2017,1,9678,6,5.10,0,0
2,2017,1,9451,4,7.30,0,0
3,2017,1,16389,2,3.30,0,1
4,2017,1,10007,3,1.20,1,0


In [58]:
## End: data normalization for fantasypros dataframes ##

In [59]:
### Begin: Enhanced data integrity checks for the fantasypros dataframes ###

In [60]:
# Enhanced data integrity checks
# no output
def generate_md5_hash(row):
    """
    Generate a consistent MD5 hash string from a row's values.
    
    Parameters:
    - row: A pandas Series representing a row in a DataFrame.
    
    Returns:
    - Hash string (MD5)
    """
    row_str = "|".join(map(str, row))
    return hashlib.md5(row_str.encode()).hexdigest()


In [61]:
# Enhanced data integrity checks
# output: Merged DataFrame with a hash column

# Generate hashes for the Merged DataFrame
dataframes["Merged DataFrame"]["row_hash"] = dataframes["Merged DataFrame"].apply(generate_md5_hash, axis=1)

# Display sample hashes
print("\n✅ Merged DataFrame - Hash Generation Complete")
display(dataframes["Merged DataFrame"].head(5))


✅ Merged DataFrame - Hash Generation Complete


,Season,Week,FantasyPros_ID,Player,PlayerTM,G,COMP,ATT,PCT,YDS,AIR,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS,SACK,KNCK,HRRY,BLITZ,POOR,DROP,RZ ATT,RTG,FPTS,ROST,COMP_rz,Y/A_rz,INT_rz,SACKS_rz,row_hash
0,2017,1,9012,Alex Smith,Alex Smith (FA),1,28,35,nan,368,182,10,3,2,2,2,3,1,6,7,2,0,2,149.0,31.0,nan,4.0,3.20,0.0,0.0,7d7821bd1f3ef5e1fe814f092cf8a66c
2,2017,1,9678,Sam Bradford,Sam Bradford (FA),1,27,32,nan,346,221,12,8,3,1,0,1,2,6,10,2,1,4,143.0,25.5,nan,6.0,5.10,0.0,0.0,a18cb95a899433f729df5cab51fe548b
4,2017,1,15520,Carson Wentz,Carson Wentz (FA),1,26,39,nan,307,190,14,4,2,1,1,2,6,6,22,3,1,3,97.0,18.9,nan,2.0,2.00,0.0,0.0,4d9d10d463892e20a6b870aecbc28748
1,2017,1,9451,Matthew Stafford,Matthew Stafford (LAR),1,29,41,nan,292,208,11,2,2,1,0,1,3,15,9,2,2,3,113.0,27.1,nan,4.0,7.30,0.0,0.0,e9c2c122b79d6b6a9ef399cdaa2686e1
7,2017,1,12092,Derek Carr,Derek Carr (FA),1,22,32,nan,262,134,13,3,0,0,0,2,1,2,17,3,4,4,114.0,18.7,nan,3.0,4.80,0.0,0.0,474ab5478a285f40c3587ee8aa2bd722


In [62]:
# Enhanced data integrity checks
# output: Advanced Stat DataFrame with a hash column

# Generate hashes for the Advanced Stats DataFrame
dataframes["Advanced Stats DataFrame"]["row_hash"] = dataframes["Advanced Stats DataFrame"].apply(generate_md5_hash, axis=1)

# Display sample hashes
print("\n✅ Advanced Stats DataFrame - Hash Generation Complete")
display(dataframes["Advanced Stats DataFrame"].head(5))


✅ Advanced Stats DataFrame - Hash Generation Complete


,Season,Week,FantasyPros_ID,Player,PlayerTM,G,COMP,ATT,PCT,YDS,AIR,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS,SACK,KNCK,HRRY,BLITZ,POOR,DROP,RZ ATT,RTG,row_hash
0,2017,1,9012,Alex Smith,Alex Smith (FA),1,28,35,nan,368,182,10,3,2,2,2,3,1,6,7,2,0,2,149.0,98d3ea214b141d56fad385fc3a22c413
1,2017,1,9451,Matthew Stafford,Matthew Stafford (LAR),1,29,41,nan,292,208,11,2,2,1,0,1,3,15,9,2,2,3,113.0,5145d9cf3962aa80dbf466bc184490dd
2,2017,1,9678,Sam Bradford,Sam Bradford (FA),1,27,32,nan,346,221,12,8,3,1,0,1,2,6,10,2,1,4,143.0,11d2fb189014e069f4e9f98d3c1a27c5
3,2017,1,14142,Trevor Siemian,Trevor Siemian (FA),1,17,28,nan,219,147,9,4,1,1,0,4,4,6,5,4,2,3,94.0,f8450e718d77a80fd18f532b6a26ae21
4,2017,1,15520,Carson Wentz,Carson Wentz (FA),1,26,39,nan,307,190,14,4,2,1,1,2,6,6,22,3,1,3,97.0,955855604bd98a514a94d475cb9fb695


In [63]:
# Enhanced data integrity checks
# output: FPTS & ROST DataFrame with a hash column

# Generate hashes for the FPTS & ROST DataFrame
dataframes["FPTS & ROST DataFrame"]["row_hash"] = dataframes["FPTS & ROST DataFrame"].apply(generate_md5_hash, axis=1)

# Display sample hashes
print("\n✅ FPTS & ROST DataFrame - Hash Generation Complete")
display(dataframes["FPTS & ROST DataFrame"].head(5))


✅ FPTS & ROST DataFrame - Hash Generation Complete


,Season,Week,FantasyPros_ID,FPTS,ROST,row_hash
0,2017,1,9012,31.0,nan,a28d717832ac1aaedada4c7378c6056f
1,2017,1,9451,27.1,nan,9fb3762f7c92b851663d0c73cb98b2dd
2,2017,1,9678,25.5,nan,33a74639804b5c2bc18745f4c806b287
3,2017,1,14142,22.7,nan,baecc84f0a52210ae2c0425a8d1a7329
4,2017,1,15520,18.9,nan,ce19d7163613292fb0707423ddb03c5c


In [64]:
# Enhanced data integrity checks
# output: Redzone Stats DataFrame with a hash column

# Generate hashes for the Redzone Stats DataFrame
dataframes["Redzone Stats DataFrame"]["row_hash"] = dataframes["Redzone Stats DataFrame"].apply(generate_md5_hash, axis=1)

# Display sample hashes
print("\n✅ Redzone Stats DataFrame - Hash Generation Complete")
display(dataframes["Redzone Stats DataFrame"].head(5))


✅ Redzone Stats DataFrame - Hash Generation Complete


,Season,Week,FantasyPros_ID,COMP_rz,Y/A_rz,INT_rz,SACKS_rz,row_hash
0,2017,1,14142,4,6.20,0,2,c7662a5cf6fd833460e5fd820cd1cd27
1,2017,1,9678,6,5.10,0,0,c087722b6972a9c849847efef6c497ae
2,2017,1,9451,4,7.30,0,0,f245c6f9ceb8656d16510884dd7a9247
3,2017,1,16389,2,3.30,0,1,7ae15f9a504d48fe97675df6d2e7aebb
4,2017,1,10007,3,1.20,1,0,acb9afdce29c68638f280b8c524f86c4


In [65]:
# ✅ Enhanced data integrity checks
# ✅ Output: renamed the merged dataframe

# Create a copy of the merged dataframe with "_hash" suffix for modification
qb_adv_fpts_rost_rz_merged_df_sorted_hash = dataframes["Merged DataFrame"].copy()

# Verify the creation of the new dataframe
print("\n✅ New Merged DataFrame Created: 'qb_adv_fpts_rost_rz_merged_df_sorted_hash'")


✅ New Merged DataFrame Created: 'qb_adv_fpts_rost_rz_merged_df_sorted_hash'


In [66]:
# ✅ Enhanced data integrity checks
# ✅ Output: columns of merged dataframe with new hash columns

# Define the new columns to be added for hashing
new_hash_columns = ['adv_stats_hash', 'fpts_rost_hash', 'redzone_hash']

# Initialize the new columns with empty strings (or NaN if preferred)
for col in new_hash_columns:
    qb_adv_fpts_rost_rz_merged_df_sorted_hash[col] = ""  # Empty strings as placeholders

# Verify creation of columns
print("\n✅ Hash columns added to the new merged dataframe:")
print(qb_adv_fpts_rost_rz_merged_df_sorted_hash.columns.tolist())


✅ Hash columns added to the new merged dataframe:
['Season', 'Week', 'FantasyPros_ID', 'Player', 'PlayerTM', 'G', 'COMP', 'ATT', 'PCT', 'YDS', 'AIR', '10+ YDS', '20+ YDS', '30+ YDS', '40+ YDS', '50+ YDS', 'SACK', 'KNCK', 'HRRY', 'BLITZ', 'POOR', 'DROP', 'RZ ATT', 'RTG', 'FPTS', 'ROST', 'COMP_rz', 'Y/A_rz', 'INT_rz', 'SACKS_rz', 'row_hash', 'adv_stats_hash', 'fpts_rost_hash', 'redzone_hash']


In [67]:
# ✅ Enhanced data integrity checks
# ✅ Output: last 5 columns of the new merged dataframe with row hashes

# Display the last 5 columns to confirm the new hash columns are present
print("\n🔍 Last 5 Columns in the New Merged DataFrame:")
print(qb_adv_fpts_rost_rz_merged_df_sorted_hash.columns[-5:])


🔍 Last 5 Columns in the New Merged DataFrame:
Index(['SACKS_rz', 'row_hash', 'adv_stats_hash', 'fpts_rost_hash', 'redzone_hash'], dtype='object')


In [68]:
# ✅ Enhanced data integrity checks - define relevant columns for each QB hash type
# ✅ Output: columns to be hashed

# Define the columns to be used for each hash type
hash_column_groups = {
    "adv_stats_hash": [
        'Season', 'Week', 'FantasyPros_ID', 'COMP', 'ATT', 'PCT', 'YDS', 'AIR',
        'SACK', 'KNCK', 'HRRY', 'DROP', 'RZ ATT', 'RTG'
    ],
    "fpts_rost_hash": [
        'Season', 'Week', 'FantasyPros_ID', 'FPTS', 'ROST'
    ],
    "redzone_hash": [
        'Season', 'Week', 'FantasyPros_ID', 'COMP_rz', 'Y/A_rz', 'INT_rz', 'SACKS_rz'
    ]
}

# Confirm the column groups are set correctly
print("\n✅ Hash Column Groups Defined:")
for hash_name, columns in hash_column_groups.items():
    print(f"  - {hash_name}: {columns}")


✅ Hash Column Groups Defined:
  - adv_stats_hash: ['Season', 'Week', 'FantasyPros_ID', 'COMP', 'ATT', 'PCT', 'YDS', 'AIR', 'SACK', 'KNCK', 'HRRY', 'DROP', 'RZ ATT', 'RTG']
  - fpts_rost_hash: ['Season', 'Week', 'FantasyPros_ID', 'FPTS', 'ROST']
  - redzone_hash: ['Season', 'Week', 'FantasyPros_ID', 'COMP_rz', 'Y/A_rz', 'INT_rz', 'SACKS_rz']


In [69]:
# ✅ Enhanced data integrity checks - view a sample of the hashes
# ✅ Output: preview of row hashes in the merged dataframe

# Apply hashing for each hash type using the existing generate_md5_hash function
for hash_name, columns in hash_column_groups.items():
    qb_adv_fpts_rost_rz_merged_df_sorted_hash[hash_name] = qb_adv_fpts_rost_rz_merged_df_sorted_hash[columns].apply(generate_md5_hash, axis=1)

# Confirm hashes were generated
print("\n✅ Hashes Generated for All Hash Types.")

# Display the first 5 rows of the hash columns for verification
display(qb_adv_fpts_rost_rz_merged_df_sorted_hash[['adv_stats_hash', 'fpts_rost_hash', 'redzone_hash']].head())


✅ Hashes Generated for All Hash Types.


,adv_stats_hash,fpts_rost_hash,redzone_hash
0,16d46c5ab853d5f18461dc14dea68fbb,a28d717832ac1aaedada4c7378c6056f,d72bae53ede0c5b9ff758fa520efc8ad
2,4893d2375bd362ba53d04e44ea858ab2,33a74639804b5c2bc18745f4c806b287,d37b0a0852b926d8207d50e456bd244d
4,a7c124dfd10ccfc8629314cd3b997275,ce19d7163613292fb0707423ddb03c5c,59b775ec5c5217719cba5e0ae7089c87
1,8e734cdc55a0fd8f34f284d53c1cb013,9fb3762f7c92b851663d0c73cb98b2dd,f4fc4b5fea8b5f30be87d13e02d07a43
7,7889d3495f922f77eae63b50ac55773e,0173aa34dd590231dd40b3209798b7ca,d2410a3e5a29ada128aaafd2dae03dfa


In [70]:
# ✅ Final fantasypros merged dataframe

# ✅ Enhanced data integrity checks - view a sample of the hashes
# ✅ Output: preview of row hashes in the merged dataframe

# Display the first 5 rows of the merged dataframe for verification
print("\n🔍 First 5 Rows of the Merged DataFrame:")
display(qb_adv_fpts_rost_rz_merged_df_sorted_hash.head(5))


🔍 First 5 Rows of the Merged DataFrame:


,Season,Week,FantasyPros_ID,Player,PlayerTM,G,COMP,ATT,PCT,YDS,AIR,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS,SACK,KNCK,HRRY,BLITZ,POOR,DROP,RZ ATT,RTG,FPTS,ROST,COMP_rz,Y/A_rz,INT_rz,SACKS_rz,row_hash,adv_stats_hash,fpts_rost_hash,redzone_hash
0,2017,1,9012,Alex Smith,Alex Smith (FA),1,28,35,nan,368,182,10,3,2,2,2,3,1,6,7,2,0,2,149.0,31.0,nan,4.0,3.20,0.0,0.0,7d7821bd1f3ef5e1fe814f092cf8a66c,16d46c5ab853d5f18461dc14dea68fbb,a28d717832ac1aaedada4c7378c6056f,d72bae53ede0c5b9ff758fa520efc8ad
2,2017,1,9678,Sam Bradford,Sam Bradford (FA),1,27,32,nan,346,221,12,8,3,1,0,1,2,6,10,2,1,4,143.0,25.5,nan,6.0,5.10,0.0,0.0,a18cb95a899433f729df5cab51fe548b,4893d2375bd362ba53d04e44ea858ab2,33a74639804b5c2bc18745f4c806b287,d37b0a0852b926d8207d50e456bd244d
4,2017,1,15520,Carson Wentz,Carson Wentz (FA),1,26,39,nan,307,190,14,4,2,1,1,2,6,6,22,3,1,3,97.0,18.9,nan,2.0,2.00,0.0,0.0,4d9d10d463892e20a6b870aecbc28748,a7c124dfd10ccfc8629314cd3b997275,ce19d7163613292fb0707423ddb03c5c,59b775ec5c5217719cba5e0ae7089c87
1,2017,1,9451,Matthew Stafford,Matthew Stafford (LAR),1,29,41,nan,292,208,11,2,2,1,0,1,3,15,9,2,2,3,113.0,27.1,nan,4.0,7.30,0.0,0.0,e9c2c122b79d6b6a9ef399cdaa2686e1,8e734cdc55a0fd8f34f284d53c1cb013,9fb3762f7c92b851663d0c73cb98b2dd,f4fc4b5fea8b5f30be87d13e02d07a43
7,2017,1,12092,Derek Carr,Derek Carr (FA),1,22,32,nan,262,134,13,3,0,0,0,2,1,2,17,3,4,4,114.0,18.7,nan,3.0,4.80,0.0,0.0,474ab5478a285f40c3587ee8aa2bd722,7889d3495f922f77eae63b50ac55773e,0173aa34dd590231dd40b3209798b7ca,d2410a3e5a29ada128aaafd2dae03dfa


In [71]:
# ✅ Enhanced data integrity checks - Generate Row Hashes for Advanced Stats DataFrame
# ✅ Output: row hashes for the Advanced Stats DataFrame

# Define columns used for generating 'adv_stats_hash' (QB-specific)
adv_stats_columns = [
    'Season', 'Week', 'FantasyPros_ID', 'COMP', 'ATT', 'PCT', 'YDS', 'AIR',
    'SACK', 'KNCK', 'HRRY', 'DROP', 'RZ ATT', 'RTG'
]

# Ensure all columns are present in the dataframe
adv_stats_columns = [col for col in adv_stats_columns if col in dataframes["Advanced Stats DataFrame"].columns]

# Generate hashes for the Advanced Stats DataFrame
print("\n📊 Generating Hashes for: Advanced Stats DataFrame")
dataframes["Advanced Stats DataFrame"]["row_hash"] = dataframes["Advanced Stats DataFrame"][adv_stats_columns].apply(generate_md5_hash, axis=1)

# Display a preview to confirm hashes were generated
print("Preview of Generated Hashes in Advanced Stats DataFrame:")
display(dataframes["Advanced Stats DataFrame"][['Season', 'Week', 'FantasyPros_ID', 'row_hash']].head())


📊 Generating Hashes for: Advanced Stats DataFrame
Preview of Generated Hashes in Advanced Stats DataFrame:


,Season,Week,FantasyPros_ID,row_hash
0,2017,1,9012,16d46c5ab853d5f18461dc14dea68fbb
1,2017,1,9451,8e734cdc55a0fd8f34f284d53c1cb013
2,2017,1,9678,4893d2375bd362ba53d04e44ea858ab2
3,2017,1,14142,e82cc2e073bebc7fc5ce92ce87d0714c
4,2017,1,15520,a7c124dfd10ccfc8629314cd3b997275


In [72]:
# ✅ Enhanced data integrity checks - Prepare DataFrames for Comparison
# ✅ Output: no output

# Select relevant columns from the Advanced Stats DataFrame
adv_stats_subset = dataframes["Advanced Stats DataFrame"][['Season', 'Week', 'FantasyPros_ID', 'row_hash']]

# Prepare the Merged DataFrame subset for comparison
merged_subset = qb_adv_fpts_rost_rz_merged_df_sorted_hash[['Season', 'Week', 'FantasyPros_ID', 'adv_stats_hash']]

In [73]:
# ✅ Enhanced data integrity checks - Create a merge DataFrame for hash comparison
# ✅ Output: a merged dataframe showing both hashes

# Merge the dataframes on the keys - JOIN ON FantasyPros_ID
comparison_df = pd.merge(
    merged_subset,
    adv_stats_subset,
    on=['Season', 'Week', 'FantasyPros_ID'],
    how='inner',
    suffixes=('_merged', '_original')
)

# Display the merged comparison dataframe
print("Comparison: adv_stats_hash column (qb_adv_fpts_rost_rz_merged_df_sorted_hash)")
print("Comparison: row_hash column (Advanced Stats DataFrame)\n")
display(comparison_df.head())

Comparison: adv_stats_hash column (qb_adv_fpts_rost_rz_merged_df_sorted_hash)
Comparison: row_hash column (Advanced Stats DataFrame)



,Season,Week,FantasyPros_ID,adv_stats_hash,row_hash
0,2017,1,9012,16d46c5ab853d5f18461dc14dea68fbb,16d46c5ab853d5f18461dc14dea68fbb
1,2017,1,9678,4893d2375bd362ba53d04e44ea858ab2,4893d2375bd362ba53d04e44ea858ab2
2,2017,1,15520,a7c124dfd10ccfc8629314cd3b997275,a7c124dfd10ccfc8629314cd3b997275
3,2017,1,9451,8e734cdc55a0fd8f34f284d53c1cb013,8e734cdc55a0fd8f34f284d53c1cb013
4,2017,1,12092,7889d3495f922f77eae63b50ac55773e,7889d3495f922f77eae63b50ac55773e


In [74]:
# ✅ Enhanced data integrity checks - Identify Matches and Mismatches
# ✅ Output: hopefully no mismatches

# Identify mismatches where hashes don't align
mismatches = comparison_df[comparison_df['adv_stats_hash'] != comparison_df['row_hash']]

# Display mismatch results
print(f"\n❌ Mismatches Found: {len(mismatches)}")
if len(mismatches) > 0:
    display(mismatches.head())
else:
    print("✅ All hashes match.")


❌ Mismatches Found: 0
✅ All hashes match.


In [75]:
# ✅ Enhanced data integrity checks - Generate Row Hashes for FPTS & ROST DataFrame
# ✅ Output: Row hashes for FPTS & ROST DataFrame

# Display current dataframe being processed
print("\n📊 Processing: FPTS & ROST DataFrame")

# Generate row hashes for FPTS & ROST DataFrame using the existing helper function
dataframes["FPTS & ROST DataFrame"]["row_hash"] = dataframes["FPTS & ROST DataFrame"][['Season', 'Week', 'FantasyPros_ID', 'FPTS', 'ROST']].apply(generate_md5_hash, axis=1)

# Display a preview of the generated hashes
display(dataframes["FPTS & ROST DataFrame"][['Season', 'Week', 'FantasyPros_ID', 'row_hash']].head())


📊 Processing: FPTS & ROST DataFrame


,Season,Week,FantasyPros_ID,row_hash
0,2017,1,9012,a28d717832ac1aaedada4c7378c6056f
1,2017,1,9451,9fb3762f7c92b851663d0c73cb98b2dd
2,2017,1,9678,33a74639804b5c2bc18745f4c806b287
3,2017,1,14142,baecc84f0a52210ae2c0425a8d1a7329
4,2017,1,15520,ce19d7163613292fb0707423ddb03c5c


In [76]:
# ✅ Enhanced data integrity checks - Check Matching IDs Before Comparison
# ✅ Output: Number of matching FantasyPros_IDs between FPTS & ROST and merged QB dataframe

# Checking Matching IDs Between the DataFrames
print("\n📊 Checking Matching IDs Before Comparison:")

# Extract IDs from both dataframes
ids_unmerged = dataframes["FPTS & ROST DataFrame"]["FantasyPros_ID"].unique()
ids_merged = qb_adv_fpts_rost_rz_merged_df_sorted_hash["FantasyPros_ID"].unique()

# Find intersection of IDs
matching_ids = set(ids_unmerged).intersection(set(ids_merged))
print(f"🔍 Total Matching IDs Found: {len(matching_ids)}")


📊 Checking Matching IDs Before Comparison:
🔍 Total Matching IDs Found: 92


In [77]:
# ✅ Enhanced data integrity checks - Compare Hashes 
# ✅ Output: Hopefully no mismatches

# Perform comparison by joining dataframes on keys: Season, Week, FantasyPros_ID
comparison_df = pd.merge(
    qb_adv_fpts_rost_rz_merged_df_sorted_hash[['Season', 'Week', 'FantasyPros_ID', 'fpts_rost_hash']],
    dataframes["FPTS & ROST DataFrame"][['Season', 'Week', 'FantasyPros_ID', 'row_hash']],
    on=['Season', 'Week', 'FantasyPros_ID'],
    how='inner',
    suffixes=('_merged', '_unmerged')
)

# Identify mismatches
mismatches = comparison_df[comparison_df['fpts_rost_hash'] != comparison_df['row_hash']]

# Display comparison results
if mismatches.empty:
    print("\n✅ All hashes match for FPTS & ROST DataFrame.")
else:
    print(f"\n❌ Mismatches Found: {len(mismatches)}")
    display(mismatches.head())

display(comparison_df.head())


✅ All hashes match for FPTS & ROST DataFrame.


,Season,Week,FantasyPros_ID,fpts_rost_hash,row_hash
0,2017,1,9012,a28d717832ac1aaedada4c7378c6056f,a28d717832ac1aaedada4c7378c6056f
1,2017,1,9678,33a74639804b5c2bc18745f4c806b287,33a74639804b5c2bc18745f4c806b287
2,2017,1,15520,ce19d7163613292fb0707423ddb03c5c,ce19d7163613292fb0707423ddb03c5c
3,2017,1,9451,9fb3762f7c92b851663d0c73cb98b2dd,9fb3762f7c92b851663d0c73cb98b2dd
4,2017,1,12092,0173aa34dd590231dd40b3209798b7ca,0173aa34dd590231dd40b3209798b7ca


In [78]:
## Fixme: **No row hash validation for the redzone data**
# ✅ Enhanced data integrity checks - Compare Hashes for Redzone Data
# ✅ Output: Display mismatches if found

# Perform comparison by joining dataframes on keys: Season, Week, FantasyPros_ID
comparison_df_redzone = pd.merge(
    qb_adv_fpts_rost_rz_merged_df_sorted_hash[['Season', 'Week', 'FantasyPros_ID', 'redzone_hash']],
    dataframes["Redzone Stats DataFrame"][['Season', 'Week', 'FantasyPros_ID', 'row_hash']],
    on=['Season', 'Week', 'FantasyPros_ID'],
    how='inner',
    suffixes=('_merged', '_unmerged')
)

# Identify mismatches
mismatches_redzone = comparison_df_redzone[comparison_df_redzone['redzone_hash'] != comparison_df_redzone['row_hash']]

# Display comparison results
if mismatches_redzone.empty:
    print("\n✅ All hashes match for Redzone Stats DataFrame.")
else:
    print(f"\n❌ Mismatches Found: {len(mismatches_redzone)}")
    display(mismatches_redzone.head())

# Display the first few rows of the comparison dataframe for reference
display(comparison_df_redzone.head())


❌ Mismatches Found: 1115


,Season,Week,FantasyPros_ID,redzone_hash,row_hash
0,2017,1,9012,d72bae53ede0c5b9ff758fa520efc8ad,5b890a20a9f0c6d09c5040b6ef419384
1,2017,1,9678,d37b0a0852b926d8207d50e456bd244d,c087722b6972a9c849847efef6c497ae
2,2017,1,15520,59b775ec5c5217719cba5e0ae7089c87,11ca7882c88a96f77b2d891a122c1c59
3,2017,1,9451,f4fc4b5fea8b5f30be87d13e02d07a43,f245c6f9ceb8656d16510884dd7a9247
4,2017,1,12092,d2410a3e5a29ada128aaafd2dae03dfa,0b49ffe6da6790944e171bffd5a88299


,Season,Week,FantasyPros_ID,redzone_hash,row_hash
0,2017,1,9012,d72bae53ede0c5b9ff758fa520efc8ad,5b890a20a9f0c6d09c5040b6ef419384
1,2017,1,9678,d37b0a0852b926d8207d50e456bd244d,c087722b6972a9c849847efef6c497ae
2,2017,1,15520,59b775ec5c5217719cba5e0ae7089c87,11ca7882c88a96f77b2d891a122c1c59
3,2017,1,9451,f4fc4b5fea8b5f30be87d13e02d07a43,f245c6f9ceb8656d16510884dd7a9247
4,2017,1,12092,d2410a3e5a29ada128aaafd2dae03dfa,0b49ffe6da6790944e171bffd5a88299


In [79]:
### End: Enhanced data integrity checks for the fantasypros dataframes ##

In [80]:
## Begin: webscraping sportsoddshistory betting lines data ###

In [81]:
# Generate Dynamic URLs for the regular season 
# output: urls with each year to scrape

# Helper: Define year range
def get_year_range(current_year, current_week):
    return list(range(2017, current_year + 1)) if current_week >= 1 else list(range(2017, current_year))

# Current time context
current_year = datetime.now().year
current_week = datetime.now().isocalendar()[1]

# Get the range and build the season URLs
season_list = get_year_range(current_year, current_week)
season_urls = [f"https://www.sportsoddshistory.com/nfl-game-season/?y={year}" for year in season_list]

# Preview
for season, url in zip(season_list, season_urls):
    print(f"{season}: {url}")

2017: https://www.sportsoddshistory.com/nfl-game-season/?y=2017
2018: https://www.sportsoddshistory.com/nfl-game-season/?y=2018
2019: https://www.sportsoddshistory.com/nfl-game-season/?y=2019
2020: https://www.sportsoddshistory.com/nfl-game-season/?y=2020
2021: https://www.sportsoddshistory.com/nfl-game-season/?y=2021
2022: https://www.sportsoddshistory.com/nfl-game-season/?y=2022
2023: https://www.sportsoddshistory.com/nfl-game-season/?y=2023
2024: https://www.sportsoddshistory.com/nfl-game-season/?y=2024
2025: https://www.sportsoddshistory.com/nfl-game-season/?y=2025


In [82]:
# Load Soup and Extract Regular Season Week Headers
# output: regular season years

# Container for (season, soup, regular season week headers)
tm_game_odds_season_soup_headers = []

for season, url in zip(season_list, season_urls):
    print(f"🔄 Fetching {season}...")
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract all <h3> tags
    all_headers = soup.find_all('h3')

    # Only keep regular season week headers (e.g., "2023 Regular Season - Week 1")
    reg_week_headers = [tag for tag in all_headers if f"{season} Regular Season - Week" in tag.text]

    # Store tuple for next stage
    tm_game_odds_season_soup_headers.append((season, soup, reg_week_headers))

🔄 Fetching 2017...
🔄 Fetching 2018...
🔄 Fetching 2019...
🔄 Fetching 2020...
🔄 Fetching 2021...
🔄 Fetching 2022...
🔄 Fetching 2023...
🔄 Fetching 2024...
🔄 Fetching 2025...


In [83]:
# Scrape All Weeks for Each Year
# output: none

# Container to collect team-level weekly DataFrames
tm_game_odds_regular_team_level_list = []

for season, soup, week_headers in tm_game_odds_season_soup_headers:
    for tag in week_headers:
        week = int(tag.text.strip().split("Week")[1])

        table = tag.find_next('table')
        if not table:
            continue
        rows = table.find('tbody').find_all('tr')

        base_data = []
        abbr_data = []

        for row in rows:
            cells = row.find_all('td')
            if len(cells) != 11:
                continue

            base_data.append([cell.get_text(strip=True) for cell in cells])

            fav_link = cells[4].find('a')
            und_link = cells[8].find('a')

            fav_abbr = fav_link['href'].split('=')[1].split('&')[0] if fav_link else None
            und_abbr = und_link['href'].split('=')[1].split('&')[0] if und_link else None

            abbr_data.append([fav_abbr, und_abbr])

        if not base_data:
            continue

        df = pd.DataFrame(base_data, columns=[
            'Day', 'Date', 'Time (ET)', 'Home_Away', 'Favorite', 'Score', 'Spread', '',
            'Underdog', 'Over/Under', 'Notes'
        ])
        df.drop(columns=[''], inplace=True)

        abbr_df = pd.DataFrame(abbr_data, columns=['Favorite_Abbr', 'Underdog_Abbr'])
        df = pd.concat([df, abbr_df], axis=1)
        df['season'] = season
        df['week'] = week

        # Expand to team-level format
        team_rows = []
        for _, row in df.iterrows():
            home_away = row['Home_Away']
            score = row['Score']

            team_rows.extend([
                {
                    'season': row['season'], 'week': row['week'],
                    'team_abbr': row['Favorite_Abbr'],
                    'opponent_abbr': row['Underdog_Abbr'],
                    'home': home_away != '@', 'role': 'Favorite',
                    'result': score.split()[0] if score else None,
                    'score': score.split()[1] if score else None,
                    'spread': row['Spread'], 'over_under': row['Over/Under']
                },
                {
                    'season': row['season'], 'week': row['week'],
                    'team_abbr': row['Underdog_Abbr'],
                    'opponent_abbr': row['Favorite_Abbr'],
                    'home': home_away == '@', 'role': 'Underdog',
                    'result': None, 'score': None,
                    'spread': row['Spread'], 'over_under': row['Over/Under']
                }
            ])

        weekly_df = pd.DataFrame(team_rows)

        # Clean the spread column but DO NOT normalize abbreviations yet
        weekly_df['spread'] = weekly_df['spread'].str.replace(r'^P.*', '0', regex=True)
        weekly_df['spread'] = weekly_df['spread'].str.replace(r'^[WL]\s*', '', regex=True)
        weekly_df['spread'] = pd.to_numeric(weekly_df['spread'].str.strip(), errors='coerce')

        tm_game_odds_regular_team_level_list.append(weekly_df)

In [84]:
# Combine and Export the Regular Season Data
# output: csv file of regular season odds lines

# Combine all regular season data
tm_game_odds_regular_team_level = pd.concat(
    tm_game_odds_regular_team_level_list,
    ignore_index=True
)

# Preview dimensions
print(f"📊 Total rows collected: {tm_game_odds_regular_team_level.shape[0]}")
print(f"✅ Columns: {list(tm_game_odds_regular_team_level.columns)}")

# Export for inspection
# tm_game_odds_regular_team_level.to_csv("tm_game_odds_regular_season.csv", index=False)
print("📝 Exported to tm_game_odds_regular_season.csv")

📊 Total rows collected: 4222
✅ Columns: ['season', 'week', 'team_abbr', 'opponent_abbr', 'home', 'role', 'result', 'score', 'spread', 'over_under']
📝 Exported to tm_game_odds_regular_season.csv


In [85]:
# extract playoff <h3> tags from each season and pair them with the corresponding soup object
# output: a list of tuples

# Identify playoff <h3> tags for each season 
playoff_h3_tags = []

for season, url in zip(season_list, season_urls):
    print(f"🔄 Checking {season} playoffs...")

    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    tag = soup.find('h3', string=f"{season} Playoffs")
    
    if tag:
        playoff_h3_tags.append((season, soup, tag))
    else:
        print(f"⚠️ No playoff tag found for {season}")

🔄 Checking 2017 playoffs...
🔄 Checking 2018 playoffs...
🔄 Checking 2019 playoffs...
🔄 Checking 2020 playoffs...
🔄 Checking 2021 playoffs...
🔄 Checking 2022 playoffs...
🔄 Checking 2023 playoffs...
🔄 Checking 2024 playoffs...
🔄 Checking 2025 playoffs...
⚠️ No playoff tag found for 2025


In [86]:
# extract all <tr> rows from the playoff table corresponding to each <h3> tag
# output: a list of tuples: (season, [row1, row2, ..., rowN])

# Extract playoff table rows from each <h3> tag
playoff_table_rows = []

for season, soup, tag in playoff_h3_tags:
    print(f"📅 Extracting table rows for {season} playoffs...")
    
    try:
        table = tag.find_next('table')
        rows = table.find('tbody').find_all('tr')
        playoff_table_rows.append((season, rows))
    except Exception as e:
        print(f"⚠️ {season} - Failed to extract table rows: {e}")

📅 Extracting table rows for 2017 playoffs...
📅 Extracting table rows for 2018 playoffs...
📅 Extracting table rows for 2019 playoffs...
📅 Extracting table rows for 2020 playoffs...
📅 Extracting table rows for 2021 playoffs...
📅 Extracting table rows for 2022 playoffs...
📅 Extracting table rows for 2023 playoffs...
📅 Extracting table rows for 2024 playoffs...


In [87]:
# Parse each playoff row into game-level dictionaries
# output: dataframe of the playoff games

# Parse <tr> rows into dictionaries with playoff data
def parse_playoff_row(row):
    try:
        cells = row.find_all('td')
        if len(cells) < 11:
            return None  # Skip malformed rows

        # Remove seed info from names (e.g., "BUF (3)" → "BUF")
        favorite_raw = cells[5].get_text(strip=True).rsplit('(', 1)[0].strip()
        underdog_raw = cells[9].get_text(strip=True).rsplit('(', 1)[0].strip()

        return {
            'Round': cells[0].get_text(strip=True),
            'Day': cells[1].get_text(strip=True),
            'Date': cells[2].get_text(strip=True),
            'Time (ET)': cells[3].get_text(strip=True),
            'Home_Away': cells[4].get_text(strip=True),
            'Favorite': favorite_raw,
            'Score': cells[6].get_text(strip=True),
            'Spread': cells[7].get_text(strip=True),
            'Underdog': underdog_raw,
            'Over/Under': cells[10].get_text(strip=True)
        }

    except Exception as e:
        print(f"⚠️ Error parsing row: {e}")
        return None

# Apply the parser
parsed_playoff_games = []

for season, rows in playoff_table_rows:
    parsed = [parse_playoff_row(row) for row in rows]
    parsed = [game for game in parsed if game is not None]
    for game in parsed:
        game['season'] = season  # Tag season
    parsed_playoff_games.extend(parsed)

# Preview the first few
pd.DataFrame(parsed_playoff_games).head()

,Round,Day,Date,Time (ET),Home_Away,Favorite,Score,Spread,Underdog,Over/Under,season
0,AFC Wild Card,Sat,"Jan 6, 2018",4:35,@,Kansas City Chiefs,L 21-22,L -8.5,Tennessee Titans,U 44,2017
1,NFC Wild Card,Sat,"Jan 6, 2018",8:15,@,Los Angeles Rams,L 13-26,L -6.5,Atlanta Falcons,U 48,2017
2,AFC Wild Card,Sun,"Jan 7, 2018",1:05,@,Jacksonville Jaguars,W 10-3,L -8,Buffalo Bills,U 39.5,2017
3,NFC Wild Card,Sun,"Jan 7, 2018",4:40,@,New Orleans Saints,W 31-26,L -6.5,Carolina Panthers,O 47.5,2017
4,NFC Divisional,Sat,"Jan 13, 2018",4:35,,Atlanta Falcons,L 10-15,L -2.5,Philadelphia Eagles,U 40,2017


In [88]:
# Convert parsed playoff games into team-level rows
# output: dataframe of the playoff games with numbered weeks

# Helper to map playoff round name → week offset
def get_playoff_week(round_name, season):
    base_week = 19 if season >= 2021 else 18
    round_name = round_name.lower()
    if "wild card" in round_name:
        return base_week
    elif "divisional" in round_name:
        return base_week + 1
    elif "championship" in round_name:
        return base_week + 2
    elif "super bowl" in round_name:
        return base_week + 3
    return None  # fallback

# Convert each game into two team rows
team_level_rows = []

for game in parsed_playoff_games:
    week = get_playoff_week(game['Round'], game['season'])

    team_level_rows.append({
        'season': game['season'],
        'week': week,
        'team_abbr': game['Favorite'],
        'opponent_abbr': game['Underdog'],
        'home': game['Home_Away'] != '@',
        'role': 'Favorite',
        'result': game['Score'].split()[0] if game['Score'] else None,
        'score': game['Score'].split()[1] if game['Score'] else None,
        'spread': game['Spread'],
        'over_under': game['Over/Under']
    })

    team_level_rows.append({
        'season': game['season'],
        'week': week,
        'team_abbr': game['Underdog'],
        'opponent_abbr': game['Favorite'],
        'home': game['Home_Away'] == '@',
        'role': 'Underdog',
        'result': None,
        'score': None,
        'spread': game['Spread'],
        'over_under': game['Over/Under']
    })

# Build team-level playoff DataFrame
tm_game_odds_playoffs_team_level = pd.DataFrame(team_level_rows)

# Clean 'spread' values
tm_game_odds_playoffs_team_level['spread'] = (
    tm_game_odds_playoffs_team_level['spread']
    .str.replace(r'^P.*', '0', regex=True)
    .str.replace(r'^[WL]\s*', '', regex=True)
    .str.strip()
)
tm_game_odds_playoffs_team_level['spread'] = pd.to_numeric(
    tm_game_odds_playoffs_team_level['spread'], errors='coerce'
)

# Preview
display(tm_game_odds_playoffs_team_level.head())

,season,week,team_abbr,opponent_abbr,home,role,result,score,spread,over_under
0,2017,18,Kansas City Chiefs,Tennessee Titans,False,Favorite,L,21-22,-8.5,U 44
1,2017,18,Tennessee Titans,Kansas City Chiefs,True,Underdog,None,None,-8.5,U 44
2,2017,18,Los Angeles Rams,Atlanta Falcons,False,Favorite,L,13-26,-6.5,U 48
3,2017,18,Atlanta Falcons,Los Angeles Rams,True,Underdog,None,None,-6.5,U 48
4,2017,18,Jacksonville Jaguars,Buffalo Bills,False,Favorite,W,10-3,-8.0,U 39.5


In [89]:
# Merge Regular Season + Playoffs into One DataFrame
# output: combined dataframe of regular season and playoff odds and lines *(not final)
# output: csv file for inspection only *not final

# Combine regular season and playoff team-level rows
tm_game_odds_all_games = pd.concat(
    [tm_game_odds_regular_team_level, tm_game_odds_playoffs_team_level],
    ignore_index=True
)

# Optional: sort for easier inspection
tm_game_odds_all_games.sort_values(by=['season', 'week'], inplace=True)

# Export final combined dataset
# tm_game_odds_all_games.to_csv("tm_game_odds_all_games.csv", index=False)

# Preview
print("✅ Combined all regular season and playoff games.")
print(f"📊 Final shape: {tm_game_odds_all_games.shape}")
display(tm_game_odds_all_games.head())

✅ Combined all regular season and playoff games.
📊 Final shape: (4418, 10)


,season,week,team_abbr,opponent_abbr,home,role,result,score,spread,over_under
0,2017,1,NEP,KCC,False,Favorite,L,27-42,-8.0,O 47.5
1,2017,1,KCC,NEP,True,Underdog,None,None,-8.0,O 47.5
2,2017,1,BUF,NYJ,False,Favorite,W,21-12,-7.0,U 42
3,2017,1,NYJ,BUF,True,Underdog,None,None,-7.0,U 42
4,2017,1,ATL,CHI,True,Favorite,W,23-17,-6.5,U 48


In [90]:
# standardize team abbreviations
# output: none

# Create One Unified Map
# Canonical full team name → abbreviation
full_name_to_abbr = {
    'Arizona Cardinals': 'ARI', 'Atlanta Falcons': 'ATL', 'Baltimore Ravens': 'BAL',
    'Buffalo Bills': 'BUF', 'Carolina Panthers': 'CAR', 'Chicago Bears': 'CHI',
    'Cincinnati Bengals': 'CIN', 'Cleveland Browns': 'CLE', 'Dallas Cowboys': 'DAL',
    'Denver Broncos': 'DEN', 'Detroit Lions': 'DET', 'Green Bay Packers': 'GB',
    'Houston Texans': 'HOU', 'Indianapolis Colts': 'IND', 'Jacksonville Jaguars': 'JAX',
    'Kansas City Chiefs': 'KC', 'Las Vegas Raiders': 'LV', 'Los Angeles Chargers': 'LAC',
    'Los Angeles Rams': 'LA', 'Miami Dolphins': 'MIA', 'Minnesota Vikings': 'MIN',
    'New England Patriots': 'NE', 'New Orleans Saints': 'NO', 'New York Giants': 'NYG',
    'New York Jets': 'NYJ', 'Philadelphia Eagles': 'PHI', 'Pittsburgh Steelers': 'PIT',
    'San Francisco 49ers': 'SF', 'Seattle Seahawks': 'SEA', 'Tampa Bay Buccaneers': 'TB',
    'Tennessee Titans': 'TEN', 'Washington Commanders': 'WAS', 'Washington Football Team': 'WAS'

}

# Abbreviation fix-ups
abbr_fix_map = {
    'ARZ': 'ARI', 'TBB': 'TB', 'NEP': 'NE', 'GBP': 'GB', 'KCC': 'KC',
    'SFF': 'SF', 'LAR': 'LA', 'NOS': 'NO', 'JAC': 'JAX', 'LVR': 'LV'
}

In [91]:
# apply the correct team abbreviations and rename the dataframe
# output: no output

# Rename to final dataframe name for clarity
tm_reg_playoffs_game_lines_odds = tm_game_odds_all_games.copy()

# Apply full name and abbreviation corrections
tm_reg_playoffs_game_lines_odds['team_abbr'] = (
    tm_reg_playoffs_game_lines_odds['team_abbr']
    .replace(full_name_to_abbr)
    .replace(abbr_fix_map)
)

tm_reg_playoffs_game_lines_odds['opponent_abbr'] = (
    tm_reg_playoffs_game_lines_odds['opponent_abbr']
    .replace(full_name_to_abbr)
    .replace(abbr_fix_map)
)

In [92]:
# ✅ Compare the team abbreviations: python nfl df to the tm odds df - the sets should match!
# ✅ Output: list of team abbreviations

# Canonical set from the base dataset
baseline_nfl_tm_abbr = set(qb_ids_ngs_pfr_stats_sorted['recent_team'].dropna().unique())

# Abbreviation sets from the cleaned odds dataframe
home_odds_lines_abbr = set(tm_reg_playoffs_game_lines_odds['team_abbr'].dropna().unique())
away_odds_lines_abbr = set(tm_reg_playoffs_game_lines_odds['opponent_abbr'].dropna().unique())

# Combined odds abbreviations
odds_abbrs = home_odds_lines_abbr.union(away_odds_lines_abbr)

# Comparison
only_in_qb_ids = baseline_nfl_tm_abbr - odds_abbrs
only_in_odds = odds_abbrs - baseline_nfl_tm_abbr

# Output
print("✅ Unique team_abbr values (home):", sorted(home_odds_lines_abbr))
print()

print("✅ Unique opponent_abbr values (away):", sorted(away_odds_lines_abbr))
print()

print("❌ Abbreviations only in qb_ids_ngs_pfr_stats_sorted:", only_in_qb_ids)
print("❌ Abbreviations only in tm_reg_playoffs_game_lines_odds:", only_in_odds)

✅ Unique team_abbr values (home): ['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL', 'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LA', 'LAC', 'LV', 'MIA', 'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'PHI', 'PIT', 'SEA', 'SF', 'TB', 'TEN', 'WAS']

✅ Unique opponent_abbr values (away): ['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL', 'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LA', 'LAC', 'LV', 'MIA', 'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'PHI', 'PIT', 'SEA', 'SF', 'TB', 'TEN', 'WAS']

❌ Abbreviations only in qb_ids_ngs_pfr_stats_sorted: set()
❌ Abbreviations only in tm_reg_playoffs_game_lines_odds: set()


In [93]:
# **** Final Dataframe of team odds ***

# output: final dataframe of team odds and lines with the csv file
# Save the file
# tm_reg_playoffs_game_lines_odds.to_csv("tm_reg_playoffs_game_lines_odds.csv", index=False)

# Display the first 10 rows
display(tm_reg_playoffs_game_lines_odds.head(10))

,season,week,team_abbr,opponent_abbr,home,role,result,score,spread,over_under
0,2017,1,NE,KC,False,Favorite,L,27-42,-8.0,O 47.5
1,2017,1,KC,NE,True,Underdog,None,None,-8.0,O 47.5
2,2017,1,BUF,NYJ,False,Favorite,W,21-12,-7.0,U 42
3,2017,1,NYJ,BUF,True,Underdog,None,None,-7.0,U 42
4,2017,1,ATL,CHI,True,Favorite,W,23-17,-6.5,U 48
5,2017,1,CHI,ATL,False,Underdog,None,None,-6.5,U 48
6,2017,1,CIN,BAL,False,Favorite,L,0-20,-2.5,U 41.5
7,2017,1,BAL,CIN,True,Underdog,None,None,-2.5,U 41.5
8,2017,1,PIT,CLE,True,Favorite,W,21-18,-10.0,U 47
9,2017,1,CLE,PIT,False,Underdog,None,None,-10.0,U 47


In [94]:
## End: webscraping sportsoddshistory betting lines data ###

In [95]:
## Begin: Build the dataframe for the DFS Fanduel and Draft Kings salary data from BigDataBall ##
# ** Files must be in the local directory ** NFL-20xx-DFS-Dataset.xlsx

In [96]:
# create a helper function to clean the the dfs salary data
# no output

def clean_column_dfs(col):
    """
    Cleans and flattens multi-index column names for DFS salary Excel files:
    - Joins tuples if multi-index
    - Removes special characters
    - Normalizes spaces
    - Converts to lowercase for matching
    """
    if isinstance(col, tuple):
        col = ' '.join(str(x) for x in col if x)

    return (
        str(col)
        .replace('\n', ' ')
        .replace('(', '')
        .replace(')', '')
        .replace('"', '')
        .replace('#', '')
        .replace('$', '')
        .replace('/', '')
        .replace('-', ' ')
        .strip()
        .lower()
        .replace('  ', ' ')
        .replace('   ', ' ')
    )

In [97]:
# Read the excel files
filepath = 'NFL-2024-DFS-Dataset.xlsx'
dfs_raw = pd.read_excel(filepath, header=[0, 1])
original_row_count = len(dfs_raw)

dfs_raw.columns = [clean_column_dfs(col) for col in dfs_raw.columns]
dfs_raw.head()  # Optional preview

,game information bigdataball dataset,game information game id,game information date,game information week,game information start time et,game information player id,game information player dst,game information team,game information opponent,game information venue rh,position draftkings,position fanduel,salary for draftkings classic contests,salary for fanduel full roster contests,fantasy points scored draftkings,fantasy points scored fanduel
0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#N/A MEANING: The game was not included in any...,NaN,NaN,NaN,NaN,NaN
1,NFL 2024 Regular Season,45540-BAL@KAN,2024-09-05,1.0,8:20 PM,BAL,Baltimore Ravens,Baltimore Ravens,Kansas City Chiefs,Road,DST,D,2700,3900.0,4.0,4.0
2,NFL 2024 Regular Season,45540-BAL@KAN,2024-09-05,1.0,8:20 PM,nelson-agholor,Nelson Agholor,Baltimore Ravens,Kansas City Chiefs,Road,WR,WR,3800,4200.0,1.6,1.1
3,NFL 2024 Regular Season,45540-BAL@KAN,2024-09-05,1.0,8:20 PM,derrick-henry,Derrick Henry,Baltimore Ravens,Kansas City Chiefs,Road,RB,RB,6900,7900.0,10.6,10.6
4,NFL 2024 Regular Season,45540-BAL@KAN,2024-09-05,1.0,8:20 PM,patrick-ricard,Patrick Ricard,Baltimore Ravens,Kansas City Chiefs,Road,RB,RB,4000,4100.0,0.0,0.0


In [98]:
# helper function does the following:
# Fanduel and Draft Kings player salary data for all positions (QB, RB, TE, WR, DST)
# creates and combines the dataframes for years 2017 - present 
# performs data validation checks
# output: none

def process_single_year(filepath, year):
    
    # Step 1: Read and clean the headers
    dfs_raw = pd.read_excel(filepath, header=[0, 1])
    original_row_count = len(dfs_raw)
    dfs_raw.columns = [clean_column_dfs(col) for col in dfs_raw.columns]

    # ✅ Step 2: Extract only relevant columns using cleaned names
    expected_cols = {
        'player': 'game information player dst',
        'week': 'game information week',
        'date': 'game information date',
        'player_id': 'game information player id',
        'dk_position': 'position draftkings',
        'fd_position': 'position fanduel',
        'dk_salary': 'salary for draftkings classic contests',
        'fd_salary': 'salary for fanduel full roster contests'
    }

    # Subset the dataframe using cleaned column names
    dfs_subset = dfs_raw[list(expected_cols.values())].copy()

    # Rename them to simple identifiers for internal use
    dfs_subset.columns = list(expected_cols.keys())

    
    dfs_subset['date'] = pd.to_datetime(dfs_subset['date'])

    team_abbreviation_mapping = {
        'NWE': 'NE',
        'SFO': 'SF',
        'OAK': 'LV',
        'KAN': 'KC',
        'TAM': 'TB',
        'NOR': 'NO',
        'LAR': 'LA',
        'GNB': 'GB'
    }
    mask_dst = dfs_subset['dk_position'] == 'DST'
    dfs_subset.loc[mask_dst, 'player_id'] = dfs_subset.loc[mask_dst, 'player_id'].replace(team_abbreviation_mapping)

    def fix_season(row):
        game_year = row['date'].year
        game_month = row['date'].month
        game_week = row['week']
        
        if game_month in [1, 2]:
            if (game_year <= 2020 and game_week >= 18):
                return game_year - 1
            elif (game_year >= 2021 and game_week >= 19):
                return game_year - 1
            elif (game_year >= 2021 and game_week == 18):
                return game_year - 1
        return game_year

    dfs_subset['season'] = dfs_subset.apply(fix_season, axis=1)

    # 🔥 Track NaNs before dropping
    season_nulls_before = dfs_subset['season'].isna().sum()

    dfs_subset = dfs_subset.dropna(subset=['season'])
    dfs_subset['season'] = dfs_subset['season'].astype(int)

    season_nulls_after = dfs_subset['season'].isna().sum()

    print(f"🔎 Season NaN rows dropped: {season_nulls_before}")
    print(f"Remaining NaN rows (should be 0): {season_nulls_after}")

    dfs_subset = dfs_subset.drop(columns=['date'])

    dfs_subset['dk_salary'] = pd.to_numeric(dfs_subset['dk_salary'], errors='coerce')
    dfs_subset['fd_salary'] = pd.to_numeric(dfs_subset['fd_salary'], errors='coerce')
    dfs_subset = dfs_subset.dropna(subset=['dk_salary', 'fd_salary'])
    dfs_subset['dk_salary'] = dfs_subset['dk_salary'].astype(int)
    dfs_subset['fd_salary'] = dfs_subset['fd_salary'].astype(int)
    dfs_subset['week'] = dfs_subset['week'].astype(int)
    
    dfs_subset = dfs_subset[['season', 'week', 'player_id', 'player', 'dk_position', 'fd_position', 'dk_salary', 'fd_salary']]
    
    unique_weeks = dfs_subset['week'].nunique()
    min_week = dfs_subset['week'].min()
    max_week = dfs_subset['week'].max()
    expected_weeks = 21 if int(year) <= 2020 else 22

    print(f"\nProcessing file: {filepath}")
    print(f"Original rows in xlsx file: {original_row_count}")
    print(f"Number of players with no salary data found in xlsx: {original_row_count - len(dfs_subset)}")
    print(f"Rows in csv file after dropping NaNs: {len(dfs_subset)}")

    if original_row_count - (original_row_count - len(dfs_subset)) == len(dfs_subset):
        print("✅ Salary Validation passed: Counts match after dropping NaNs.")
        salary_validation = 'Passed'
    else:
        print("❌ Salary Validation failed: Counts mismatch!")
        salary_validation = 'Failed'

    print(f"Weeks detected: {min_week} to {max_week}")
    print(f"Total unique weeks found: {unique_weeks}")
    print("🔔 Reminder: Missing final playoff week (e.g., Super Bowl) is normal if no salary data exists.")

    if unique_weeks == expected_weeks or unique_weeks == expected_weeks - 1:
        print(f"✅ Week Validation passed: {unique_weeks} weeks found (expected {expected_weeks}).\n")
        week_validation = 'Passed'
    else:
        print(f"❌ Week Validation failed: {unique_weeks} weeks found, expected {expected_weeks}.\n")
        week_validation = 'Failed'
    
    return dfs_subset, {
        'year': int(year),
        'original_rows': original_row_count,
        'nan_rows': original_row_count - len(dfs_subset),
        'rows_after_drop': len(dfs_subset),
        'min_week': min_week,
        'max_week': max_week,
        'unique_weeks': unique_weeks,
        'expected_weeks': expected_weeks,
        'salary_validation': salary_validation,
        'week_validation': week_validation
    }

In [99]:
# ** dataframe of Fanduel and Draft Kings Salaries FOR all positions ** 

# main control flow implements the helper function 
# output: combined dataframe and csv files of all seasons fanduel draft kings player salary data 
# output: data validation checks

# Find all matching files
file_list = sorted(glob.glob('NFL-*-DFS-Dataset.xlsx'))

# Handle if no files found
if not file_list:
    print("❌ No xlsx files detected.\nPlease download and place the BigDataBall NFL DFS Excel files into the same directory as this Jupyter Notebook file.")
else:
    # Process each file
    all_years_dfs = []
    validation_records = []
    file_years = []

    for file in file_list:
        year = file.split('-')[1]  # Extract year from filename
        file_years.append(int(year))
        
        year_df, validation_info = process_single_year(file, year)

        # ** csv file ***
        # Save per-year CSV
        # year_df.to_csv(f'nfl_fd_dk_salary_{year}.csv', index=False)
        
        # Append to master list
        all_years_dfs.append(year_df)
        validation_records.append(validation_info)

    # Create validation summary DataFrame
    validation_summary_df = pd.DataFrame(validation_records)
    print("\n📋 Validation Summary:")
    display(validation_summary_df)

    # Combine all years into one big dataframe
    nfl_fd_dk_salary_combined = pd.concat(all_years_dfs, ignore_index=True)

    # Determine latest season dynamically
    current_season = max(file_years)

    # Export final combined CSV
    final_filename = f'nfl_fd_dk_salary_2017_{current_season}.csv'

    # If the file already exists, create a backup
    if os.path.exists(final_filename):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        backup_filename = f'nfl_fd_dk_salary_2017_{current_season}_backup_{timestamp}.csv'
        shutil.copy(final_filename, backup_filename)
        print(f"🛡️ Backup created: {backup_filename}")


    # *** csv file ***
    # nfl_fd_dk_salary_combined.to_csv(final_filename, index=False)

    print(f"\n✅ Final combined CSV saved as: {final_filename}")

    # Display a quick preview
    display(nfl_fd_dk_salary_combined.head())

🔎 Season NaN rows dropped: 1
Remaining NaN rows (should be 0): 0

Processing file: NFL-2017-DFS-Dataset.xlsx
Original rows in xlsx file: 7664
Number of players with no salary data found in xlsx: 799
Rows in csv file after dropping NaNs: 6865
✅ Salary Validation passed: Counts match after dropping NaNs.
Weeks detected: 1 to 20
Total unique weeks found: 20
🔔 Reminder: Missing final playoff week (e.g., Super Bowl) is normal if no salary data exists.
✅ Week Validation passed: 20 weeks found (expected 21).

🔎 Season NaN rows dropped: 1
Remaining NaN rows (should be 0): 0

Processing file: NFL-2018-DFS-Dataset.xlsx
Original rows in xlsx file: 7216
Number of players with no salary data found in xlsx: 197
Rows in csv file after dropping NaNs: 7019
✅ Salary Validation passed: Counts match after dropping NaNs.
Weeks detected: 1 to 20
Total unique weeks found: 20
🔔 Reminder: Missing final playoff week (e.g., Super Bowl) is normal if no salary data exists.
✅ Week Validation passed: 20 weeks found 

,year,original_rows,nan_rows,rows_after_drop,min_week,max_week,unique_weeks,expected_weeks,salary_validation,week_validation
0,2017,7664,799,6865,1,20,20,21,Passed,Passed
1,2018,7216,197,7019,1,20,20,21,Passed,Passed
2,2019,7252,268,6984,1,20,20,21,Passed,Passed
3,2020,7572,385,7187,1,20,20,21,Passed,Passed
4,2021,7994,1650,6344,1,21,21,22,Passed,Passed
5,2022,7872,138,7734,1,21,21,22,Passed,Passed
6,2023,7897,65,7832,1,21,21,22,Passed,Passed
7,2024,7912,136,7776,1,21,21,22,Passed,Passed



✅ Final combined CSV saved as: nfl_fd_dk_salary_2017_2024.csv


,season,week,player_id,player,dk_position,fd_position,dk_salary,fd_salary
0,2017,1,KC,Kansas City Chiefs,DST,DST,2300,4300
1,2017,1,NE,New England Patriots,DST,DST,3400,4500
2,2017,1,NaN,Alex Smith,QB,QB,5400,6900
3,2017,1,NaN,Tom Brady,QB,QB,7600,8900
4,2017,1,NaN,Anthony Sherman,RB,RB,3000,4500


In [100]:
# team abbreviations validations check
def validate_team_defenses(dfs_df, year_label, baseline_set):
    dfs_team_defense = dfs_df.loc[dfs_df['dk_position'] == 'DST', 'player_id']
    dfs_nfl_tm_abbr = set(dfs_team_defense.dropna().unique())
    
    difference_dfs = dfs_nfl_tm_abbr - baseline_set
    difference_baseline = baseline_set - dfs_nfl_tm_abbr
    
    print(f"\nValidating Team Defenses for {year_label}:")
    if not difference_dfs and not difference_baseline:
        print(f"✅ Team defenses match for {year_label}")
    else:
        print(f"❌ Team mismatch detected for {year_label}")
        print(f"Teams only in DFS: {difference_dfs}")
        print(f"Teams only in Baseline: {difference_baseline}")

# Validate each yearly dataframe
for df, info in zip(all_years_dfs, validation_records):
    validate_team_defenses(df, year_label=info['year'], baseline_set=baseline_nfl_tm_abbr)

# Validate the full combined dataframe
validate_team_defenses(nfl_fd_dk_salary_combined, year_label='Combined', baseline_set=baseline_nfl_tm_abbr)


Validating Team Defenses for 2017:
✅ Team defenses match for 2017

Validating Team Defenses for 2018:
✅ Team defenses match for 2018

Validating Team Defenses for 2019:
✅ Team defenses match for 2019

Validating Team Defenses for 2020:
✅ Team defenses match for 2020

Validating Team Defenses for 2021:
✅ Team defenses match for 2021

Validating Team Defenses for 2022:
✅ Team defenses match for 2022

Validating Team Defenses for 2023:
✅ Team defenses match for 2023

Validating Team Defenses for 2024:
✅ Team defenses match for 2024

Validating Team Defenses for Combined:
✅ Team defenses match for Combined


In [101]:
# ✅ Final QB dataframe of Fanduel and Draft Kings player Salaries
# ✅ Output: CSV file + preview

# Determine current season based on available data
current_season = nfl_fd_dk_salary_combined['season'].max()

# Extract QB players where DraftKings position is QB
qb_fd_dk_salary_2017_current_df = nfl_fd_dk_salary_combined.loc[
    nfl_fd_dk_salary_combined['dk_position'] == 'QB'
]

# *** csv file ***
qb_csv_filename = f'qb_fd_dk_salary_2017_{current_season}.csv'
# qb_fd_dk_salary_2017_current_df.to_csv(qb_csv_filename, index=False)

print(f"✅ QB DFS dataframe created and saved as {qb_csv_filename}")

# Optional: Display a quick preview
display(qb_fd_dk_salary_2017_current_df.head())


✅ QB DFS dataframe created and saved as qb_fd_dk_salary_2017_2024.csv


,season,week,player_id,player,dk_position,fd_position,dk_salary,fd_salary
2,2017,1,NaN,Alex Smith,QB,QB,5400,6900
3,2017,1,NaN,Tom Brady,QB,QB,7600,8900
30,2017,1,NaN,Carson Palmer,QB,QB,6000,7300
31,2017,1,NaN,Jake Rudock,QB,QB,4000,6000
32,2017,1,NaN,Matthew Stafford,QB,QB,6100,7600


In [102]:
## End: Build the dataframe for the DFS Fanduel and Draft Kings salary data from BigDataBall ##

In [103]:
### Begin: Data Normalization and Merge Process ###

In [104]:
# Dataframes ##
# qb_ids_ngs_pfr_stats_sorted
# qb_adv_fpts_rost_rz_merged_df_sorted_hash (not going to use this one)
# tm_reg_playoffs_game_lines_odds
# qb_fd_dk_salary_2017_current_df


## Apply Alternative for merging fantasypros ##
# qb_fp_advanced_stats_df
# qb_fpts_perct_rost_df
# qb_redzone_stats_df

In [105]:
# ✅ View columns of each FantasyPros QB dataframe prior to normalization
# ✅ Output: columns in each QB FP dataframe

print('qb_fp_advanced_stats_df')
print(qb_fp_advanced_stats_df.columns.tolist())
print()

print('qb_fpts_perct_rost_df')
print(qb_fpts_perct_rost_df.columns.tolist())
print()

print('qb_redzone_stats_df')
print(qb_redzone_stats_df.columns.tolist())

qb_fp_advanced_stats_df
['Year', 'Week', 'FantasyPros_ID', 'Player', 'Player', 'G', 'COMP', 'ATT', 'PCT', 'YDS', 'AIR', '10+ YDS', '20+ YDS', '30+ YDS', '40+ YDS', '50+ YDS', 'SACK', 'KNCK', 'HRRY', 'BLITZ', 'POOR', 'DROP', 'RZ ATT', 'RTG']

qb_fpts_perct_rost_df
['FantasyPros_ID', 'FPTS', 'ROST', 'Year', 'Week']

qb_redzone_stats_df
['FantasyPros_ID', 'COMP_rz', 'Y/A_rz', 'INT_rz', 'SACKS_rz', 'Year', 'Week']


In [106]:
# helper function for normalization methods
def normalize_dataframe(df: pd.DataFrame, type_map: dict = None) -> pd.DataFrame:
    """
    Clean and normalize dataframe:
    - Strip whitespace from object columns
    - Lowercase common ID/name fields
    - Replace special NA tokens with np.nan
    - Cast to types from provided type_map
    """
    df = df.copy()

    # Replace common string-based missing values with np.nan
    df.replace(["N/A", "NA", "-", ""], np.nan, inplace=True)

    # Strip whitespace from string/object columns
    for col in df.select_dtypes(include='object').columns:
        try:
            df[col] = df[col].astype(str).str.strip()
        except Exception as e:
            print(f"⚠️ Could not strip column '{col}': {e}")

    # Lowercase likely ID/name fields (if present)
    for key in ['player', 'player_name', 'player_id', 'fantasypros_id', 'FantasyPros_ID', 'merge_name']:
        if key in df.columns:
            df[key] = df[key].astype(str).str.lower()

    # Apply type conversions as defined in type_map
    if type_map:
        for col, dtype in type_map.items():
            if col in df.columns:
                try:
                    df[col] = df[col].astype(dtype)
                except Exception as e:
                    print(f"⚠️ Warning: could not convert column '{col}' to {dtype}. Reason: {e}")

    return df

In [107]:
def clean_percentage_columns(df: pd.DataFrame, columns: list) -> pd.DataFrame:
    df = df.copy()
    for col in columns:
        if col in df.columns:
            try:
                # Convert to string first (safe for NaNs)
                df[col] = df[col].astype(str).str.replace('%', '', regex=False)
                df[col] = df[col].astype(float)
            except Exception as e:
                print(f"⚠️ Could not clean and convert '{col}': {e}")
    return df

In [108]:
def inspect_dataframe_types(df: pd.DataFrame, name: str = "DataFrame") -> None:
    print(f"📋 Inspecting: {name}")
    print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    summary = pd.DataFrame({
        'dtype': df.dtypes,
        'na_count': df.isna().sum()
    }).sort_index()
    display(summary)

In [109]:
# normalize data types for merging 
# QB Baseline type map
type_map_qb = {
    'season': 'int32',
    'week': 'int32',
    'player_id': 'string',
    'fantasypros_id': 'string',
}

# FP Advanced Stats type map
type_map_fp_adv_qb = {
    'fantasypros_id': 'string',
    'season': 'int32',
    'week': 'int32',
    'Player': 'string',
    'G': 'int32',
    'COMP': 'float32',
    'ATT': 'float32',
    'PCT': 'float32',
    'YDS': 'float32',
    'AIR': 'float32',
    '10+ YDS': 'float32',
    '20+ YDS': 'float32',
    '30+ YDS': 'float32',
    '40+ YDS': 'float32',
    '50+ YDS': 'float32',
    'SACK': 'float32',
    'KNCK': 'float32',
    'HRRY': 'float32',
    'BLITZ': 'float32',
    'POOR': 'float32',
    'DROP': 'float32',
    'RZ ATT': 'float32',
    'RTG': 'float32',
}

# FP Fantasy Points + Rostered type map
type_map_fp_fpts_qb = {
    'fantasypros_id': 'string',
    'season': 'int32',
    'week': 'int32',
    'FPTS': 'float32',
    'ROST': 'string',
}

# FP Redzone type map
type_map_fp_rz_qb = {
    'fantasypros_id': 'string',
    'season': 'int32',
    'week': 'int32',
    'COMP_rz': 'float32',
    'Y/A_rz': 'float32',
    'INT_rz': 'int32',
    'SACKS_rz': 'int32',
}

In [110]:
# ✅ Apply the normalization function to the baseline NFL Python stats dataframe
# ✅ No output

qb_ids_ngs_pfr_stats_sorted_normalized = normalize_dataframe(
    qb_ids_ngs_pfr_stats_sorted,
    type_map_qb
)

# Fix FP ID formatting
qb_ids_ngs_pfr_stats_sorted_normalized['fantasypros_id'] = (
    qb_ids_ngs_pfr_stats_sorted_normalized['fantasypros_id']
    .astype(str).str.replace(r'\.0$', '', regex=True)
)

In [111]:
# ✅ Rename merge keys consistently BEFORE normalization to the fantasypros dataframes
# ✅ No output

qb_fp_advanced_stats_df = qb_fp_advanced_stats_df.rename(columns={
    'Year': 'season',
    'week': 'week',
    'FantasyPros_ID': 'fantasypros_id'
})

qb_fpts_perct_rost_df = qb_fpts_perct_rost_df.rename(columns={
    'Year': 'season',
    'week': 'week',
    'FantasyPros_ID': 'fantasypros_id'
})

qb_redzone_stats_df = qb_redzone_stats_df.rename(columns={
    'Year': 'season',
    'week': 'week',
    'FantasyPros_ID': 'fantasypros_id'
})

# Remove duplicate 'Player' column
qb_fp_advanced_stats_df = qb_fp_advanced_stats_df.loc[:, ~qb_fp_advanced_stats_df.columns.duplicated()]

# Clean percentages BEFORE normalization
qb_redzone_stats_df = clean_percentage_columns(
    qb_redzone_stats_df,
    ['PCT_rz']  # QB-specific column (not REC or TGT PCT like WR)
)

qb_fp_advanced_stats_df = clean_percentage_columns(
    qb_fp_advanced_stats_df,
    ['PCT']
)

In [112]:
# ✅ Apply the type_map data normalizations to the FantasyPros dataframes
# ✅ No output

qb_fp_advanced_stats_df = normalize_dataframe(
    qb_fp_advanced_stats_df,
    type_map_fp_adv_qb
)

qb_fpts_perct_rost_df = normalize_dataframe(
    qb_fpts_perct_rost_df,
    type_map_fp_fpts_qb
)

qb_redzone_stats_df = normalize_dataframe(
    qb_redzone_stats_df,
    type_map_fp_rz_qb
)

In [113]:
# ✅ Standardize all column names to lowercase after normalization

qb_fp_advanced_stats_df.columns = qb_fp_advanced_stats_df.columns.str.lower()
qb_fpts_perct_rost_df.columns = qb_fpts_perct_rost_df.columns.str.lower()
qb_redzone_stats_df.columns = qb_redzone_stats_df.columns.str.lower()

In [114]:
# ✅ View columns of each FantasyPros QB dataframe after normalization
# ✅ Output: columns in each FP dataframe

print('qb_fp_advanced_stats_df')
print(qb_fp_advanced_stats_df.columns.tolist())
print()

print('qb_fpts_perct_rost_df')
print(qb_fpts_perct_rost_df.columns.tolist())
print()

print('qb_redzone_stats_df')
print(qb_redzone_stats_df.columns.tolist())

qb_fp_advanced_stats_df
['season', 'week', 'fantasypros_id', 'player', 'g', 'comp', 'att', 'pct', 'yds', 'air', '10+ yds', '20+ yds', '30+ yds', '40+ yds', '50+ yds', 'sack', 'knck', 'hrry', 'blitz', 'poor', 'drop', 'rz att', 'rtg']

qb_fpts_perct_rost_df
['fantasypros_id', 'fpts', 'rost', 'season', 'week']

qb_redzone_stats_df
['fantasypros_id', 'comp_rz', 'y/a_rz', 'int_rz', 'sacks_rz', 'season', 'week']


In [115]:
# ✅ Merge the NFL Python Stats and FantasyPros dataframes
# ✅ No output

# Start from the normalized baseline
qb_nfl_py_fp_merged = qb_ids_ngs_pfr_stats_sorted_normalized.copy()

# Merge: Advanced stats
qb_nfl_py_fp_merged = qb_nfl_py_fp_merged.merge(
    qb_fp_advanced_stats_df,
    on=['fantasypros_id', 'season', 'week'],
    how='left',
    suffixes=('', '_fp_adv')
)

# Merge: FPTS + ROST
qb_nfl_py_fp_merged = qb_nfl_py_fp_merged.merge(
    qb_fpts_perct_rost_df,
    on=['fantasypros_id', 'season', 'week'],
    how='left',
    suffixes=('', '_fp_fpts')
)

# Merge: Redzone stats
qb_nfl_py_fp_merged = qb_nfl_py_fp_merged.merge(
    qb_redzone_stats_df,
    on=['fantasypros_id', 'season', 'week'],
    how='left',
    suffixes=('', '_fp_rz')
)

In [116]:
# ✅ Merged Dataframe: NFL Python Stats and FantasyPros

# ✅ Output: merged NFL Python + FP validation stats
# ✅ Output: merged dataframe preview and optional CSV export

print(f"📊 Final merged shape: {qb_nfl_py_fp_merged.shape}")

print(f"📦 FP Advanced Stats Columns Merged: {qb_fp_advanced_stats_df.shape[1] - 3} / {qb_fp_advanced_stats_df.shape[1] - 3}")
print(f"📦 FP Fantasy Points & ROST Columns Merged: 2 / 2")
print(f"📦 FP Redzone Columns Merged: {qb_redzone_stats_df.shape[1] - 3} / {qb_redzone_stats_df.shape[1] - 3}")

# Calculate merge coverage
adv_merged = qb_nfl_py_fp_merged[qb_fp_advanced_stats_df.columns.difference(['fantasypros_id', 'season', 'week']).tolist()].dropna(how='all').shape[0]
fpts_merged = qb_nfl_py_fp_merged[qb_fpts_perct_rost_df.columns.difference(['fantasypros_id', 'season', 'week']).tolist()].dropna(how='all').shape[0]
rz_merged = qb_nfl_py_fp_merged[qb_redzone_stats_df.columns.difference(['fantasypros_id', 'season', 'week']).tolist()].dropna(how='all').shape[0]

print(f"📈 FP Advanced Stats coverage: {adv_merged} rows ({adv_merged / len(qb_fp_advanced_stats_df) * 100:.2f}%)")
print(f"📈 FP FPTS/ROST coverage: {fpts_merged} rows ({fpts_merged / len(qb_fpts_perct_rost_df) * 100:.2f}%)")
print(f"📈 FP Redzone Stats coverage: {rz_merged} rows ({rz_merged / len(qb_redzone_stats_df) * 100:.2f}%)")

# Display first 5 rows
display(qb_nfl_py_fp_merged.head(10))

# *** csv file ***
# qb_nfl_py_fp_merged.to_csv("qb_nfl_py_fp_merged.csv", index=False)
# print("📁 Exported: qb_nfl_py_fp_merged.csv")

📊 Final merged shape: (5267, 113)
📦 FP Advanced Stats Columns Merged: 20 / 20
📦 FP Fantasy Points & ROST Columns Merged: 2 / 2
📦 FP Redzone Columns Merged: 4 / 4
📈 FP Advanced Stats coverage: 1120 rows (100.00%)
📈 FP FPTS/ROST coverage: 5003 rows (71.82%)
📈 FP Redzone Stats coverage: 4308 rows (99.84%)


,season,season_type,week,player_id,player_name,position,position_group,recent_team,mfl_id,sportradar_id,fantasypros_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,completions,attempts,passing_yards,passing_tds,rushing_yards,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota,player_display_name,team_abbr,avg_time_to_throw,avg_completed_air_yards,avg_intended_air_yards,avg_air_yards_differential,aggressiveness,max_completed_air_distance,avg_air_yards_to_sticks,pass_yards,pass_touchdowns,passer_rating,completion_percentage,expected_completion_percentage,completion_percentage_above_expectation,avg_air_distance,max_air_distance,player_gsis_id,player_first_name,player_last_name,player_short_name,game_type,team,opponent,pfr_player_name,pfr_player_id,passing_drops,passing_drop_pct,receiving_drop,receiving_drop_pct,passing_bad_throws,passing_bad_throw_pct,times_sacked,times_blitzed,times_hurried,times_hit,times_pressured,times_pressured_pct,def_times_blitzed,def_times_hurried,def_times_hitqb,player,g,comp,att,pct,yds,air,10+ yds,20+ yds,30+ yds,40+ yds,50+ yds,sack,knck,hrry,blitz,poor,drop,rz att,rtg,fpts,rost,comp_rz,y/a_rz,int_rz,sacks_rz
0,2017,REG,1,00-0023436,a.smith,QB,QB,KC,7813,2fda010a-8c62-4c07-b601-4ba03f57e6af,9012,2218.0,268.0,alexsmith/2506340,8416.0,7177.0,NaN,552642.0,SmitAl03,alex-smith-3,4306.0,3119.0,NaN,7177.0,217357.0,6739.0,NaN,Alex Smith,alex smith,28,35,368.0,4,3.0,0.0,3.0,16.0,0,0,260.0,201.0,17.0,17.560724,0,1.415385,0.300653,Alex Smith,KC,3.023914,5.651429,7.390571,-1.739143,14.285714,51.773205,-0.580857,368.0,4.0,148.571429,80.000000,70.111395,9.888605,18.876969,51.773205,00-0023436,Alexander,Smith,A.Smith,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alex Smith,1.0,28.0,35.0,80.0,368.0,182.0,10.0,3.0,2.0,2.0,2.0,3.0,1.0,6.0,7.0,2.0,0.0,2.0,149.0,31.0,5.0%,4.0,3.2,0.0,0.0
1,2017,REG,1,00-0027854,s.bradford,QB,QB,MIN,9817,cc3640b0-7560-431f-84ab-599e9dc8cac6,9678,5526.0,621.0,sambradford/497095,13197.0,23976.0,6558.0,1123599.0,BradSa00,sam-bradford-1,6452.0,5161.0,NaN,23976.0,333276.0,11527.0,NaN,Sam Bradford,sam bradford,27,32,346.0,3,-3.0,0.0,1.0,5.0,0,0,239.0,136.0,14.0,21.096104,0,1.447699,0.349325,Sam Bradford,MIN,2.568281,8.178519,7.892500,0.286019,18.750000,48.379334,-0.232500,346.0,3.0,142.968750,84.375000,59.131884,25.243116,20.657261,48.379334,00-0027854,Samuel,Bradford,S.Bradford,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sam Bradford,1.0,27.0,32.0,84.0,346.0,221.0,12.0,8.0,3.0,1.0,0.0,1.0,2.0,6.0,10.0,2.0,1.0,4.0,143.0,25.5,0.0%,6.0,5.1,0.0,0.0
2,2017,REG,1,00-0026143,m.ryan,QB,QB,ATL,9099,7e648a0b-fdc8-4661-a587-5826f2cac11b,9444,4317.0,24.0,mattryan/310,11237.0,8780.0,5371.0,420095.0,RyanMa00,matt-ryan-1,5610.0,4637.0,NaN,8780.0,216263.0,732.0,216263.0,Matt Ryan,matt ryan,21,30,321.0,1,11.0,0.0,2.0,13.0,0,0,216.0,158.0,12.0,13.515724,0,1.486111,0.257653,Matt Ryan,ATL,2.577133,7.064286,7.096667,-0.032381,26.666667,39.611728,-1.936667,321.0,1.0,116.111111,70.000000,60.505824,9.494176,19.851132,41.219875,00-0026143,Matthew,Ryan,M.Ryan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.9,0.1%,1.0,2.0,0.0,0.0
3,2017,REG,1,00-0023459,a.rodgers,QB,QB,GB,7836,0ce48193-e2fa-466e-a986-33f751add206,9001,2241.0,96.0,aaronrodgers/2506363,8439.0,7200.0,3452.0,419780.0,RodgAa00,aaron-rodgers-1,4307.0,3118.0,235.0,7200.0,213957.0,2593.0,213957.0,Aaron Rodgers,aaron rodgers,28,42,311.0,1,21.0,1.0,4.0,25.0,0,0,289.0,170.0,16.0,2.171718,0,1.076125,0.098124,Aaron Rodgers,GB,2.997238,4.837857,7.115238,-2.277381,19.047619,39.088602,-2.480000,311.0,1.0,86.507937,66.666667,63.435491,3

In [117]:
# ✅ Compare match rates using different columns
# ✅ Output: match rates show whether to join on 'recent_team' or 'team_abbr'

# Option A: merge on 'team_abbr'
merged_team_abbr = qb_nfl_py_fp_merged.merge(
    tm_reg_playoffs_game_lines_odds,
    how='left',
    left_on=['season', 'week', 'team_abbr'],
    right_on=['season', 'week', 'team_abbr']
)

# Option B: merge on 'recent_team'
merged_recent_team = qb_nfl_py_fp_merged.merge(
    tm_reg_playoffs_game_lines_odds,
    how='left',
    left_on=['season', 'week', 'recent_team'],
    right_on=['season', 'week', 'team_abbr']
)

# Count matches for each
matched_abbr = merged_team_abbr['spread'].notna().sum()
matched_recent = merged_recent_team['spread'].notna().sum()
total_rows = len(qb_nfl_py_fp_merged)

# Calculate match rates
rate_abbr = round(100 * matched_abbr / total_rows, 2)
rate_recent = round(100 * matched_recent / total_rows, 2)

print(f"Match rate using team_abbr:     {matched_abbr} / {total_rows} rows matched ({rate_abbr}%)")
print(f"Match rate using recent_team:   {matched_recent} / {total_rows} rows matched ({rate_recent}%)")

Match rate using team_abbr:     4255 / 5267 rows matched (80.79%)
Match rate using recent_team:   5257 / 5267 rows matched (99.81%)


In [118]:
# ✅ *** DataFrame of NFL Python stats, FantasyPros, and SportsOddsHistory ***
# ✅ Merge the NFL Python stats and FantasyPros dataframe with the odds / lines dataframe
# ✅ Output: merged dataframe and csv file

qb_nfl_py_fp_odds_merged = qb_nfl_py_fp_merged.merge(
    tm_reg_playoffs_game_lines_odds,
    how='left',
    left_on=['season', 'week', 'recent_team'],
    right_on=['season', 'week', 'team_abbr'],
    validate='many_to_one'
)

# ✅ Row count check
print("Row check:")
print("Before merge:", len(qb_nfl_py_fp_merged))
print("After merge: ", len(qb_nfl_py_fp_odds_merged))

# ✅ Column count check
print("\nColumn check:")
print("Before merge:", len(qb_nfl_py_fp_merged.columns))
print("After merge: ", len(qb_nfl_py_fp_odds_merged.columns))

# ✅ Display sample
print("\nSample rows:")
display(qb_nfl_py_fp_odds_merged.head())

# ✅ *** csv file ***
# qb_nfl_py_fp_odds_merged.to_csv('qb_nfl_py_fp_odds_merged.csv', index=False)

Row check:
Before merge: 5267
After merge:  5267

Column check:
Before merge: 113
After merge:  121

Sample rows:


,season,season_type,week,player_id,player_name,position,position_group,recent_team,mfl_id,sportradar_id,fantasypros_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,completions,attempts,passing_yards,passing_tds,rushing_yards,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota,player_display_name,team_abbr_x,avg_time_to_throw,avg_completed_air_yards,avg_intended_air_yards,avg_air_yards_differential,aggressiveness,max_completed_air_distance,avg_air_yards_to_sticks,pass_yards,pass_touchdowns,passer_rating,completion_percentage,expected_completion_percentage,completion_percentage_above_expectation,avg_air_distance,max_air_distance,player_gsis_id,player_first_name,player_last_name,player_short_name,game_type,team,opponent,pfr_player_name,pfr_player_id,passing_drops,passing_drop_pct,receiving_drop,receiving_drop_pct,passing_bad_throws,passing_bad_throw_pct,times_sacked,times_blitzed,times_hurried,times_hit,times_pressured,times_pressured_pct,def_times_blitzed,def_times_hurried,def_times_hitqb,player,g,comp,att,pct,yds,air,10+ yds,20+ yds,30+ yds,40+ yds,50+ yds,sack,knck,hrry,blitz,poor,drop,rz att,rtg,fpts,rost,comp_rz,y/a_rz,int_rz,sacks_rz,team_abbr_y,opponent_abbr,home,role,result,score,spread,over_under
0,2017,REG,1,00-0023436,a.smith,QB,QB,KC,7813,2fda010a-8c62-4c07-b601-4ba03f57e6af,9012,2218.0,268.0,alexsmith/2506340,8416.0,7177.0,NaN,552642.0,SmitAl03,alex-smith-3,4306.0,3119.0,NaN,7177.0,217357.0,6739.0,NaN,Alex Smith,alex smith,28,35,368.0,4,3.0,0.0,3.0,16.0,0,0,260.0,201.0,17.0,17.560724,0,1.415385,0.300653,Alex Smith,KC,3.023914,5.651429,7.390571,-1.739143,14.285714,51.773205,-0.580857,368.0,4.0,148.571429,80.000000,70.111395,9.888605,18.876969,51.773205,00-0023436,Alexander,Smith,A.Smith,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alex Smith,1.0,28.0,35.0,80.0,368.0,182.0,10.0,3.0,2.0,2.0,2.0,3.0,1.0,6.0,7.0,2.0,0.0,2.0,149.0,31.0,5.0%,4.0,3.2,0.0,0.0,KC,NE,True,Underdog,None,None,-8.0,O 47.5
1,2017,REG,1,00-0027854,s.bradford,QB,QB,MIN,9817,cc3640b0-7560-431f-84ab-599e9dc8cac6,9678,5526.0,621.0,sambradford/497095,13197.0,23976.0,6558.0,1123599.0,BradSa00,sam-bradford-1,6452.0,5161.0,NaN,23976.0,333276.0,11527.0,NaN,Sam Bradford,sam bradford,27,32,346.0,3,-3.0,0.0,1.0,5.0,0,0,239.0,136.0,14.0,21.096104,0,1.447699,0.349325,Sam Bradford,MIN,2.568281,8.178519,7.892500,0.286019,18.750000,48.379334,-0.232500,346.0,3.0,142.968750,84.375000,59.131884,25.243116,20.657261,48.379334,00-0027854,Samuel,Bradford,S.Bradford,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sam Bradford,1.0,27.0,32.0,84.0,346.0,221.0,12.0,8.0,3.0,1.0,0.0,1.0,2.0,6.0,10.0,2.0,1.0,4.0,143.0,25.5,0.0%,6.0,5.1,0.0,0.0,MIN,NO,False,Favorite,W,29-19,-3.0,O 47.5
2,2017,REG,1,00-0026143,m.ryan,QB,QB,ATL,9099,7e648a0b-fdc8-4661-a587-5826f2cac11b,9444,4317.0,24.0,mattryan/310,11237.0,8780.0,5371.0,420095.0,RyanMa00,matt-ryan-1,5610.0,4637.0,NaN,8780.0,216263.0,732.0,216263.0,Matt Ryan,matt ryan,21,30,321.0,1,11.0,0.0,2.0,13.0,0,0,216.0,158.0,12.0,13.515724,0,1.486111,0.257653,Matt Ryan,ATL,2.577133,7.064286,7.096667,-0.032381,26.666667,39.611728,-1.936667,321.0,1.0,116.111111,70.000000,60.505824,9.494176,19.851132,41.219875,00-0026143,Matthew,Ryan,M.Ryan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.9,0.1%,1.0,2.0,0.0,0.0,ATL,CHI,True,Favorite,W,23-17,-6.5,U 48
3,2017,REG,1,00-0023459,a.rodgers,QB,QB,GB,7836,0ce48193-e2fa-466e-a986-33f751add206,9001,2241.0,96.0,aaronrodgers/2506363,8439.0,7200.0,3452.0,419780.0,RodgAa00,aaron-rodgers-1,4307.0,3118.0,235.0,7200.0,213957.0,2593.0,213957.0,Aaron Rodgers,aaron rodgers,28,42,311.0,1,

In [119]:
# ✅ Create key values for merging and match tests
# ✅ Output: sample dataframe of matched rows and validation stats

# Standardize and construct merge keys safely ---
qb_fd_dk_salary_2017_current_df = qb_fd_dk_salary_2017_current_df.copy()
qb_fd_dk_salary_2017_current_df.loc[:, 'merge_name'] = (
    qb_fd_dk_salary_2017_current_df['player']
    .str.lower()
    .str.replace(r'[^a-z ]', '', regex=True)
)

qb_fd_dk_salary_2017_current_df.loc[:, 'merge_key'] = (
    qb_fd_dk_salary_2017_current_df['merge_name'] + '_' +
    qb_fd_dk_salary_2017_current_df['season'].astype(str) + '_' +
    qb_fd_dk_salary_2017_current_df['week'].astype(str)
)

qb_nfl_py_fp_odds_merged = qb_nfl_py_fp_odds_merged.copy()
qb_nfl_py_fp_odds_merged.loc[:, 'merge_key'] = (
    qb_nfl_py_fp_odds_merged['merge_name'] + '_' +
    qb_nfl_py_fp_odds_merged['season'].astype(str) + '_' +
    qb_nfl_py_fp_odds_merged['week'].astype(str)
)

# Inner join for validation ---
cols_salary = ['season', 'week', 'merge_key', 'merge_name', 'dk_salary', 'fd_salary']
test_matches = qb_nfl_py_fp_odds_merged.merge(
    qb_fd_dk_salary_2017_current_df[cols_salary],
    on=['season', 'week', 'merge_key'],
    how='inner',
    suffixes=('_main', '_salary')
)

# Match statistics
print(f"✅ Inner join matches: {len(test_matches):,}")

num_matches = test_matches['merge_key'].nunique()
total_salary_rows = qb_fd_dk_salary_2017_current_df['merge_key'].nunique()
match_percent = (num_matches / total_salary_rows) * 100

print(f"✅ Match rate: {num_matches:,} / {total_salary_rows:,} salary rows matched ({match_percent:.2f}%)")

# Display random sample for visual verification ---
sample_matches = test_matches.sample(10, random_state=42)[
    ['season', 'week', 'merge_key', 'merge_name_main', 'merge_name_salary', 'dk_salary', 'fd_salary']
]

display(sample_matches)

✅ Inner join matches: 4,959
✅ Match rate: 4,942 / 5,262 salary rows matched (93.92%)


,season,week,merge_key,merge_name_main,merge_name_salary,dk_salary,fd_salary
3135,2022,6,jacoby brissett_2022_6,jacoby brissett,jacoby brissett,5400,6600
2191,2020,12,kirk cousins_2020_12,kirk cousins,kirk cousins,6100,7100
1295,2019,3,carson wentz_2019_3,carson wentz,carson wentz,5600,7700
2902,2021,18,dak prescott_2021_18,dak prescott,dak prescott,7100,7700
168,2017,6,philip rivers_2017_6,philip rivers,philip rivers,6600,7200
1128,2018,16,lamar jackson_2018_16,lamar jackson,lamar jackson,6000,7700
4529,2024,7,patrick mahomes_2024_7,patrick mahomes,patrick mahomes,7000,8000
4280,2023,20,cj stroud_2023_20,cj stroud,cj stroud,6700,7200
4421,2024,4,andy dalton_2024_4,andy dalton,andy dalton,5300,7000
1020,2018,13,jared goff_2018_13,jared goff,jared goff,6400,8400


In [120]:
# ✅ Compare fuzzy similarity between merge_name_main and merge_name_salary
# ✅ Note: key_match should = rapidfuzz so rapidfuzz not necessary but good for verification
# ✅ Output: comparison of the two methods

test_matches['fuzzy_ratio'] = test_matches.apply(
    lambda row: fuzz.token_sort_ratio(row['merge_name_main'], row['merge_name_salary']),
    axis=1
)

# Check exact vs fuzzy
exact_matches = (test_matches['merge_name_main'] == test_matches['merge_name_salary']).sum()
fuzzy_matches_90plus = (test_matches['fuzzy_ratio'] >= 90).sum()

print(f"🎯 Exact name matches: {exact_matches:,} / {len(test_matches):,}")
print(f"👌 Fuzzy matches ≥ 90: {fuzzy_matches_90plus:,} / {len(test_matches):,}")

🎯 Exact name matches: 4,959 / 4,959
👌 Fuzzy matches ≥ 90: 4,959 / 4,959


In [121]:
# ✅ *** Final Merge ***
# ✅ Output: complete merge of NFL Python stats, FantasyPros, SportsOddsHistory, and BigDataBall salary data

# Merge salary data into baseline
qb_nfl_py_fp_odds_salary_merged = qb_nfl_py_fp_odds_merged.merge(
    qb_fd_dk_salary_2017_current_df[['season', 'week', 'merge_key', 'dk_salary', 'fd_salary']],
    on=['season', 'week', 'merge_key'],
    how='left'
)

# Summary match stats
num_salary_matches = qb_nfl_py_fp_odds_salary_merged['dk_salary'].notna().sum()
total_rows = len(qb_nfl_py_fp_odds_salary_merged)
print(f"💰 Salaries merged into: {num_salary_matches:,} / {total_rows:,} rows ({(num_salary_matches / total_rows) * 100:.2f}%)")

# Optional: remove merge_key if you're done with it
# qb_nfl_py_fp_odds_salary_merged.drop(columns=['merge_key'], inplace=True)

# ** csv file **
# qb_nfl_py_fp_odds_salary_merged.to_csv("qb_nfl_py_fp_odds_salary_merged.csv", index=False)
# print("📁 Saved as: qb_nfl_py_fp_odds_salary_merged.csv")


💰 Salaries merged into: 4,959 / 5,284 rows (93.85%)


In [122]:
### End: Data Normalization and Merge Process ###

In [123]:
### Begin: Feature Engineering ###

In [124]:
# ✅ Feature Engineering – Drop Unnecessary Columns
# ✅ Output: updated dataframe and csv file with modified columns

# List of columns to drop
cols_to_drop = [
    'player_id', 'player_name', 'position_group', 'mfl_id', 'sportradar_id',
    'fantasypros_id', 'pff_id', 'sleeper_id', 'nfl_id', 'espn_id', 'yahoo_id',
    'fleaflicker_id', 'cbs_id', 'pfr_id', 'cfbref_id', 'rotowire_id',
    'rotoworld_id', 'ktc_id', 'stats_id', 'stats_global_id', 'fantasy_data_id',
    'swish_id', 'merge_name', 'player_gsis_id', 'player_first_name',
    'player_last_name', 'player_short_name', 'game_type', 'team', 'opponent',
    'pfr_player_name', 'pfr_player_id', 'player', 'team_abbr_y', 'merge_key'
]

# Initial state
initial_shape = qb_nfl_py_fp_odds_salary_merged.shape
initial_col_count = initial_shape[1]

print("Before modification:")
print(f"Shape: {initial_shape}")
print(f"Total Columns: {initial_col_count}")

# Drop only existing columns
cols_existing = [col for col in cols_to_drop if col in qb_nfl_py_fp_odds_salary_merged.columns]
qb_nfl_py_fp_odds_salary_merged_mod_cols = qb_nfl_py_fp_odds_salary_merged.drop(columns=cols_existing)

# Post-drop validation
final_shape = qb_nfl_py_fp_odds_salary_merged_mod_cols.shape
final_col_count = final_shape[1]
dropped_count = len(cols_existing)
expected_final_col_count = initial_col_count - dropped_count

print("\nAfter modification:")
print(f"Shape: {final_shape}")
print(f"Total Columns After Drop: {final_col_count}")
print(f"Number of Columns Dropped: {dropped_count}")
print(f"Expected Final Column Count: {expected_final_col_count}")
print(f"Column Count Validation Passed: {final_col_count == expected_final_col_count}")

# Confirm none of the dropped columns remain
all_dropped = all(col not in qb_nfl_py_fp_odds_salary_merged_mod_cols.columns for col in cols_to_drop)
print(f"All Specified Columns Successfully Dropped: {all_dropped}")

# ** csv file **
output_csv_path = "qb_nfl_py_fp_odds_salary_merged_mod_cols.csv"
# qb_nfl_py_fp_odds_salary_merged_mod_cols.to_csv(output_csv_path, index=False)
# print(f"\nModified dataframe saved to {output_csv_path}")


Before modification:
Shape: (5284, 124)
Total Columns: 124

After modification:
Shape: (5284, 89)
Total Columns After Drop: 89
Number of Columns Dropped: 35
Expected Final Column Count: 89
Column Count Validation Passed: True
All Specified Columns Successfully Dropped: True


In [125]:
# ✅ Reorder columns

# Define new column order
priority_cols = [
    'season', 'season_type', 'week', 'name', 'position', 'recent_team',
    'fpts', 'dk_salary', 'fd_salary', 'rost',
    'opponent_abbr', 'home', 'role', 'result', 'score', 'spread', 'over_under'
]

# Capture original state
original_shape = qb_nfl_py_fp_odds_salary_merged_mod_cols.shape
original_columns = qb_nfl_py_fp_odds_salary_merged_mod_cols.columns.tolist()

print("Original Shape:", original_shape)
print("Original First 20 Columns:", original_columns[:20])

# Build final column order
remaining_cols = [col for col in original_columns if col not in priority_cols]
final_col_order = priority_cols + remaining_cols

# Reorder the columns
qb_nfl_py_fp_odds_salary_merged_mod_cols = qb_nfl_py_fp_odds_salary_merged_mod_cols[final_col_order]

# Post-reorder validation
new_shape = qb_nfl_py_fp_odds_salary_merged_mod_cols.shape
new_columns = qb_nfl_py_fp_odds_salary_merged_mod_cols.columns.tolist()

print("\nNew Shape:", new_shape)
print("New First 20 Columns:", new_columns[:20])

# Validation Checks
print("\n✅ Shape Integrity:", original_shape == new_shape)
print("✅ Column Count Matches:", len(original_columns) == len(new_columns))
print("✅ All Columns Preserved:", set(original_columns) == set(new_columns))

# ** csv file **
output_reordered_csv = "qb_nfl_py_fp_odds_salary_merged_mod_cols_reordered.csv"
# qb_nfl_py_fp_odds_salary_merged_mod_cols.to_csv(output_reordered_csv, index=False)
# print(f"\nReordered dataframe saved to {output_reordered_csv}")

Original Shape: (5284, 89)
Original First 20 Columns: ['season', 'season_type', 'week', 'position', 'recent_team', 'name', 'completions', 'attempts', 'passing_yards', 'passing_tds', 'rushing_yards', 'interceptions', 'sacks', 'sack_yards', 'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards', 'passing_yards_after_catch', 'passing_first_downs', 'passing_epa']

New Shape: (5284, 89)
New First 20 Columns: ['season', 'season_type', 'week', 'name', 'position', 'recent_team', 'fpts', 'dk_salary', 'fd_salary', 'rost', 'opponent_abbr', 'home', 'role', 'result', 'score', 'spread', 'over_under', 'completions', 'attempts', 'passing_yards']

✅ Shape Integrity: True
✅ Column Count Matches: True
✅ All Columns Preserved: True


In [126]:
# ✅ Apply Rolling Averages and Aggregates - 3, 5, and 7 week averages
# ✅ Output: updated dataframe with aggregates (optional csv file)

# Start from sorted copy of the main DF
qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg = (
    qb_nfl_py_fp_odds_salary_merged_mod_cols
    .sort_values(by=['name', 'season', 'week'])
    .reset_index(drop=True)
)

# Feature map: full column -> short prefix
feature_map = {
    'attempts': 'att',
    'completions': 'comp',
    'passing_yards': 'pass_yds',
    'passing_tds': 'pass_tds',
    'interceptions': 'ints',
    'fpts': 'fpts'
}

windows = [3, 5, 7]

# Apply rolling averages and lag features
for full_col, short in feature_map.items():
    grouped = qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg.groupby(['name', 'season'])

    # Rolling averages
    for window in windows:
        col_name = f"{short}_{window}wk_avg"
        qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg[col_name] = grouped[full_col].apply(
            lambda x: x.rolling(window=window, min_periods=window).mean().shift(1)
        ).reset_index(drop=True)

    # Lag feature (1-game lookback)
    lag_col = f"{short}_lag_1"
    qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg[lag_col] = grouped[full_col].shift(1).reset_index(drop=True)

# Final integrity check
print("✅ Final shape:", qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg.shape)

# ** csv file **
# qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg.to_csv(
#     "qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg.csv",
#     index=False,
#     float_format="%.2f"
# )
# print("📤 Exported to: qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg.csv")

✅ Final shape: (5284, 113)


In [127]:
# ✅ Validation tests of aggregates
# ✅ Output: there should be no aggregates prior to week 4

# Structural Check — No aggregates in first 3 weeks of a season
def check_early_aggregates(df, cols, earliest_week=4):
    early = df[df['week'] < earliest_week]
    violations = early[cols].notna().sum()
    print("🚨 Aggregates present before week", earliest_week)
    print(violations[violations > 0])

# Boundary Check — Rolling aggregates must reset per season
def check_season_boundaries(df, col_prefix):
    errors = []
    for short in col_prefix:
        col_name = f'{short}_3wk_avg'
        season_transitions = df.groupby(['name'])['season'].diff().fillna(0)
        cross_season_rows = df[season_transitions != 0]
        if cross_season_rows[col_name].notna().any():
            errors.append(col_name)
    if errors:
        print("❌ Rolling values leaked across seasons:", errors)
    else:
        print("✅ No cross-season leakage detected.")

# Shape check
def check_shape(df, expected_cols_added):
    print("✅ Final shape:", df.shape)
    print("✅ Final columns:", df.columns[-expected_cols_added:])

# === Apply Checks ===
rolling_cols = [f"{short}_{w}wk_avg" for short in ['att', 'comp', 'pass_yds', 'pass_tds', 'ints', 'fpts'] for w in [3, 5, 7]]
lag_cols = [f"{short}_lag_1" for short in ['att', 'comp', 'pass_yds', 'pass_tds', 'ints', 'fpts']]

check_early_aggregates(qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg, rolling_cols)
check_season_boundaries(qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg, ['att', 'comp', 'pass_yds', 'pass_tds', 'ints', 'fpts'])
check_shape(qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg, expected_cols_added=len(rolling_cols + lag_cols))

🚨 Aggregates present before week 4
Series([], dtype: int64)
✅ No cross-season leakage detected.
✅ Final shape: (5284, 113)
✅ Final columns: Index(['att_3wk_avg', 'att_5wk_avg', 'att_7wk_avg', 'att_lag_1', 'comp_3wk_avg', 'comp_5wk_avg', 'comp_7wk_avg', 'comp_lag_1', 'pass_yds_3wk_avg', 'pass_yds_5wk_avg', 'pass_yds_7wk_avg', 'pass_yds_lag_1', 'pass_tds_3wk_avg', 'pass_tds_5wk_avg', 'pass_tds_7wk_avg', 'pass_tds_lag_1', 'ints_3wk_avg', 'ints_5wk_avg', 'ints_7wk_avg', 'ints_lag_1', 'fpts_3wk_avg', 'fpts_5wk_avg', 'fpts_7wk_avg', 'fpts_lag_1'], dtype='object')


In [128]:
# ✅ Add Trend Features (deltas) - recent performance over/under (3wk, 5wk, 7wk) averages
# ✅ Output: updated dataframe with deltas (optional csv file)

# New dataframe
qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend = qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg.copy()

# Column map
feature_map = {
    'attempts': 'att',
    'completions': 'comp',
    'passing_yards': 'pass_yds',
    'passing_tds': 'pass_tds',
    'interceptions': 'ints',
    'fpts': 'fpts'
}

windows = [3, 5, 7]

# Create delta (deviation from trend) features
for full_col, short in feature_map.items():
    for window in windows:
        avg_col = f"{short}_{window}wk_avg"
        delta_col = f"{short}_{window}wk_delta"
        qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend[delta_col] = (
            qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend[full_col] -
            qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend[avg_col]
        )

# Summary and export
delta_cols = [f"{short}_{w}wk_delta" for short in feature_map.values() for w in windows]
print("✅ Added delta columns:", delta_cols)
print("✅ Final shape:", qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend.shape)

# ** csv file **
# qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend.to_csv(
#     "qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend.csv",
#     index=False,
#     float_format="%.2f"
# )
# print("📤 Exported to: qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend.csv")

✅ Added delta columns: ['att_3wk_delta', 'att_5wk_delta', 'att_7wk_delta', 'comp_3wk_delta', 'comp_5wk_delta', 'comp_7wk_delta', 'pass_yds_3wk_delta', 'pass_yds_5wk_delta', 'pass_yds_7wk_delta', 'pass_tds_3wk_delta', 'pass_tds_5wk_delta', 'pass_tds_7wk_delta', 'ints_3wk_delta', 'ints_5wk_delta', 'ints_7wk_delta', 'fpts_3wk_delta', 'fpts_5wk_delta', 'fpts_7wk_delta']
✅ Final shape: (5284, 131)


In [129]:
# ✅ Create boolean columns
# ✅ Output: updated dataframe with booleans (optional csv file)

# Start from the previous trend-enhanced dataframe
qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool = qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend.copy()

# Define boolean columns as 0/1 integers
qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['pass_yds_ge_250'] = (qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['passing_yards'] >= 250).astype(int)
qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['pass_yds_ge_300'] = (qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['passing_yards'] >= 300).astype(int)

qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['tds_ge_3'] = (qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['passing_tds'] >= 3).astype(int)
qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['tds_ge_4'] = (qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['passing_tds'] >= 4).astype(int)

qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['zero_int'] = (qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['interceptions'] == 0).astype(int)
qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['multi_int'] = (qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['interceptions'] >= 2).astype(int)

qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['boom_week'] = (qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['fpts'] >= 25).astype(int)
qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['bust_week'] = (qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['fpts'] < 10).astype(int)

# If 'home' is already boolean, convert to int
qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['is_home_game'] = qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool['home'].astype(int)

# Final shape and column check
print("✅ Final shape:", qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool.shape)
print("✅ New boolean columns added.")

# ** csv file **
# qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool.to_csv(
#     "qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool.csv",
#     index=False,
#     float_format="%.2f"
# )
# print("📤 Exported to: qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool.csv")

✅ Final shape: (5284, 140)
✅ New boolean columns added.


In [130]:
# ✅ *** Dataframe: this can be used as the final dataframe but the filename is long ***

# ✅ Split the over / under column into two columns: o_u and total
# ✅ Output: updated dataframe with o_u and total columns (optional csv file) 

# Copy from final boolean-enriched dataframe
qb_nfl_py_fp_odds_salary_features = qb_nfl_py_fp_odds_salary_merged_mod_cols_sort_agg_trend_bool.copy()

# Extract 'O' or 'U' and map to "over"/"under"
qb_nfl_py_fp_odds_salary_features['O_U'] = (
    qb_nfl_py_fp_odds_salary_features['over_under']
    .str[0]
    .map({'O': 'over', 'U': 'under'})
)

# Extract the numeric total (handles int or float)
qb_nfl_py_fp_odds_salary_features['Total'] = (
    qb_nfl_py_fp_odds_salary_features['over_under']
    .str.extract(r'(\d+\.?\d*)')[0]
    .astype(float)
)

# Validation
print("✅ Final shape:", qb_nfl_py_fp_odds_salary_features.shape)
print("✅ Sample 'O_U' values:", qb_nfl_py_fp_odds_salary_features['O_U'].unique())
print("✅ Sample 'Total' values:", qb_nfl_py_fp_odds_salary_features['Total'].dropna().unique()[:5])

# ** csv file **
# qb_nfl_py_fp_odds_salary_features.to_csv(
#     "qb_nfl_py_fp_odds_salary_features.csv",
#     index=False,
#     float_format="%.2f"
# )
# print("📤 Exported to: qb_nfl_py_fp_odds_salary_features.csv")

✅ Final shape: (5284, 142)
✅ Sample 'O_U' values: ['under' 'over' nan]
✅ Sample 'Total' values: [40.5 42.5 44.  42.  51. ]


In [131]:
### End: Feature Engineering ###

In [132]:
### Begin: Final Dataframe ###

In [133]:
### *** Final Dataframe *** ###

# Rename final dataframe
qb_nfl_df_sorted_final = qb_nfl_py_fp_odds_salary_features.copy()

# ** csv file **
qb_nfl_df_sorted_final.to_csv(
    "qb_nfl_df_sorted_final.csv",
    index=False,
    float_format="%.2f"
)

print("✅ Final dataframe saved as 'qb_nfl_df_sorted_final.csv'")
print("✅ Final shape:", qb_nfl_df_sorted_final.shape)

✅ Final dataframe saved as 'qb_nfl_df_sorted_final.csv'
✅ Final shape: (5284, 142)


In [134]:
### End: Final Dataframe ###

In [135]:
### Begin: Final Dataframe Summary Statistics ###

In [136]:
# ✅ Final Dataframe Summary Statistics
# ✅ Output: csv output of the dataframe summary statistics on data types and missing values

# Total columns and datatypes
total_cols = qb_nfl_df_sorted_final.shape[1]
dtypes_summary = qb_nfl_df_sorted_final.dtypes.value_counts().sort_values(ascending=False)
dtypes_percent = (dtypes_summary / total_cols * 100).round(2)

print("🧠 Data Type Distribution (by count and %):")
for dtype, count in dtypes_summary.items():
    print(f"{str(dtype):15} {count:>3} columns  ({dtypes_percent[dtype]:>5.1f}%)")

# Object-type columns
object_cols = qb_nfl_df_sorted_final.select_dtypes(include='object').columns.tolist()
print(f"\n🚨 Object-type columns found ({len(object_cols)} total / {total_cols} columns):")
print(object_cols)

# Missing value summary by count and % of rows
row_count = len(qb_nfl_df_sorted_final)
na_counts = qb_nfl_df_sorted_final.isna().sum()
na_percent = (na_counts / row_count * 100).round(2)
na_summary = pd.DataFrame({'Missing': na_counts, 'Percent': na_percent})
na_summary = na_summary[na_summary['Missing'] > 0].sort_values(by='Percent', ascending=False)

print(f"\n⚠️ Missing Value Summary (non-zero only) — Top {len(na_summary)} columns:")
display(na_summary)

# ** csv file **
na_summary.to_csv("qb_df_final_summary_stats.csv")

🧠 Data Type Distribution (by count and %):
float64          76 columns  ( 53.5%)
float32          35 columns  ( 24.6%)
object           12 columns  (  8.4%)
int64             9 columns  (  6.3%)
int32             8 columns  (  5.6%)
string            1 columns  (  0.7%)
bool              1 columns  (  0.7%)

🚨 Object-type columns found (12 total / 142 columns):
['season_type', 'name', 'position', 'recent_team', 'opponent_abbr', 'role', 'result', 'score', 'over_under', 'player_display_name', 'team_abbr_x', 'O_U']

⚠️ Missing Value Summary (non-zero only) — Top 106 columns:


,Missing,Percent
receiving_drop_pct,5284,100.00
def_times_blitzed,5284,100.00
def_times_hurried,5284,100.00
def_times_hitqb,5284,100.00
receiving_drop,5284,100.00
...,...,...
rost,264,5.00
pacr,253,4.79
passing_epa,230,4.35
O_U,50,0.95


In [137]:
### End: Final Dataframe Summary Statistics ###